### README

***I seperate to run each part in this notebook due to my computational resource limitation. It start with initial training then restart kernel, hyperparameter tuning then restart kernel, training experiment then restart kernel, and evaluation experiment then restart kernel by every process start with the same preprocess pipeline.*** 

***Remark that I have runned all of the pipeline on few number of sample, epoch, and iteration before upload on GitHub.***

***In this notebook I have set them back to values for experiment in the paper (sample, epoch, and number of experiment iteration).***

***I have runned all of the pipeline with values for experiment in the paper (sample, epoch, and number of experiment iteration), and it's ok.***

***Any things that I can help you, please tell me.***

### DeepSeek Model on HuggingFace

***[Original Model Link!](https://huggingface.co/deepseek-ai/deepseek-moe-16b-chat/tree/main)***

***[Model Architecture Link!](https://huggingface.co/deepseek-ai/deepseek-moe-16b-chat/blob/main/modeling_deepseek.py)***

***[Model Configuration Link!](https://huggingface.co/deepseek-ai/deepseek-moe-16b-chat/blob/main/configuration_deepseek.py)***

# Download Dataset

In [1]:
!git clone https://github.com/lexmen318/MultiWOZ-coref.git

Cloning into 'MultiWOZ-coref'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 60 (delta 19), reused 4 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (60/60), 29.40 MiB | 44.40 MiB/s, done.
Resolving deltas: 100% (19/19), done.


# Install and import necessary libraries

In [3]:
%%capture
!pip install pyspellchecker
!pip install nlpaug
!pip install dateparser
!pip install CurrencyConverter
!pip install dateparser word2number
!pip install contractions
!pip install gensim
!pip install GPUtil
!pip install rouge_score
!pip install nvidia-ml-py3
!pip install pynvml
!pip install rouge-score
!pip install rouge
!pip install optuna
!pip install evaluate
!nvidia-smi

In [1]:
import sys
import platform
import subprocess
import importlib.util
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
import json
import time
import pandas as pd
from typing import Dict, List, Tuple
import shutil
import math
from tensorflow.keras import mixed_precision
from tensorflow.keras.preprocessing.sequence import pad_sequences
from datasets import Dataset
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, trainers
import psutil
import GPUtil
import gc
import re
from collections import Counter
from math import sqrt, log, exp
from tqdm.auto import tqdm
import optuna
import contractions
import dateparser
from word2number import w2n
from sklearn.model_selection import train_test_split
from datasets import Dataset, Features, Value, Sequence, ClassLabel
import pickle
from sklearn.preprocessing import LabelEncoder
from gensim.models import Word2Vec, KeyedVectors
from nltk.tokenize import word_tokenize
from collections import defaultdict
import nltk
from typing import List, Dict, Any
from sklearn.preprocessing import MultiLabelBinarizer
from tensorflow.keras.preprocessing.text import Tokenizer
from nltk.tokenize import sent_tokenize
import warnings
from tensorflow import keras
from tensorflow.keras import models, optimizers, losses, metrics, callbacks
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, accuracy_score
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
import uuid
from tokenizers import Tokenizer, models, pre_tokenizers, trainers, decoders
import logging
from tensorflow.keras.preprocessing.text import Tokenizer as KerasTokenizer
from collections import Counter, defaultdict
import zipfile
import random
import nvidia_smi

nltk.download('punkt', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)

# Set the TF logging level to ERROR or higher
# 1 = INFO, 2 = WARNING, 3 = ERROR
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
tf.get_logger().setLevel(logging.ERROR)

os.environ["TOKENIZERS_PARALLELISM"] = "false"

2025-10-17 12:34:32.578812: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-10-17 12:34:32.578918: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-10-17 12:34:32.580368: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-17 12:34:32.588742: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-17 12:34:33.359410: W tensorflow/compiler/tf2

# Check system information

In [2]:
def get_os_info():
    """Retrieves operating system information."""
    os_name = platform.system()
    os_version = platform.release()
    return os_name, os_version

def get_python_info():
    """Retrieves Python version information."""
    python_version = sys.version
    return python_version

def get_cpu_info():
    """Retrieves CPU information."""
    cpu_name = platform.processor()
    cpu_cores = psutil.cpu_count(logical=False) 
    cpu_threads = psutil.cpu_count(logical=True) 
    return cpu_name, cpu_cores, cpu_threads

def get_memory_info():
    """Retrieves memory information."""
    mem = psutil.virtual_memory()
    total_memory_gb = mem.total / (1024 ** 3) 
    available_memory_gb = mem.available / (1024 ** 3)
    return total_memory_gb, available_memory_gb

def get_gpu_info():
    """Retrieves GPU information using nvidia-smi command."""
    try:
        subprocess.run(["nvidia-smi", "-h"], check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        result = subprocess.run(["nvidia-smi", "--query-gpu=name,driver_version", "--format=csv,noheader"],
                                 capture_output=True, text=True, check=True)
        cuda_result = subprocess.run(["nvidia-smi"], capture_output=True, text=True, check=True)
        cuda_version_line = next((line for line in cuda_result.stdout.split('\n') if "CUDA Version" in line), None)
        cuda_version = cuda_version_line.split(":")[-1].strip() if cuda_version_line else "N/A"

        gpus = []
        for line in result.stdout.strip().split('\n'):
            name, driver_version = line.split(', ')
            gpus.append({'name': name, 'driver_version': driver_version})
        return gpus, cuda_version
    except (subprocess.CalledProcessError, FileNotFoundError):
        return "N/A", "N/A"

def check_deepspeed():
    """Checks for DeepSpeed installation and version."""
    deepspeed_spec = importlib.util.find_spec("deepspeed")
    if deepspeed_spec is not None:
        try:
            import deepspeed
            return deepspeed.__version__
        except ImportError:
            return "Installed, but version cannot be determined"
    else:
        return "Not Installed"

def check_tensorflow():
    """Checks for TensorFlow installation and version."""
    try:
        import tensorflow as tf
        return tf.__version__
    except ImportError:
        return "Not Installed"

os_name, os_version = get_os_info()
python_version = get_python_info()
cpu_name, cpu_cores, cpu_threads = get_cpu_info()
total_memory_gb, available_memory_gb = get_memory_info()
gpus, cuda_version = get_gpu_info()
deepspeed_version = check_deepspeed()
tensorflow_version = check_tensorflow()

print("--- System Information ---")
print(f"Operating System: {os_name} {os_version}")
print(f"Python Version: {python_version}")
print("--- CPU Information ---")
print(f"CPU: {cpu_name}")
print(f"Physical Cores: {cpu_cores}")
print(f"Logical Threads: {cpu_threads}")
print("--- Memory Information ---")
print(f"Total Memory: {total_memory_gb:.2f} GB")
print(f"Available Memory: {available_memory_gb:.2f} GB")
print("--- GPU Information ---")
if gpus == "N/A":
    print("GPU: N/A (No NVIDIA GPU or nvidia-smi not found)")
    print(f"CUDA Version: N/A")
else:
    for i, gpu in enumerate(gpus):
        print(f"GPU {i + 1}: {gpu['name']}, Driver Version: {gpu['driver_version']}")
    print(f"CUDA Version: {cuda_version}")
print("--- DeepSpeed Information ---")
print(f"DeepSpeed Version: {deepspeed_version}")
print("--- TensorFlow Information ---")
print(f"TensorFlow Version: {tensorflow_version}")

[2025-10-17 12:34:37,556] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
--- System Information ---
Operating System: Linux 5.19.0-45-generic
Python Version: 3.11.7 (main, Dec  8 2023, 18:56:58) [GCC 11.4.0]
--- CPU Information ---
CPU: x86_64
Physical Cores: 8
Logical Threads: 8
--- Memory Information ---
Total Memory: 44.08 GB
Available Memory: 40.67 GB
--- GPU Information ---
GPU 1: NVIDIA RTX A4000, Driver Version: 550.144.03
CUDA Version: 12.4     |
--- DeepSpeed Information ---
DeepSpeed Version: 0.10.3
--- TensorFlow Information ---
TensorFlow Version: 2.15.0


## Load dataset

In [45]:
# Path for downloaded zip file of MultiWOZ2_3 dataset
zip_file_path = "MultiWOZ-coref/MultiWOZ2_3.zip"

# Removes the '.zip' extension from the file path to get a directory name.
destination_dir = "MultiWOZ-coref-extract"

# Define expected final data paths
dataset_dir = os.path.join(destination_dir, "MultiWOZ2_3")
data_file = os.path.join(dataset_dir, "data.json")
dialogue_acts_file = os.path.join(dataset_dir, "dialogue_acts.json")
ontology_file = os.path.join(dataset_dir, "ontology.json")

# Check if all required files are present (zip and data files)
required_files = {
    'zip': zip_file_path,
    'data': data_file,
    'ontology': ontology_file,
    'dialogue_acts': dialogue_acts_file
}

print(f"Creating directory if it doesn't exist: {destination_dir}...")
os.makedirs(destination_dir, exist_ok=True)

# Extract if any data files are missing
data_files = {k: v for k, v in required_files.items() if k != 'zip'}
if any(not os.path.exists(path) for path in data_files.values()):
    print(f"Extracting {zip_file_path} to {destination_dir}")
    # Use 'with' statement for clean memory
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(destination_dir)
    print("Extraction successful!")
else:
    print("Data files found in destination")

# Load files assuming they exist and are correctly formatted
print("Load dataset...")
loaded_data = {
    key: json.load(open(path, 'r')) if os.path.exists(path) else {}
    for key, path in data_files.items()
}

# Assign loaded data or None if loading failed (empty dict)
data = loaded_data.get('data')
ontology = loaded_data.get('ontology')
dialogue_acts = loaded_data.get('dialogue_acts')
print("Load dataset sucessful")

Creating directory if it doesn't exist: MultiWOZ-coref-extract...
Data files found in destination
Load dataset...
Load dataset sucessful


In [46]:
# Get domain for dialogue
def get_primary_domain(dialogue):
    """Extract the primary domain from dialogue structure."""
    if "new_goal" in dialogue and dialogue["new_goal"]:
        for domain in dialogue["new_goal"]:
            if domain != "user_action":
                return domain
    
    if "goal" in dialogue and dialogue["goal"]:
        for domain in dialogue["goal"]:
            if domain != "user_action" and isinstance(dialogue["goal"][domain], dict):
                return domain
            
    domains = set()
    for turn in dialogue.get("log", []):
        metadata = turn.get("metadata", {})
        for domain in metadata:
            if domain != "user_action" and isinstance(metadata[domain], dict):
                domains.add(domain)
                
    return domains.pop() if domains else "unknown"

def normalize_dialogue_id(dialogue_id):
    """Normalize dialogue ID to match between data and dialogue acts."""
    if dialogue_id.endswith('.json'):
        return dialogue_id[:-5]
    return dialogue_id

# Process and merge data with dialogue acts
raw_data = []

for dialogue_id, dialogue in data.items():
    normalized_id = normalize_dialogue_id(dialogue_id)
    acts = dialogue_acts.get(normalized_id, {})
    processed_acts = {}
    for turn_key, turn_acts in acts.items():
        if turn_acts == "No Annotation":
            processed_acts[str(turn_key)] = {}
        else:
            processed_acts[str(turn_key)] = turn_acts
    
    raw_data.append({
        "dialogue_id": dialogue_id,
        "dialogue": dialogue,
        "domain": get_primary_domain(dialogue),
        "dialogue_acts": processed_acts,
        "ontology": ontology
    })

# Visualize Raw Dataset

In [47]:
def format_dialogue_acts(acts_dict):
    """Format dialogue acts for better display."""
    if not acts_dict:
        return "No acts available"
    formatted = []
    for act_type, slots in acts_dict.items():
        slot_strs = []
        for slot in slots:
            if ':' in slot:
                key, value = slot.split(':', 1)
                slot_strs.append(f"{key}={value}")
            else:
                slot_strs.append(str(slot))
        formatted.append(f"{act_type}({', '.join(slot_strs)})")
    return "; ".join(formatted)

def randomly_check_first_dialogue_turns_with_acts_and_ontology(split_name, split_data, num_samples=2, num_turns=4):
    """
    Randomly checks a few dialogues and displays the dialogue text, acts, and ontology.
    """
    print(f"\nRandomly Checking {num_samples} Dialogues (First {num_turns} Turns) from {split_name} ")
    if not split_data:
        print(f"No data in {split_name} split.")
        return
    
    if not isinstance(split_data, (list, tuple)):
        split_data = list(split_data)
    
    if len(split_data) == 0:
        print(f"No data in {split_name} split after conversion.")
        return
        
    random_dialogues = random.sample(split_data, min(num_samples, len(split_data)))
    
    for dialogue_item in random_dialogues:
        dialogue_id = dialogue_item["dialogue_id"]
        raw_dialogue_object = dialogue_item["dialogue"]
        if isinstance(raw_dialogue_object, list):
            turns = raw_dialogue_object
        elif isinstance(raw_dialogue_object, dict):
            turns = raw_dialogue_object.get("log", [])
        else:
            print(f"Warning: Unexpected dialogue format for dialogue ID {dialogue_id}. Skipping.")
            continue
            
        print(f"\nDialogue ID: {dialogue_id}")
        print(f"Domain: {dialogue_item.get('domain', 'unknown')}")
        print(f"First {min(num_turns, len(turns))} turns:")
        
        dialogue_acts_for_dialogue = dialogue_item.get("dialogue_acts", {})
        domain = dialogue_item.get('domain', 'unknown')
        
        domain_ontology = {
            slot: values
            for slot, values in dialogue_item.get("ontology", {}).items()
            if slot.startswith(f"{domain}-")
        }

        print(f"    Ontology for domain '{domain}': {json.dumps(domain_ontology, indent=2)}")

        for i in range(min(num_turns, len(turns))):
            turn = turns[i]
            # In the raw data, the 'text' key is at this level
            text_content = turn.get('text', '')
            # use the loop index for the turn id
            turn_idx = i                                      
            turn_key = str(turn_idx)

            turn_type = "User" if turn_idx % 2 == 0 else "System"
            print(f"  Turn {turn_idx} ({turn_type}):")
            
            # Use the text content from the turn dictionary
            print(f"    Text: {text_content}")
            
            turn_acts_data = dialogue_acts_for_dialogue.get(turn_key, {})
            print(f"    Dialogue Acts: {format_dialogue_acts(turn_acts_data)}")
            
randomly_check_first_dialogue_turns_with_acts_and_ontology("Raw Data", raw_data)


Randomly Checking 2 Dialogues (First 4 Turns) from Raw Data 

Dialogue ID: SNG01661.json
Domain: train
First 4 turns:
    Ontology for domain 'train': {
  "train-book people": [
    "2",
    "8",
    "7",
    "15",
    "5",
    "1",
    "dontcare",
    "6",
    "3",
    "9",
    "4",
    "10"
  ],
  "train-arriveBy": [
    "19:57",
    "12:00",
    "19:30",
    "11:54",
    "05:30",
    "16:30",
    "18:30",
    "06:45",
    "19:58",
    "23:27",
    "16:06",
    "07:35",
    "08:30",
    "12:07",
    "10:30",
    "12:06",
    "15:45",
    "18:35",
    "18:07",
    "10:32",
    "06:43",
    "8",
    "20:38",
    "18:23",
    "20:06",
    "09:01",
    "20:08",
    "19:54",
    "10:15",
    "09:30",
    "11:52",
    "18:00",
    "07:30",
    "08:07",
    "16:00",
    "08:56",
    "17:23",
    "14:07",
    "10:08",
    "11:00",
    "08:44",
    "21:45",
    "15:00",
    "10:54",
    "17:51",
    "03:00",
    "10:45",
    "16:08",
    "13:06",
    "12:45",
    "20:45",
    "22:01",
    "1

# Split data into train and test dataset

In [48]:
# Split the data
train_data_raw, test_data_raw = train_test_split(
    raw_data,
    test_size=0.2,
    stratify=[d["domain"] for d in raw_data],
    random_state=42
)

# Get sample of the dataset function
def get_samples(dataset, num_samples):
    if not dataset:
        print("Warning: Dataset is empty. Returning an empty list.")
        return []
    if not isinstance(dataset, (list, tuple)):
        try:
            dataset_list = list(dataset)
        except TypeError:
            print("Error: Dataset could not be converted to a list for sampling.")
            return []
    else:
        dataset_list = list(dataset)
    domain_counts = {}
    
    for item in dataset_list:
        if 'domain' in item:
            domain = item['domain']
            domain_counts[domain] = domain_counts.get(domain, 0) + 1
        else:
            print(f"Warning: Item {item} does not have a 'domain' key. Skipping for stratification.")
    total_items = len(dataset_list)
    if total_items == 0:
        print("Warning: Dataset has no items. Returning an empty list.")
        return []
    actual_samples_to_take = min(num_samples, total_items)
    if actual_samples_to_take < num_samples:
        print(f"Warning: Dataset size ({total_items}) is less than requested samples ({num_samples}). Taking {actual_samples_to_take} samples.")
    sampled_data = []
    random.seed(42)
    items_by_domain = {domain: [] for domain in domain_counts}
    
    for item in dataset_list:
        if 'domain' in item:
            items_by_domain[item['domain']].append(item)
            
    for domain, count in domain_counts.items():
        domain_proportion = count / total_items
        num_domain_samples = round(actual_samples_to_take * domain_proportion)
        num_domain_samples = min(num_domain_samples, len(items_by_domain[domain]))
        sampled_domain_items = random.sample(items_by_domain[domain], num_domain_samples)
        sampled_data.extend(sampled_domain_items)
        
    if len(sampled_data) < actual_samples_to_take:
        remaining_items = [item for item in dataset_list if item not in sampled_data]
        num_to_add = actual_samples_to_take - len(sampled_data)
        if remaining_items and num_to_add > 0:
            sampled_data.extend(random.sample(remaining_items, min(num_to_add, len(remaining_items))))
    elif len(sampled_data) > actual_samples_to_take:
        num_to_remove = len(sampled_data) - actual_samples_to_take
        for _ in range(num_to_remove):
            sampled_data.pop(random.randrange(len(sampled_data)))
            
    random.shuffle(sampled_data)
    
    return sampled_data

train_data_raw = get_samples(train_data_raw, 1600)
print(f"Sampled train_dataset size: {len(train_data_raw)}")

test_data_raw = get_samples(test_data_raw, 400)
print(f"Sampled test_dataset size: {len(test_data_raw)}")

Sampled train_dataset size: 1600
Sampled test_dataset size: 400


In [49]:
def format_dialogue_acts(acts_dict):
    """Format dialogue acts for better display."""
    if not acts_dict:
        return "No acts available"
    formatted = []
    for act_type, slots in acts_dict.items():
        slot_strs = []
        for slot in slots:
            if ':' in slot:
                key, value = slot.split(':', 1)
                slot_strs.append(f"{key}={value}")
            else:
                slot_strs.append(str(slot))
        formatted.append(f"{act_type}({', '.join(slot_strs)})")
    return "; ".join(formatted)

def randomly_check_first_dialogue_turns_with_acts_and_ontology(split_name, split_data, num_samples=2, num_turns=4):
    """
    Randomly checks a few dialogues and displays the dialogue text, acts, and ontology.
    """
    print(f"\n Randomly Checking {num_samples} Dialogues (First {num_turns} Turns) from {split_name} ")
    if not split_data:
        print(f"No data in {split_name} split.")
        return
    
    if not isinstance(split_data, (list, tuple)):
        split_data = list(split_data)
    
    if len(split_data) == 0:
        print(f"No data in {split_name} split after conversion.")
        return
        
    random_dialogues = random.sample(split_data, min(num_samples, len(split_data)))
    
    for dialogue_item in random_dialogues:
        dialogue_id = dialogue_item["dialogue_id"]
        raw_dialogue_object = dialogue_item["dialogue"]
        if isinstance(raw_dialogue_object, list):
            turns = raw_dialogue_object
        elif isinstance(raw_dialogue_object, dict):
            turns = raw_dialogue_object.get("log", [])
        else:
            print(f"Warning: Unexpected dialogue format for dialogue ID {dialogue_id}. Skipping.")
            continue
            
        print(f"\nDialogue ID: {dialogue_id}")
        print(f"Domain: {dialogue_item.get('domain', 'unknown')}")
        print(f"First {min(num_turns, len(turns))} turns:")
        
        dialogue_acts_for_dialogue = dialogue_item.get("dialogue_acts", {})
        domain = dialogue_item.get('domain', 'unknown')
        
        domain_ontology = {
            slot: values
            for slot, values in dialogue_item.get("ontology", {}).items()
            if slot.startswith(f"{domain}-")
        }

        print(f"    Ontology for domain '{domain}': {json.dumps(domain_ontology, indent=2)}")

        for i in range(min(num_turns, len(turns))):
            turn = turns[i]
            # In the raw data, the 'text' key is at this level
            text_content = turn.get('text', '')
            # use the loop index for the turn id
            turn_idx = i 
            turn_key = str(turn_idx)

            turn_type = "User" if turn_idx % 2 == 0 else "System"
            print(f"  Turn {turn_idx} ({turn_type}):")
            
            # Use the text content from the turn dictionary
            print(f"    Text: {text_content}")
            
            turn_acts_data = dialogue_acts_for_dialogue.get(turn_key, {})
            print(f"    Dialogue Acts: {format_dialogue_acts(turn_acts_data)}")


randomly_check_first_dialogue_turns_with_acts_and_ontology("Train", train_data_raw)
randomly_check_first_dialogue_turns_with_acts_and_ontology("Test", test_data_raw)


 Randomly Checking 2 Dialogues (First 4 Turns) from Train 

Dialogue ID: SNG0238.json
Domain: hospital
First 4 turns:
    Ontology for domain 'hospital': {
  "hospital-department": [
    "antenatal",
    "childrens surgical and medicine",
    "haematology and haematological oncology",
    "infectious diseases",
    "medical decisions unit",
    "teenage cancer trust unit",
    "transitional care",
    "john farman intensive care unit",
    "urology",
    "intermediate dependancy area",
    "respiratory medicine",
    "neonatal unit",
    "inpatient occupational therapy",
    "gynaecology",
    "medicine for the elderly",
    "neurology",
    "trauma high dependency unit",
    "cardiology",
    "cardiology and coronary care unit",
    "hepatobillary and gastrointestinal surgery regional referral centre",
    "acute medical assessment unit",
    "gastroenterology",
    "oral and maxillofacial surgery and ent",
    "hepatology",
    "acute medicine for the elderly",
    "transplant high 

# Data Visualization

## Display Dialougue

In [50]:
def randomly_check_first_dialogue_turns(split_name, split_data, num_samples=2, num_turns=2):
    print(f"\n Randomly Checking {num_samples} Dialogues (First {num_turns} Turns) from {split_name} ")
    if not split_data:
        print(f"No data in {split_name} split.")
        return
    
    if not isinstance(split_data, (list, tuple)):
        split_data = list(split_data)
    
    if len(split_data) == 0:
        print(f"No data in {split_name} split after conversion.")
        return
        
    random_dialogues = random.sample(split_data, min(num_samples, len(split_data)))
    
    for dialogue in random_dialogues:
        dialogue_id = dialogue["dialogue_id"]
        turns = dialogue["dialogue"].get("log", []) 
        
        print(f"\nDialogue ID: {dialogue_id}")
        print(f"Domain: {dialogue.get('domain', 'unknown')}")
        print(f"First {min(num_turns, len(turns))} turns:")
        
        for i in range(min(num_turns, len(turns))):
            turn = turns[i]
            print(f"  Turn {turn['turn_id']}:")
            print(f"    User: {turn['text']}" if turn['turn_id'] % 2 == 0 else f"    System: {turn['text']}")


randomly_check_first_dialogue_turns("Train", train_data_raw)
randomly_check_first_dialogue_turns("Test", test_data_raw)


 Randomly Checking 2 Dialogues (First 2 Turns) from Train 

Dialogue ID: SNG0462.json
Domain: restaurant
First 2 turns:
  Turn 0:
    User: Hi ! Can you give me some information on the Royal Spice restaurant ?
  Turn 1:
    System: Of course ! It 's a cheap Indian restaurant in the north at Victoria Avenue Chesteron cb41eh . The phone number is 01733553355 . May I help with anything else ?

Dialogue ID: MUL0244.json
Domain: train
First 2 turns:
  Turn 0:
    User: Yes , I would like to book a train that is leaving Monday , and is going to Cambridge .
  Turn 1:
    System: There are 202 trains to cambridge on monday . Can you tell me your departure station and the time you 'd like to travel ?

 Randomly Checking 2 Dialogues (First 2 Turns) from Test 

Dialogue ID: WOZ20573.json
Domain: restaurant
First 2 turns:
  Turn 0:
    User: Hello . Can you help me find the address of an inexpensive restaurant in the south part of town ?
  Turn 1:
    System: There are two restaurants that are in

In [51]:
def format_dialogue_acts(acts_dict):
    """Format dialogue acts for better display."""
    if not acts_dict:
        return "No acts available"
    formatted = []
    for act_type, slots in acts_dict.items():
        slot_strs = []
        for slot in slots:
            if ':' in slot:
                key, value = slot.split(':', 1)
                slot_strs.append(f"{key}={value}")
            else:
                slot_strs.append(str(slot))
        formatted.append(f"{act_type}({', '.join(slot_strs)})")
    return "; ".join(formatted)

def randomly_check_first_dialogue_turns_with_acts_and_ontology(split_name, split_data, num_samples=2, num_turns=4):
    """
    Randomly checks a few dialogues and displays the dialogue text, acts, and ontology.
    """
    print(f"\n Randomly Checking {num_samples} Dialogues (First {num_turns} Turns) from {split_name} ")
    if not split_data:
        print(f"No data in {split_name} split.")
        return
    
    if not isinstance(split_data, (list, tuple)):
        split_data = list(split_data)
    
    if len(split_data) == 0:
        print(f"No data in {split_name} split after conversion.")
        return
        
    random_dialogues = random.sample(split_data, min(num_samples, len(split_data)))
    
    for dialogue_item in random_dialogues:
        dialogue_id = dialogue_item["dialogue_id"]
        raw_dialogue_object = dialogue_item["dialogue"]
        if isinstance(raw_dialogue_object, list):
            turns = raw_dialogue_object
        elif isinstance(raw_dialogue_object, dict):
            turns = raw_dialogue_object.get("log", [])
        else:
            print(f"Warning: Unexpected dialogue format for dialogue ID {dialogue_id}. Skipping.")
            continue
            
        print(f"\nDialogue ID: {dialogue_id}")
        print(f"Domain: {dialogue_item.get('domain', 'unknown')}")
        print(f"First {min(num_turns, len(turns))} turns:")
        
        # Dialogue acts and ontology are still accessed from the top-level item
        dialogue_acts_for_dialogue = dialogue_item.get("dialogue_acts", {})
        domain = dialogue_item.get('domain', 'unknown')
        
        domain_ontology = {
            slot: values
            for slot, values in dialogue_item.get("ontology", {}).items()
            if slot.startswith(f"{domain}-")
        }

        print(f"    Ontology for domain '{domain}': {json.dumps(domain_ontology, indent=2)}")

        for i in range(min(num_turns, len(turns))):
            turn = turns[i]
            # In the raw data, the 'text' key is at this level
            text_content = turn.get('text', '')
            # use the loop index for the turn id
            turn_idx = i
            turn_key = str(turn_idx)

            turn_type = "User" if turn_idx % 2 == 0 else "System"
            print(f"  Turn {turn_idx} ({turn_type}):")
            
            # Use the text content from the turn dictionary
            print(f"    Text: {text_content}")
            
            turn_acts_data = dialogue_acts_for_dialogue.get(turn_key, {})
            print(f"    Dialogue Acts: {format_dialogue_acts(turn_acts_data)}")

randomly_check_first_dialogue_turns_with_acts_and_ontology("Train", train_data_raw)
randomly_check_first_dialogue_turns_with_acts_and_ontology("Test", test_data_raw)


 Randomly Checking 2 Dialogues (First 4 Turns) from Train 

Dialogue ID: SNG1089.json
Domain: attraction
First 4 turns:
    Ontology for domain 'attraction': {
  "attraction-area": [
    "north",
    "centre|west",
    "east",
    "south",
    "centre",
    "west",
    "dontcare"
  ],
  "attraction-name": [
    "cambridge arts theater",
    "scudamores punting co",
    "trinity college",
    "history of science museum",
    "old schools",
    "christs college|saint catharines",
    "aylesbray lodge guest house",
    "scott polar",
    "churchills college",
    "museum of archaelogy and anthropology",
    "cambridge artworks",
    "pembroke college",
    "milton country park",
    "regency gallery",
    "cambridge contemporary art museum",
    "little saint marys church",
    "sheeps green and lammas land park fen causeway",
    "the junction",
    "college",
    "peoples portraits exhibition at girton college",
    "st catharines college",
    "kings college|hughes hall",
    "cambrid

## Counts number of words in each sentence in each dialogue

In [52]:
def get_longest_sentence_word_length(datasets: Dict[str, List[Dict[str, Any]]]) -> Dict[str, int]:
    longest_sentence_lengths = {}
    
    for dataset_name, dataset in datasets.items():
        max_words_in_sentence = 0
        
        for dialogue_entry in dataset:
            # The 'dialogue' entry is a dictionary containing a 'log' key, which is a list of turns.
            dialogue_log = dialogue_entry.get('dialogue', {}).get('log', [])
            
            for turn in dialogue_log:
                # The turn itself is a dictionary with a 'text' key.
                sentence_text = turn.get('text', '').strip()
                
                if sentence_text:
                    words = sentence_text.split()
                    max_words_in_sentence = max(max_words_in_sentence, len(words))
        
        longest_sentence_lengths[dataset_name] = max_words_in_sentence
        print(f"Longest sentence in {dataset_name}: {max_words_in_sentence} words")
    
    return longest_sentence_lengths

all_datasets = {
    'train_dataset': train_data_raw,
    'test_dataset': test_data_raw,
}

longest_lengths = get_longest_sentence_word_length(all_datasets)

print("\nSummary of longest sentence lengths:")
for dataset_name, length in longest_lengths.items():
    print(f"{dataset_name}: {length} words")

Longest sentence in train_dataset: 67 words
Longest sentence in test_dataset: 56 words

Summary of longest sentence lengths:
train_dataset: 67 words
test_dataset: 56 words


## Check the number of turns in dialogue

In [53]:
def find_longest_turn_id(all_datasets):
    max_turn_id = 0

    for dataset_name, dialogues_dataset in all_datasets.items():
        print(f"Checking dataset: {dataset_name}")
        for dialogue in dialogues_dataset:
            # The list of turns is under the 'log' key inside the 'dialogue' dictionary
            dialogue_turns = dialogue.get('dialogue', {}).get('log', [])
            
            for turn in dialogue_turns:
                # The 'turn_id' is an integer,if turn_id is an integer
                if 'turn_id' in turn:
                    max_turn_id = max(max_turn_id, turn['turn_id'])
    
    return max_turn_id

all_datasets = {
    'train_dataset': train_data_raw,
    'test_dataset': test_data_raw
}

longest_turn_id = find_longest_turn_id(all_datasets)
print(f"\nThe longest turn_id found across all datasets is: {longest_turn_id}")

Checking dataset: train_dataset
Checking dataset: test_dataset

The longest turn_id found across all datasets is: 43


# Data Preprocessing

## Text cleaning and Normalization

In [54]:
# --- Preprocess code ---
def normalize_general_text(text):
    """
    Performs general text normalization on a string.
    Expands contractions, removes extra spaces, and standardizes punctuation.
    """
    # Use the contractions library to expand contractions
    text = contractions.fix(text, slang=False)
    
    # Add lower letter for text normalization
    # text = text.lower()
    
    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Add a space before punctuation to aid tokenization later
    text = re.sub(r'([.?!,])', r' \1', text)
    
    return text

def normalize_time_in_text(text):
    def time_repl_hhmm_ampm(match):
        try:
            parsed_time = dateparser.parse(match.group(0))
            return parsed_time.strftime('%H:%M') if parsed_time else match.group(0)
        except:
            return match.group(0)
    text = re.sub(r'\b(\d{1,2}:\d{2})\s*(am|pm)\b', time_repl_hhmm_ampm, text, flags=re.IGNORECASE)

    def time_repl_hh_ampm(match):
        try:
            parsed_time = dateparser.parse(match.group(0))
            return parsed_time.strftime('%H:%M') if parsed_time else match.group(0)
        except:
            return match.group(0)
    text = re.sub(r'\b(\d{1,2})\s*(am|pm)\b', time_repl_hh_ampm, text, flags=re.IGNORECASE)
    text = re.sub(r'\bnoon\b', '12:00', text, flags=re.IGNORECASE)
    text = re.sub(r'\bmidday\b', '12:00', text, flags=re.IGNORECASE)
    text = re.sub(r'\bmidnight\b', '00:00', text, flags=re.IGNORECASE)
    return text

CURRENCY_SYMBOLS_MAP = {
    '£': 'GBP',
    '$': 'USD',
    '€': 'EUR',
    '¥': 'JPY',
    '₹': 'INR',
}

CURRENCY_KEYWORDS_MAP = {
    'pound': 'GBP', 'pounds': 'GBP', 'quid': 'GBP', 'gbp': 'GBP', 'sterling': 'GBP',
    'dollar': 'USD', 'dollars': 'USD', 'buck': 'USD', 'bucks': 'USD', 'usd': 'USD',
    'euro': 'EUR', 'euros': 'EUR', 'eur': 'EUR',
    'yen': 'JPY', 'jpy': 'JPY',
    'rupee': 'INR', 'rupees': 'INR', 'inr': 'INR',
}

ALL_CURRENCY_KEYWORDS_SORTED = sorted(CURRENCY_KEYWORDS_MAP.keys(), key=len, reverse=True)
ALL_CURRENCY_SYMBOLS_SORTED = sorted(CURRENCY_SYMBOLS_MAP.keys(), key=len, reverse=True)

NUMBER_WORDS_FOR_REGEX = (
    r"zero|one|two|three|four|five|six|seven|eight|nine|ten|eleven|twelve|"
    r"thirteen|fourteen|fifteen|sixteen|seventeen|eighteen|nineteen|twenty|"
    r"thirty|forty|fifty|sixty|seventy|eighty|ninety|hundred|thousand|million|billion"
)

COMPLEX_NUMBER_WORDS_PATTERN = rf"\b((?:{NUMBER_WORDS_FOR_REGEX})(?:\s+(?:and\s+)?(?:{NUMBER_WORDS_FOR_REGEX}))*)\b"

def normalize_currency_in_text(text):
    """
    Normalizes currency expressions in a string.
    Examples: "twenty pounds" -> "20 GBP", "£20" -> "20 GBP"
    """
    def word_num_keyword_replacer(match):
        num_word_str = match.group(1)
        currency_key_str = match.group(2).lower()
        try:
            # Use w2n to convert word to number
            num_val = w2n.word_to_num(num_word_str)
            currency_code = CURRENCY_KEYWORDS_MAP.get(currency_key_str, CURRENCY_KEYWORDS_MAP.get(currency_key_str.rstrip('s'), currency_key_str.upper()))
            return f"{num_val} {currency_code.upper()}"
        except ValueError:
            return match.group(0)

    currency_keywords_regex_part = "|".join([re.escape(k) for k in ALL_CURRENCY_KEYWORDS_SORTED])
    pattern_word_num_then_keyword = rf"({COMPLEX_NUMBER_WORDS_PATTERN})\s+({currency_keywords_regex_part})\b"
    text = re.sub(pattern_word_num_then_keyword, word_num_keyword_replacer, text, flags=re.IGNORECASE)

    for symbol in ALL_CURRENCY_SYMBOLS_SORTED:
        code = CURRENCY_SYMBOLS_MAP[symbol]
        text = re.sub(rf'{re.escape(symbol)}\s*(\d+\.?\d*)', rf'\1 {code}', text)
        text = re.sub(rf'(\d+\.?\d*)\s*{re.escape(symbol)}', rf'\1 {code}', text)

    for keyword in ALL_CURRENCY_KEYWORDS_SORTED:
        code = CURRENCY_KEYWORDS_MAP[keyword]
        text = re.sub(rf'(\d+\.?\d*)\s+{re.escape(keyword)}\b', rf'\1 {code}', text, flags=re.IGNORECASE)
        text = re.sub(rf'\b{re.escape(keyword)}\s+(\d+\.?\d*)', rf'\1 {code}', text, flags=re.IGNORECASE)

    all_target_codes = set(CURRENCY_SYMBOLS_MAP.values()) | set(CURRENCY_KEYWORDS_MAP.values())
    for code_val in all_target_codes:
        text = re.sub(rf'\b{re.escape(code_val)}\b', code_val.upper(), text, flags=re.IGNORECASE)

    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'(\d)([A-Z]{3}\b)', r'\1 \2', text)
    text = re.sub(r'(\b[A-Z]{3})(\d)', r'\1 \2', text)
    text = re.sub(r'\s+', ' ', text).strip()

    return text

def flatten_dialogue(dialogue_data):
    flattened_turns = []
    turns = dialogue_data.get("log", [])
    for turn_idx, turn in enumerate(turns):
        raw_text = turn.get("text", "")
        # Apply general text normalization first
        normalized_text = normalize_general_text(raw_text)
        # Apply existing normalizations to the already normalized text
        normalized_text = normalize_time_in_text(normalized_text)
        normalized_text = normalize_currency_in_text(normalized_text)

        is_user_turn = turn_idx % 2 == 0
        
        # Extract state information from MultiWOZ structure
        metadata = turn.get("metadata", {})
        dialogue_state = {}
        
        # Extract state information from metadata
        for domain, domain_data in metadata.items():
            if isinstance(domain_data, dict) and "semi" in domain_data:
                # This is the standard MultiWOZ state structure
                semi_data = domain_data["semi"]
                # Only add if there's actual state data
                if semi_data:  
                    dialogue_state[domain] = semi_data
        
        turn_data = {
            "turn_id": turn_idx,
            "user_utterance": normalized_text if is_user_turn else "",
            "system_response": "" if is_user_turn else normalized_text,
            # Use the properly extracted state
            "state": dialogue_state 
        }
        flattened_turns.append(turn_data)
        
    return flattened_turns

def normalize_dialogue_acts(dialogue_acts, dialogue_data):
    normalized_acts = {}
    
    for dialogue_id, turns_data in dialogue_data.items():
        normalized_id = normalize_dialogue_id(dialogue_id)
        acts = dialogue_acts.get(normalized_id, {})
        turns = turns_data.get("log", [])
        normalized_turn_acts = {}
        
        for turn_idx in range(len(turns)):
            turn_key = str(turn_idx)
            turn_acts_raw = acts.get(turn_key, {})
            processed_acts = {}
            
            if isinstance(turn_acts_raw, dict):
                for act_type, act_values in turn_acts_raw.items():
                    # Convert act_type to lowercase
                    lower_act_type = act_type.lower()
                    if isinstance(act_values, dict):
                        # Convert keys and values to lowercase
                        processed_acts[lower_act_type] = [
                            f"{k.lower()}:{v.lower()}" if isinstance(k, str) and isinstance(v, str) else f"{k}:{v}" for k, v in act_values.items()
                        ]
                    elif isinstance(act_values, list):
                        flat_values = []
                        for item in act_values:
                            if isinstance(item, list):
                                # Convert items in inner lists to lowercase
                                flat_values.extend([str(x).lower() if isinstance(x, str) else str(x) for x in item])
                            else:
                                # Convert item to lowercase
                                flat_values.append(str(item).lower() if isinstance(item, str) else str(item))
                        processed_acts[lower_act_type] = flat_values
                    else:
                        # Convert single string values to lowercase
                        processed_acts[lower_act_type] = [str(act_values).lower() if isinstance(act_values, str) else str(act_values)]
            
            normalized_turn_acts[turn_key] = processed_acts
        normalized_acts[dialogue_id] = normalized_turn_acts
        
    return normalized_acts

def process_data(raw_data_subset, dialogue_acts, ontology):
    
    dialogue_data_map = {d["dialogue_id"]: d["dialogue"] for d in raw_data_subset}
    normalized_dialogue_acts = normalize_dialogue_acts(dialogue_acts, dialogue_data_map)
    processed_data = []
    
    for item in raw_data_subset:
        dialogue_id = item["dialogue_id"]
        dialogue = item["dialogue"]
        domain = item["domain"]
        domain_ontology = {
            slot: values
            for slot, values in ontology.items()
            if slot.startswith(f"{domain}-")
        }
        
        processed_data.append({
            "dialogue_id": dialogue_id,
            "domain": domain,
            "dialogue": flatten_dialogue(dialogue),
            "dialogue_acts": normalized_dialogue_acts.get(dialogue_id, {}),
            "ontology": domain_ontology
        })
    
    return processed_data

def format_dialogue_acts(acts_dict):
    """Format dialogue acts for better display."""
    if not acts_dict:
        return "No acts available"
    
    formatted = []
    for act_type, slots in acts_dict.items():
        slot_strs = []
        for slot in slots:
            if ':' in slot:
                key, value = slot.split(':', 1)
                slot_strs.append(f"{key}={value}")
            else:
                slot_strs.append(str(slot))
        formatted.append(f"{act_type}({', '.join(slot_strs)})")
        
    return "; ".join(formatted)

def randomly_check_first_dialogue_turns_with_acts_and_ontology(split_name, split_data, num_samples=2, num_turns=4):
    """
    Randomly checks a few dialogues and displays the dialogue text, acts, and ontology.
    """
    print(f"\n Randomly Checking {num_samples} Dialogues (First {num_turns} Turns) from {split_name} ")
    
    if not split_data:
        print(f"No data in {split_name} split.")
        return
    
    if not isinstance(split_data, (list, tuple)):
        split_data = list(split_data)
    
    if len(split_data) == 0:
        print(f"No data in {split_name} split after conversion.")
        return
        
    random_dialogues = random.sample(split_data, min(num_samples, len(split_data)))
    
    for dialogue_item in random_dialogues:
        dialogue_id = dialogue_item["dialogue_id"]
        
        # Check if the 'dialogue' key contains the flattened turns list or the raw object.
        raw_dialogue_object = dialogue_item["dialogue"]
        
        if isinstance(raw_dialogue_object, list):
            # This is already processed data
            turns = raw_dialogue_object
        elif isinstance(raw_dialogue_object, dict):
            # This is raw data, so we need to get the 'log' key.
            turns = raw_dialogue_object.get("log", [])
        else:
            print(f"Warning: Unexpected dialogue format for dialogue ID {dialogue_id}. Skipping.")
            continue
            
        print(f"\nDialogue ID: {dialogue_id}")
        print(f"Domain: {dialogue_item.get('domain', 'unknown')}")
        
        # Dialogue acts and ontology
        dialogue_acts_for_dialogue = dialogue_item.get("dialogue_acts", {})
        domain = dialogue_item.get('domain', 'unknown')
        
        domain_ontology = {
            slot: values
            for slot, values in dialogue_item.get("ontology", {}).items()
            if slot.startswith(f"{domain}-")
        }

        print(f"    Ontology for domain '{domain}': {json.dumps(domain_ontology, indent=2)}")
        print(f"First {min(num_turns, len(turns))} turns:")
        
        for i in range(min(num_turns, len(turns))):
            turn = turns[i]
            turn_idx = turn.get("turn_id") if isinstance(turn, dict) and "turn_id" in turn else i
            
            # Use the text content from the turn dictionary
            if "user_utterance" in turn and turn["user_utterance"]:
                text_content = turn["user_utterance"]
                turn_type = "User"
            elif "system_response" in turn and turn["system_response"]:
                text_content = turn["system_response"]
                turn_type = "System"
            else:
                # Fallback for raw data
                text_content = turn.get('text', '')
                turn_type = "User" if turn_idx % 2 == 0 else "System"

            print(f"  Turn {turn_idx} ({turn_type}):")
            
            print(f"    Text: {text_content}")
            
            # Get the acts from the processed data's dialogue_acts dictionary
            turn_acts_data = dialogue_acts_for_dialogue.get(str(turn_idx), {})
            print(f"    Dialogue Acts: {format_dialogue_acts(turn_acts_data)}")
            
            # Only print state if the dictionary is not empty
            state_data = turn.get("state", {})
            if state_data:
                print(f"    State: {state_data}")

# Call process_data on both datasets
print("\nPreprocessing Datasets")
processed_train_data = process_data(train_data_raw, dialogue_acts, ontology)
processed_test_data = process_data(test_data_raw, dialogue_acts, ontology)

print(f"Preprocessed train_data size: {len(processed_train_data)}")
print(f"Preprocessed test_data size: {len(processed_test_data)}")

# Display preprocessed dataset
randomly_check_first_dialogue_turns_with_acts_and_ontology("Processed Train", processed_train_data)
randomly_check_first_dialogue_turns_with_acts_and_ontology("Processed Test", processed_test_data) 


Preprocessing Datasets
Preprocessed train_data size: 1600
Preprocessed test_data size: 400

 Randomly Checking 2 Dialogues (First 4 Turns) from Processed Train 

Dialogue ID: SNG0668.json
Domain: restaurant
    Ontology for domain 'restaurant': {
  "restaurant-book day": [
    "sunday|thursday",
    "wednesday",
    "saturday",
    "sunday",
    "saturday|thursday",
    "friday",
    "dontcare",
    "monday",
    "thursday",
    "tuesday"
  ],
  "restaurant-book people": [
    "2",
    "7",
    "8",
    "5",
    "1",
    "6",
    "4|7",
    "3",
    "4"
  ],
  "restaurant-book time": [
    "12:00",
    "13:30",
    "10:00",
    "21:00",
    "7pm",
    "1345",
    "19:30",
    "21:45",
    "15:00",
    "01:00",
    "16:30",
    "14:15",
    "03:00",
    "10:45",
    "18:30",
    "15:15",
    "17:00|16:00",
    "4pm",
    "15:30|16:30",
    "9",
    "14:00",
    "17:45",
    "09:45",
    "1715",
    "16:45",
    "12:45",
    "1330",
    "20:45",
    "11:00",
    "12:15",
    "10:30",
  

# Feature Engineering

## Split features and target labels

In [55]:
# Special tokens
PAD_TOKEN = "<pad>"
BOS_TOKEN = "<bos>"
EOS_TOKEN = "<eos>"
SEP_TOKEN = "<sep>"

def prepare_deepseek_dataset(dialogues):
    """
    Prepare dataset in DeepSeekMoE format by separating each question and answer
    pair in a dialogue.
    """
    sequences = []
    
    for dialogue in dialogues:
        for turn in dialogue:
            user_utterance = turn.get("user_utterance")
            system_response = turn.get("system_response")

            if user_utterance:
                # Format the user's question with the BOS token
                question_text = f"{BOS_TOKEN} {user_utterance}"
                sequences.append({"text": question_text})
            
            if system_response:
                # Format the system's response with the EOS token
                answer_text = f"{system_response} {EOS_TOKEN}"
                sequences.append({"text": answer_text})
                
    return sequences

print("\nPreparing DeepSeekMoE Dataset ")
train_sequences = prepare_deepseek_dataset([d["dialogue"] for d in processed_train_data])
test_sequences = prepare_deepseek_dataset([d["dialogue"] for d in processed_test_data])

# Save text for evaluation
with open("test_evaluation_texts.txt", "w", encoding="utf-8") as f:
    for seq in test_sequences:
        f.write(seq["text"] + "\n")

print(f"Train sequences: {len(train_sequences)}")
print(f"Test sequences: {len(test_sequences)}")

# Display examples
print("\nExample Sequences ")
for i in range(2):
    print(f"Example {i+1}: {train_sequences[i]['text'][:300]}...")


Preparing DeepSeekMoE Dataset 
Train sequences: 22140
Test sequences: 5588

Example Sequences 
Example 1: <bos> Yes I need a cheap restaurant in the Cambridge area on the north side of town . What do you suggest ?...
Example 2: thank you I will go to royal spice . <eos>...


## Convert to Hugging Face datasets

In [56]:
# Convert to Hugging Face datasets
train_dataset = Dataset.from_list(train_sequences)
test_dataset = Dataset.from_list(test_sequences)

# Check dataset after covert
train_dataset

Dataset({
    features: ['text'],
    num_rows: 22140
})

## Tokenization and Vocabulary Building

In [57]:
# Special tokens
BOS_TOKEN = "<bos>"
EOS_TOKEN = "<eos>"
PAD_TOKEN = "<pad>"
SEP_TOKEN = "<sep>"

# Max length Configurations
max_seq_len = 70
vocab_size = 100000

# Tokenization
print("\nTokenization ")
tokenizer = Tokenizer(models.BPE())
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=True)
tokenizer.decoder = decoders.ByteLevel()

# Create corpus for tokenizer training
corpus_file = "deepseek_corpus.txt"
with open(corpus_file, "w", encoding="utf-8") as f:
    for item in train_sequences:
        f.write(item["text"] + "\n")

# Train tokenizer
trainer = trainers.BpeTrainer(
    special_tokens=[BOS_TOKEN, EOS_TOKEN, PAD_TOKEN, SEP_TOKEN],
    # Number of vocaburary in corpus
    vocab_size=vocab_size
)

tokenizer.train([corpus_file], trainer=trainer)
os.remove(corpus_file)

# Configure tokenizer
tokenizer.enable_truncation(max_length=max_seq_len)

tokenizer.enable_padding(
    direction="right",
    pad_id=tokenizer.token_to_id(PAD_TOKEN),
    pad_token=PAD_TOKEN
)

print(f"Vocabulary size: {tokenizer.get_vocab_size()}")
PAD_TOKEN_ID = tokenizer.token_to_id(PAD_TOKEN)

# Prepared dataset
def tokenize_function(examples):
    """Tokenize text sequences"""
    encoded = tokenizer.encode_batch(examples["text"])
    return {
        "input_ids": [e.ids for e in encoded],
        "attention_mask": [e.attention_mask for e in encoded]
    }

# Tokenize datasets
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

# Save tokenizer
tokenizer.save("deepseek_tokenizer.json")


Tokenization 



Vocabulary size: 11480


Map:   0%|          | 0/22140 [00:00<?, ? examples/s]

Map:   0%|          | 0/5588 [00:00<?, ? examples/s]

***Remark: max_seq_len check from visualization process in longest sentence***

***Remark: Set as large number first then adjust number of word adjust follow print result of Vocabulary size from tokenizer.***

## Visualize the result (train dataset)

In [58]:
# Visualize a samples of the dataset
num_samples = min(3, len(tokenized_train))

for i in range(num_samples):
    sample = tokenized_train[i]
    print("\n" + "-" * 50)
    print(f"Sample {i+1} ")
    print(f"Original Text: {sample['text']}")
    print(f"Input IDs: {sample['input_ids']}")
    print(f"Attention Mask: {sample['attention_mask']}")
    
    # Decoder
    decoded_text = tokenizer.decode(sample['input_ids'])
    print(f"Decoded Text: {decoded_text}")

print("-" * 50)


--------------------------------------------------
Sample 1 
Original Text: <bos> Yes I need a cheap restaurant in the Cambridge area on the north side of town . What do you suggest ?
Input IDs: [0, 244, 97, 164, 89, 453, 225, 131, 102, 264, 291, 168, 102, 511, 721, 182, 286, 93, 294, 240, 100, 832, 108, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
Attention Mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Decoded Text:  Yes I need a cheap restaurant in the Cambridge area on the north side of town . What do you suggest ?

--------------------------------------------------
Sample 2 
Original Text: thank you I will go to royal spice . <eos>
Input IDs: [348, 100, 97, 224, 359, 113, 2254, 2644, 93, 87, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
Attention Mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0

## Visualize the result (test dataset)

In [59]:
# Visualize a samples of the dataset

num_samples = min(3, len(tokenized_test))

for i in range(num_samples):
    sample = tokenized_train[i]
    print("\n" + "-" * 50)
    print(f"Sample {i+1} ")
    print(f"Original Text: {sample['text']}")
    print(f"Input IDs: {sample['input_ids']}")
    print(f"Attention Mask: {sample['attention_mask']}")
    
    # Decoder
    decoded_text = tokenizer.decode(sample['input_ids'])
    print(f"Decoded Text: {decoded_text}")

print("=" * 50)


--------------------------------------------------
Sample 1 
Original Text: <bos> Yes I need a cheap restaurant in the Cambridge area on the north side of town . What do you suggest ?
Input IDs: [0, 244, 97, 164, 89, 453, 225, 131, 102, 264, 291, 168, 102, 511, 721, 182, 286, 93, 294, 240, 100, 832, 108, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
Attention Mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Decoded Text:  Yes I need a cheap restaurant in the Cambridge area on the north side of town . What do you suggest ?

--------------------------------------------------
Sample 2 
Original Text: thank you I will go to royal spice . <eos>
Input IDs: [348, 100, 97, 224, 359, 113, 2254, 2644, 93, 87, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
Attention Mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0

## Split data into train and validation

In [60]:
# Split into train/validation
train_val_split = tokenized_train.train_test_split(test_size=0.2, seed=42)
tokenized_train = train_val_split["train"]
tokenized_val = train_val_split["test"]

# Check the size of each dataset
print(f"Size of training dataset: {len(tokenized_train)}")
print(f"Size of validation dataset: {len(tokenized_val)}")

Size of training dataset: 17712
Size of validation dataset: 4428


## Convert to TensorFlow datasets

In [61]:
# Btch Size for train model
batch_size = 2

# Convert to TensorFlow datasets
def to_tf_dataset(hf_dataset, tokenizer, max_seq_len, pad_token_id, batch_size):
    """
    Convert a Hugging Face dataset to a TensorFlow dataset with proper formatting.
    """
    input_ids = pad_sequences(
        hf_dataset["input_ids"],
        maxlen=max_seq_len,
        padding="post",
        truncating="post",
        value=pad_token_id
    )
    
    # Create next token prediction targets by shifting the input IDs.
    labels = np.roll(input_ids, -1, axis=1)
    # Ignore loss for last position padding
    labels[:, -1] = pad_token_id
    
    # Use attention mask to ignore pad tokens during loss calculation
    attention_mask = pad_sequences(
        hf_dataset["attention_mask"],
        maxlen=max_seq_len,
        padding="post",
        truncating="post",
        value=0
    )
    
    dataset = tf.data.Dataset.from_tensor_slices((
        {
            "input_ids": input_ids,
            "attention_mask": attention_mask
        },
        labels
    ))
    return dataset.shuffle(500).batch(batch_size).prefetch(tf.data.AUTOTUNE)

tf_train_dataset = to_tf_dataset(tokenized_train, tokenizer, max_seq_len, PAD_TOKEN_ID, batch_size)
tf_val_dataset = to_tf_dataset(tokenized_val, tokenizer, max_seq_len, PAD_TOKEN_ID, batch_size)
tf_test_dataset = to_tf_dataset(tokenized_test, tokenizer, max_seq_len, PAD_TOKEN_ID, batch_size)

# Initial model

## DeepSeekMoE model

In [20]:
# Disable XLA (XLA JIT compilation) to prevent CUDA graph errors and ensure stability
tf.config.optimizer.set_jit(False)  # Disabled XLA to prevent CUDA graph errors

# Configure GPU memory growth to prevent memory allocation issues
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Enable mixed precision training with FP16 for improved performance and reduced memory usage
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

# Configuration parameters from the model and preprocessing pipeline
vocab_size = 12000
max_seq_len = 70
batch_size = 2
PAD_TOKEN = "<pad>"
BOS_TOKEN = "<bos>"
EOS_TOKEN = "<eos>"
SEP_TOKEN = "<sep>"

# Print TensorFlow version, mixed precision policy, and GPU availability for debugging
print(f"TensorFlow version: {tf.__version__}")
print(f"Mixed precision policy: {mixed_precision.global_policy().name}")
print(f"GPU available: {len(gpus) > 0}")

# DEEPSEEK MOE COMPONENTS
class DeepSeekRMSNorm(layers.Layer):
    """RMS Normalization layer """
    def __init__(self, hidden_size, eps=1e-6, **kwargs):
        super().__init__(**kwargs)
        self.hidden_size = hidden_size  # Dimension of the hidden representations
        self.variance_epsilon = eps     # Small constant to avoid division by zero
        
    def build(self, input_shape):
        # Initialize learnable weight parameter for scaling normalized output
        self.weight = self.add_weight(
            name='weight',
            shape=(self.hidden_size,),
            initializer='ones',
            trainable=True
        )
        super().build(input_shape)
        
    def call(self, x):
        input_dtype = x.dtype
        x = tf.cast(x, tf.float32)  # Cast input to float32 for stable computation
        
        # Compute variance as mean of squared inputs along the last dimension
        variance = tf.reduce_mean(tf.square(x), axis=-1, keepdims=True)
        # Normalize input using RMS formula with epsilon for numerical stability
        x = x * tf.math.rsqrt(variance + self.variance_epsilon)
        
        # Scale normalized output with learnable weight and cast back to original dtype
        return self.weight * tf.cast(x, input_dtype)

class RotaryPositionEmbedding(layers.Layer):
    """Rotary Position Embedding (RoPE) with dynamic NTK scaling for positional encoding"""
    def __init__(self, dim, max_position_embeddings=2048, base=10000, scaling_type=None, scaling_factor=1.0, **kwargs):
        super().__init__(**kwargs)
        self.dim = dim                              # Dimension of the head embeddings
        self.max_position_embeddings = max_position_embeddings  # Maximum sequence length
        self.base = base                           # Base value for frequency calculation
        self.scaling_type = scaling_type           # Type of scaling ("linear" or "dynamic")
        self.scaling_factor = scaling_factor       # Scaling factor for RoPE
        
    def build(self, input_shape):
        # Compute inverse frequencies for rotary embeddings
        inv_freq = 1.0 / (self.base ** (tf.range(0, self.dim, 2, dtype=tf.float32) / tf.cast(self.dim, tf.float32)))
        self.inv_freq = tf.constant(inv_freq, dtype=tf.float32)
        self._compute_rotary_cache()  # Precompute cosine and sine caches
        super().build(input_shape)
        
    def _compute_rotary_cache(self):
        """Compute cosine and sine caches for rotary embeddings with optional scaling"""
        if self.scaling_type == "linear":
            # Linear scaling: divide positions by scaling factor
            t = tf.range(self.max_position_embeddings, dtype=tf.float32) / self.scaling_factor
        elif self.scaling_type == "dynamic":
            # Dynamic NTK scaling for extended context lengths
            if self.max_position_embeddings > self.base:
                base = self.base * (
                    (self.scaling_factor * self.max_position_embeddings / self.base) - 
                    (self.scaling_factor - 1)
                ) ** (self.dim / (self.dim - 2))
                inv_freq = 1.0 / (base ** (tf.range(0, self.dim, 2, dtype=tf.float32) / tf.cast(self.dim, tf.float32)))
                self.inv_freq = tf.constant(inv_freq, dtype=tf.float32)
            t = tf.range(self.max_position_embeddings, dtype=tf.float32)
        else:
            # Default case: no scaling
            t = tf.range(self.max_position_embeddings, dtype=tf.float32)
            
        # Compute frequency matrix for rotary embeddings
        freqs = tf.einsum('i,j->ij', t, self.inv_freq)
        emb = tf.concat([freqs, freqs], axis=-1)  # Concatenate for full embedding dimension
        
        # Cache cosine and sine values as non-trainable variables
        self.cos_cached = tf.Variable(tf.cos(emb), trainable=False, name='cos_cached')
        self.sin_cached = tf.Variable(tf.sin(emb), trainable=False, name='sin_cached')
        
    def call(self, x, position_ids=None, seq_len=None):
        # Get input tensor dimensions
        batch_size, seq_len_x, num_heads, head_dim = tf.unstack(tf.shape(x))
        
        # Generate position IDs if not provided
        if position_ids is None:
            position_ids = tf.range(seq_len_x, dtype=tf.int32)
            position_ids = tf.tile(tf.reshape(position_ids, [1, -1]), [batch_size, 1])
        
        # Gather precomputed cosine and sine values for given position IDs
        cos = tf.gather(self.cos_cached, position_ids, axis=0)  # Shape: [batch_size, seq_len_x, head_dim]
        sin = tf.gather(self.sin_cached, position_ids, axis=0)  # Shape: [batch_size, seq_len_x, head_dim]
        
        # Split cosine and sine for application to x1 and x2
        cos_1, cos_2 = tf.split(cos, 2, axis=-1)
        sin_1, sin_2 = tf.split(sin, 2, axis=-1)
        
        # Reshape for broadcasting with attention heads
        cos_1 = tf.reshape(cos_1, [batch_size, seq_len_x, 1, head_dim // 2])
        sin_1 = tf.reshape(sin_1, [batch_size, seq_len_x, 1, head_dim // 2])
        cos_2 = tf.reshape(cos_2, [batch_size, seq_len_x, 1, head_dim // 2])
        sin_2 = tf.reshape(sin_2, [batch_size, seq_len_x, 1, head_dim // 2])
        
        # Repeat to match number of attention heads
        cos_1 = tf.repeat(cos_1, num_heads, axis=2)
        sin_1 = tf.repeat(sin_1, num_heads, axis=2)
        cos_2 = tf.repeat(cos_2, num_heads, axis=2)
        sin_2 = tf.repeat(sin_2, num_heads, axis=2)
        
        # Cast to input dtype for mixed precision compatibility
        x_dtype = x.dtype
        cos_1 = tf.cast(cos_1, x_dtype)
        sin_1 = tf.cast(sin_1, x_dtype)
        cos_2 = tf.cast(cos_2, x_dtype)
        sin_2 = tf.cast(sin_2, x_dtype)
        
        # Split input tensor into two parts for rotary application
        x1, x2 = tf.split(x, 2, axis=-1)
        
        # Apply rotary embeddings to compute rotated representations
        rotated_x1 = x1 * cos_1 - x2 * sin_1
        rotated_x2 = x1 * sin_2 + x2 * cos_2
        
        # Concatenate rotated parts to restore original shape
        return tf.concat([rotated_x1, rotated_x2], axis=-1)

class DeepSeekMLP(layers.Layer):
    """Multi-Layer Perceptron (MLP) with Swish activation"""
    def __init__(self, hidden_size, intermediate_size, hidden_act="swish", **kwargs):
        super().__init__(**kwargs)
        self.hidden_size = hidden_size           # Input and output dimension
        self.intermediate_size = intermediate_size  # Intermediate layer dimension
        self.hidden_act = hidden_act            # Activation function (Swish/SILU)
        
    def build(self, input_shape):
        # Define gate projection layer with reduced initializer variance for stability
        self.gate_proj = layers.Dense(
            self.intermediate_size,
            kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.001),
            use_bias=False
        )
        # Define up projection layer
        self.up_proj = layers.Dense(
            self.intermediate_size,
            kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.001),
            use_bias=False
        )
        # Define down projection layer to return to hidden size
        self.down_proj = layers.Dense(
            self.hidden_size,
            kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.001),
            use_bias=False
        )
        super().build(input_shape)
        
    def call(self, x):
        # Apply gate and up projections
        gate_output = self.gate_proj(x)
        up_output = self.up_proj(x)
        # Apply Swish (SiLU) activation to gate output and multiply with up output
        activated = tf.nn.silu(gate_output) * up_output
        # Project back to hidden size
        return self.down_proj(activated)

class MoEGate(layers.Layer):
    """Mixture of Experts (MoE) gating mechanism with load balancing auxiliary loss"""
    def __init__(self, num_experts, num_experts_per_tok, hidden_size, 
                 scoring_func='softmax', norm_topk_prob=False, 
                 aux_loss_alpha=0.001, seq_aux=True, **kwargs):
        super().__init__(**kwargs)
        self.num_experts = num_experts                  # Total number of experts
        self.num_experts_per_tok = num_experts_per_tok  # Number of experts per token
        self.hidden_size = hidden_size                  # Input dimension
        self.scoring_func = scoring_func                # Scoring function for expert selection
        self.norm_topk_prob = norm_topk_prob            # Normalize top-k probabilities
        self.aux_loss_alpha = aux_loss_alpha            # Weight for auxiliary loss
        self.seq_aux = seq_aux                          # Compute auxiliary loss per sequence
        
    def build(self, input_shape):
        # Initialize gating weights with reduced variance for stability
        self.weight = self.add_weight(
            name='gate_weight',
            shape=(self.num_experts, self.hidden_size),
            initializer=tf.keras.initializers.RandomNormal(stddev=0.001),
            trainable=True
        )
        # Initialize expert assignment counter for tracking during training
        self.expert_counts = tf.Variable(tf.zeros([self.num_experts], dtype=tf.int32), trainable=False)
        super().build(input_shape)
        
    def call(self, hidden_states, training=False):
        # Get input dimensions
        batch_size, seq_len, h = tf.unstack(tf.shape(hidden_states))
        hidden_states_flat = tf.reshape(hidden_states, [-1, h])
        # Compute gating logits
        logits = tf.matmul(hidden_states_flat, self.weight, transpose_b=True)
        
        # Clip logits for numerical stability
        logits = tf.clip_by_value(logits, -50.0, 50.0)
        
        # Apply scoring function (softmax) to compute expert scores
        if self.scoring_func == 'softmax':
            temperature = 0.5                                         # Temperature for scaling logits
            scaled_logits = tf.cast(logits, tf.float32) / temperature
            scores = tf.nn.softmax(scaled_logits, axis=-1)
            scores = tf.cast(scores, hidden_states.dtype)
        else:
            scores = tf.nn.softmax(tf.cast(logits, tf.float32), axis=-1)
            scores = tf.cast(scores, hidden_states.dtype)

        # Select top-k experts and their weights
        topk_weights, topk_indices = tf.math.top_k(
            scores, k=self.num_experts_per_tok, sorted=False
        )

        # Normalize top-k weights if specified
        if self.num_experts_per_tok > 1 and self.norm_topk_prob:
            denominator = tf.reduce_sum(topk_weights, axis=-1, keepdims=True) + 1e-20
            topk_weights = topk_weights / denominator

        # Update expert assignment counts during training
        if training:
            flat_topk_indices = tf.reshape(topk_indices, [-1])
            counts = tf.reduce_sum(tf.one_hot(flat_topk_indices, depth=self.num_experts), axis=0)
            self.expert_counts.assign_add(tf.cast(counts, tf.int32))

        aux_loss = None
        if training and self.aux_loss_alpha > 0.0:
            if self.seq_aux:
                # Sequence-level auxiliary loss for load balancing
                scores_seq = tf.reshape(scores, [batch_size, seq_len, -1])
                topk_indices_seq = tf.reshape(topk_indices, [batch_size, -1])
                ce = tf.zeros([batch_size, self.num_experts], dtype=tf.float32)
                mask = tf.one_hot(topk_indices_seq, depth=self.num_experts)
                ce = tf.reduce_sum(mask, axis=1)
                seq_len_float = tf.cast(seq_len, tf.float32)
                num_experts_per_tok_float = tf.cast(self.num_experts_per_tok, tf.float32)
                num_experts_float = tf.cast(self.num_experts, tf.float32)
                ce = ce / (seq_len_float * num_experts_per_tok_float / num_experts_float)
                scores_seq_mean = tf.reduce_mean(tf.cast(scores_seq, tf.float32), axis=1)
                aux_loss = tf.reduce_sum(ce * scores_seq_mean, axis=1)
                aux_loss = tf.reduce_mean(aux_loss) * self.aux_loss_alpha
            else:
                # Token-level auxiliary loss
                mask_ce = tf.one_hot(tf.reshape(topk_indices, [-1]), depth=self.num_experts)
                ce = tf.reduce_mean(tf.cast(mask_ce, tf.float32), axis=0)
                Pi = tf.reduce_mean(tf.cast(scores, tf.float32), axis=0)
                fi = ce * self.num_experts
                aux_loss = tf.reduce_sum(Pi * fi) * self.aux_loss_alpha

            # Add auxiliary loss to the layer's losses
            self.add_loss(aux_loss)

        return topk_indices, topk_weights, aux_loss

class DeepSeekMoE(layers.Layer):
    """Mixture of Experts (MoE) layer with shared and routed experts"""
    def __init__(self, hidden_size, n_routed_experts, n_shared_experts, 
                 num_experts_per_tok, moe_intermediate_size, **kwargs):
        super().__init__(**kwargs)
        self.hidden_size = hidden_size                      # Input and output dimension
        self.n_routed_experts = n_routed_experts            # Number of routed experts
        self.n_shared_experts = n_shared_experts            # Number of shared experts
        self.num_experts_per_tok = num_experts_per_tok      # Experts per token
        self.moe_intermediate_size = moe_intermediate_size  # MoE intermediate size
        
    def build(self, input_shape):
        # Initialize list of routed expert MLPs
        self.routed_experts = [
            DeepSeekMLP(
                self.hidden_size, 
                self.moe_intermediate_size,
                name=f"routed_expert_{i}"
            ) for i in range(self.n_routed_experts)
        ]
        
        # Initialize shared experts if specified
        if self.n_shared_experts is not None:
            shared_intermediate_size = self.moe_intermediate_size * self.n_shared_experts
            self.shared_experts = DeepSeekMLP(
                self.hidden_size,
                shared_intermediate_size,
                name="shared_experts"
            )
        else:
            self.shared_experts = None
            
        # Initialize MoE gating mechanism
        self.gate = MoEGate(
            self.n_routed_experts,
            self.num_experts_per_tok,
            self.hidden_size,
            name="moe_gate"
        )
        
        # Build each expert and gate
        for expert in self.routed_experts:
            expert.build(input_shape)
        if self.shared_experts:
            self.shared_experts.build(input_shape)
        self.gate.build(input_shape)
        
        super().build(input_shape)
        
    def call(self, hidden_states, training=False):
        identity = hidden_states  # Store input for residual connection
        orig_shape = tf.shape(hidden_states)
        # Get top-k expert indices and weights
        topk_idx, topk_weight, aux_loss = self.gate(hidden_states, training=training)
        hidden_states_flat = tf.reshape(hidden_states, [-1, self.hidden_size])
        
        if training:
            # Expand inputs for top-k experts during training
            hidden_states_expanded = tf.repeat(
                hidden_states_flat, self.num_experts_per_tok, axis=0
            )
            flat_topk_idx = tf.reshape(topk_idx, [-1])
            combined_output = tf.zeros_like(hidden_states_expanded)
            # Process each expert's input
            for i, expert in enumerate(self.routed_experts):
                expert_mask = tf.equal(flat_topk_idx, i)
                if tf.reduce_any(expert_mask):
                    expert_input = tf.boolean_mask(hidden_states_expanded, expert_mask)
                    expert_output = expert(expert_input)
                    update_indices = tf.where(expert_mask)
                    combined_output = tf.tensor_scatter_nd_update(
                        combined_output,
                        update_indices,
                        expert_output
                    )
                    
            # Apply expert weights and reshape output
            topk_weight_flat = tf.reshape(topk_weight, [-1, 1])
            weighted_output = combined_output * topk_weight_flat
            weighted_output = tf.reshape(
                weighted_output,
                [-1, self.num_experts_per_tok, self.hidden_size]
            )
            routed_output = tf.reduce_sum(weighted_output, axis=1)
            routed_output = tf.reshape(routed_output, orig_shape)
        else:
            # Use efficient inference path
            routed_output = self._moe_infer(hidden_states_flat, topk_idx, topk_weight)
            routed_output = tf.reshape(routed_output, orig_shape)
            
        # Add shared expert output if available
        if self.shared_experts is not None:
            shared_output = self.shared_experts(identity)
            final_output = routed_output + shared_output
        else:
            final_output = routed_output
            
        # Apply residual connection
        return final_output + identity
        
    def _moe_infer(self, x, expert_indices, expert_weights):
        """Efficient inference path for MoE layer"""
        batch_size = tf.shape(x)[0]
        expert_cache = tf.zeros_like(x)
        # Clip expert indices to valid range
        expert_indices = tf.clip_by_value(expert_indices, 0, self.n_routed_experts - 1)
        flat_expert_indices = tf.reshape(expert_indices, [-1])
        flat_expert_weights = tf.reshape(expert_weights, [-1, 1])
        total_tokens = tf.shape(x)[0]
        token_indices_base = tf.range(total_tokens, dtype=tf.int64)
        token_indices_base = tf.repeat(token_indices_base, self.num_experts_per_tok, axis=0)
        
        # Process each expert's tokens
        for i in range(self.n_routed_experts):
            expert_mask = tf.equal(flat_expert_indices, i)
            if tf.reduce_any(expert_mask):
                token_indices = tf.boolean_mask(token_indices_base, expert_mask)
                expert_tokens = tf.gather(x, token_indices)
                expert_out = self.routed_experts[i](expert_tokens)
                expert_out_weighted = expert_out * tf.gather(flat_expert_weights, tf.where(expert_mask)[:, 0])
                expert_cache = tf.tensor_scatter_nd_add(
                    expert_cache,
                    tf.expand_dims(token_indices, 1),
                    expert_out_weighted
                )
                
        return expert_cache

class DeepSeekAttention(layers.Layer):
    """Multi-head attention with Rotary Position Embeddings and Grouped Query Attention (GQA) support"""
    def __init__(self, hidden_size, num_attention_heads, num_key_value_heads=None,
                 attention_dropout=0.0, max_position_embeddings=2048,
                 rope_theta=10000.0, rope_scaling=None, attention_bias=False, **kwargs):
        super().__init__(**kwargs)
        self.hidden_size = hidden_size                                         # Input and output dimension
        self.num_attention_heads = num_attention_heads                         # Number of attention heads
        self.num_key_value_heads = num_key_value_heads or num_attention_heads  # Number of key/value heads (GQA)
        self.attention_dropout = attention_dropout                             # Dropout rate for attention weights
        self.max_position_embeddings = max_position_embeddings                 # Maximum sequence length
        self.rope_theta = rope_theta                                           # Base period for RoPE
        self.rope_scaling = rope_scaling                                       # RoPE scaling configuration
        self.attention_bias = attention_bias                                   # Whether to use bias in projections
        self.head_dim = hidden_size // num_attention_heads                     # Dimension per head
        self.num_key_value_groups = num_attention_heads // self.num_key_value_heads  # Groups for GQA
        
    def build(self, input_shape):
        # Initialize query projection layer
        self.q_proj = layers.Dense(
            self.num_attention_heads * self.head_dim,
            use_bias=self.attention_bias,
            kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.001),
            name="q_proj"
        )
        # Initialize key projection layer
        self.k_proj = layers.Dense(
            self.num_key_value_heads * self.head_dim,
            use_bias=self.attention_bias,
            kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.001),
            name="k_proj"
        )
        # Initialize value projection layer
        self.v_proj = layers.Dense(
            self.num_key_value_heads * self.head_dim,
            use_bias=self.attention_bias,
            kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.001),
            name="v_proj"
        )
        # Initialize output projection layer
        self.o_proj = layers.Dense(
            self.hidden_size,
            use_bias=self.attention_bias,
            kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.001),
            name="o_proj"
        )
        
        # Initialize RoPE with specified scaling
        scaling_type = self.rope_scaling.get("type") if self.rope_scaling else None
        scaling_factor = self.rope_scaling.get("factor", 1.0) if self.rope_scaling else 1.0
        self.rotary_emb = RotaryPositionEmbedding(
            self.head_dim,
            self.max_position_embeddings,
            self.rope_theta,
            scaling_type,
            scaling_factor,
            name="rotary_emb"
        )
        self.dropout_layer = layers.Dropout(self.attention_dropout)
        self.rotary_emb.build(input_shape)
        super().build(input_shape)
        
    def call(self, hidden_states, attention_mask=None, position_ids=None, training=False):
        # Get input dimensions
        batch_size, seq_len, _ = tf.unstack(tf.shape(hidden_states))
        
        # Project inputs to query, key, and value
        q = self.q_proj(hidden_states)
        k = self.k_proj(hidden_states)
        v = self.v_proj(hidden_states)
        
        # Reshape for multi-head attention
        q = tf.reshape(q, [batch_size, seq_len, self.num_attention_heads, self.head_dim])
        k = tf.reshape(k, [batch_size, seq_len, self.num_key_value_heads, self.head_dim])
        v = tf.reshape(v, [batch_size, seq_len, self.num_key_value_heads, self.head_dim])
        
        # Apply rotary embeddings to query and key
        q_rotated = self.rotary_emb(q, position_ids=position_ids)
        k_rotated = self.rotary_emb(k, position_ids=position_ids)
        
        # Transpose for attention computation
        q_rotated = tf.transpose(q_rotated, [0, 2, 1, 3])
        k_rotated = tf.transpose(k_rotated, [0, 2, 1, 3])
        v = tf.transpose(v, [0, 2, 1, 3])
        
        # Repeat key and value for GQA if necessary
        if self.num_key_value_groups > 1:
            k_rotated = tf.repeat(k_rotated, self.num_key_value_groups, axis=1)
            v = tf.repeat(v, self.num_key_value_groups, axis=1)
            
        # Compute scaled dot-product attention
        scale = tf.math.rsqrt(tf.cast(self.head_dim, tf.float32))
        scale = tf.cast(scale, q_rotated.dtype)
        attn_weights = tf.matmul(q_rotated, k_rotated, transpose_b=True) * scale
        
        # Apply attention mask if provided
        if attention_mask is not None:
            attention_mask = tf.cast(attention_mask, attn_weights.dtype)
            if len(attention_mask.shape) == 2:
                attention_mask = tf.expand_dims(tf.expand_dims(attention_mask, 1), 1)
            elif len(attention_mask.shape) == 3:
                attention_mask = tf.expand_dims(attention_mask, 1)
            attn_weights += attention_mask
            
        # Clip attention weights for numerical stability
        attn_weights = tf.clip_by_value(attn_weights, -50.0, 50.0)
        attn_weights = tf.cast(attn_weights, tf.float32)
        # Apply softmax with max subtraction for stability
        attn_weights = attn_weights - tf.reduce_max(attn_weights, axis=-1, keepdims=True)
        attn_weights = tf.nn.softmax(attn_weights, axis=-1)
        attn_weights = tf.cast(attn_weights, q_rotated.dtype)
        
        # Apply dropout to attention weights during training
        attn_weights = self.dropout_layer(attn_weights, training=training)
        # Compute attention output
        attn_output = tf.matmul(attn_weights, v)
        attn_output = tf.transpose(attn_output, [0, 2, 1, 3])
        attn_output = tf.reshape(attn_output, [batch_size, seq_len, self.hidden_size])
        
        # Project attention output to hidden size
        return self.o_proj(attn_output)

class DeepSeekDecoderLayer(layers.Layer):
    """Transformer decoder layer combining attention and MoE/MLP with residual connections"""
    def __init__(self, config, layer_idx, **kwargs):
        super().__init__(**kwargs)
        self.config = config
        self.layer_idx = layer_idx
        
        # Initialize self-attention with GQA and RoPE
        self.self_attn = DeepSeekAttention(
            hidden_size=config.hidden_size,
            num_attention_heads=config.num_attention_heads,
            num_key_value_heads=config.num_key_value_heads,
            attention_dropout=config.attention_dropout,
            max_position_embeddings=config.max_position_embeddings,
            rope_theta=config.rope_theta,
            rope_scaling=config.rope_scaling,
            attention_bias=config.attention_bias,
            name="self_attn"
        )
        
        # Determine whether to use MoE or dense MLP based on layer index and config
        self.use_moe = (
            config.n_routed_experts is not None and
            layer_idx >= config.first_k_dense_replace and
            layer_idx % config.moe_layer_freq == 0
        )
        
        if self.use_moe:
            # Initialize MoE layer for routed experts
            self.mlp = DeepSeekMoE(
                hidden_size=config.hidden_size,
                n_routed_experts=config.n_routed_experts,
                n_shared_experts=config.n_shared_experts,
                num_experts_per_tok=config.num_experts_per_tok,
                moe_intermediate_size=config.moe_intermediate_size,
                name="moe"
            )
        else:
            # Initialize dense MLP layer
            self.mlp = DeepSeekMLP(
                hidden_size=config.hidden_size,
                intermediate_size=config.intermediate_size,
                hidden_act=config.hidden_act,
                name="mlp"
            )
            
        # Initialize input and post-attention normalization layers
        self.input_layernorm = DeepSeekRMSNorm(
            config.hidden_size, eps=config.rms_norm_eps, name="input_layernorm"
        )
        self.post_attention_layernorm = DeepSeekRMSNorm(
            config.hidden_size, eps=config.rms_norm_eps, name="post_attention_layernorm"
        )
        
    def call(self, hidden_states, attention_mask=None, position_ids=None, training=False):
        # Apply input normalization and self-attention with residual connection
        residual = hidden_states
        hidden_states = self.input_layernorm(hidden_states)
        attn_output = self.self_attn(
            hidden_states,
            attention_mask=attention_mask,
            position_ids=position_ids,
            training=training
        )
        hidden_states = residual + attn_output
        
        # Apply post-attention normalization and MLP/MoE with residual connection
        residual = hidden_states
        hidden_states = self.post_attention_layernorm(hidden_states)
        mlp_output = self.mlp(hidden_states, training=training)
        hidden_states = residual + mlp_output
        
        return hidden_states

# MODEL CONFIGURATION 

class DeepSeekConfig:
    """
    This is the configuration class to store the configuration of a [`DeepseekMoEModel`]. It is used to instantiate an DeepSeek
    model according to the specified arguments, defining the model architecture. Instantiating a configuration with the
    defaults below will yield a configuration for a small, stable, and computationally efficient MoE chatbot model.

    Args:
        vocab_size (`int`, *optional*, defaults to 12000):
            Vocabulary size of the Deep model. Defines the number of different tokens that can be represented by the
            `inputs_ids` passed when calling [`DeepseekModel`]. (Optimized for chatbot use case).
            
        hidden_size (`int`, *optional*, defaults to 256):
            Dimension of the hidden representations. (Optimized hidden size for stability and performance).
            
        intermediate_size (`int`, *optional*, defaults to 512):
            Dimension of the MLP representations. (Balanced intermediate size for MLP layers).
            
        moe_intermediate_size (`int`, *optional*, defaults to 256):
            Dimension of the MoE representations. (MoE-specific intermediate dimension).
            
        num_hidden_layers (`int`, *optional*, defaults to 4):
            Number of hidden layers in the Transformer decoder. (Optimal layer count for chatbot tasks).
            
        num_attention_heads (`int`, *optional*, defaults to 4):
            Number of attention heads for each attention layer in the Transformer decoder. (Balanced attention heads for computation).
        
        n_shared_experts (`int`, *optional*, defaults to 1):
            Number of shared experts, None means dense model. (Set to 1, a single shared expert for parameter efficiency).
        
        n_routed_experts (`int`, *optional*, defaults to 4):
            Number of routed experts, None means dense model. (Set to 4 for specialization).
            
        num_experts_per_tok (`int`, *optional*, defaults to 2):
            Number of selected experts, None means dense model. (Set to 2 experts per token for balanced load).
            
        moe_layer_freq (`int`, *optional*, defaults to 2):
            The frequency of the MoE layer: one expert layer for every `moe_layer_freq - 1` dense layers.
            (MoE every other layer for computational balance).
            
        first_k_dense_replace (`int`, *optional*, defaults to 1):
            - Number of dense layers in shallow layers(embed->dense->dense->...->dense->moe->moe...->lm_head).
            - k dense layers (First layer dense for stability).
        norm_topk_prob (`bool`, *optional*, defaults to False):
            Whether to normalize the weights of the routed experts. (No normalization for expert weights).
            
        scoring_func (`str`, *optional*, defaults to 'softmax'):
            Method of computing expert weights. (Standard softmax for expert selection).
            
        aux_loss_alpha (`float`, *optional*, defaults to 0.001):
            Auxiliary loss weight coefficient. (Default auxiliary loss weight).
            
        seq_aux = (`bool`, *optional*, defaults to True):
            Whether to compute the auxiliary loss for each individual sample. (Sequence-level auxiliary loss).
            
        num_key_value_heads (`int`, *optional*, defaults to 2):
            This is the number of key_value heads that should be used to implement Grouped Query Attention. If
            `num_key_value_heads=num_attention_heads`, the model will use Multi Head Attention (MHA), if
            `num_key_value_heads=1` the model will use Multi Query Attention (MQA) otherwise GQA is used.
            (Set to 2 for GQA and memory efficiency).
            
        hidden_act (`str` or `function`, *optional*, defaults to "swish"):
            The non-linear activation function (function or string) in the decoder. (Swish activation for better gradients).
        
        max_position_embeddings (`int`, *optional*, defaults to 256):
            The maximum sequence length that this model might ever be used with. (Sufficient context length for chatbot).
        
        initializer_range (`float`, *optional*, defaults to 0.001):
            The standard deviation of the truncated_normal_initializer for initializing all weight matrices. (Reduced initializer range for stability).
        
        rms_norm_eps (`float`, *optional*, defaults to 1e-6):
            The epsilon used by the rms normalization layers. (Standard RMS norm epsilon).
        
        use_cache (`bool`, *optional*, defaults to True):
            Whether or not the model should return the last key/values attentions (not used by all models). Only
            relevant if `config.is_decoder=True`. (Enable caching for inference).
        
        pad_token_id (`int`, *optional*):
            Padding token id.
        bos_token_id (`int`, *optional*, defaults to 1):
            Beginning of stream token id.
        eos_token_id (`int`, *optional*, defaults to 2):
            End of stream token id.
        
        pretraining_tp (`int`, *optional*, defaults to 1):
            Experimental feature. Tensor parallelism rank used during pretraining. Please refer to [this
            document](https://huggingface.co/docs/transformers/parallelism) to understand more about it. This value is
            necessary to ensure exact reproducibility of the pretraining results. Please refer to [this
            issue](https://github.com/pytorch/pytorch/issues/76232).
        
        tie_word_embeddings (`bool`, *optional*, defaults to False):
            Whether to tie weight embeddings
        
        rope_theta (`float`, *optional*, defaults to 10000.0):
            The base period of the RoPE embeddings. (Standard RoPE base period).
        
        rope_scaling (`Dict`, *optional*, defaults to {"type": "linear", "factor": 2.0}):
            Dictionary containing the scaling configuration for the RoPE embeddings. Currently supports two scaling
            strategies: linear and dynamic. Their scaling factor must be a float greater than 1. The expected format is
            `{"type": strategy name, "factor": scaling factor}`. When using this flag, don't update
            `max_position_embeddings` to the expected new maximum. (Linear RoPE scaling enabled).
        
        attention_bias (`bool`, defaults to False, *optional*, defaults to False):
            Whether to use a bias in the query, key, value and output projection layers during self-attention. (No attention bias for efficiency).
        
        attention_dropout (`float`, *optional*, defaults to 0.1):
            The dropout ratio for the attention probabilities. (Increased dropout for regularization).
    """

    def __init__(
        self,
        vocab_size=vocab_size,          # Default vocabulary size optimized for chatbot
        hidden_size=256,                # Optimized hidden size for stability and performance
        intermediate_size=512,          # Balanced intermediate size for MLP layers
        moe_intermediate_size=256,      # MoE-specific intermediate dimension
        num_hidden_layers=4,            # Optimal layer count for chatbot tasks
        num_attention_heads=4,          # Balanced attention heads for computation
        num_key_value_heads=2,          # GQA for memory efficiency
        n_shared_experts=1,             # Single shared expert for parameter efficiency
        n_routed_experts=4,             # Four routed experts for specialization
        num_experts_per_tok=2,          # Two experts per token for balanced load
        moe_layer_freq=2,               # MoE every other layer for computational balance
        first_k_dense_replace=1,        # First layer dense for stability
        norm_topk_prob=False,           # No normalization for expert weights
        scoring_func='softmax',         # Standard softmax for expert selection
        aux_loss_alpha=0.001,           # Default auxiliary loss weight
        seq_aux=True,                   # Sequence-level auxiliary loss
        hidden_act="swish",             # Swish activation for better gradients
        max_position_embeddings=256,    # Sufficient context length for chatbot
        initializer_range=0.001,        # Reduced initializer range for stability
        rms_norm_eps=1e-6,              # Standard RMS norm epsilon
        use_cache=True,                 # Enable caching for inference
        rope_theta=10000.0,             # Standard RoPE base period
        rope_scaling={"type": "linear", "factor": 2.0},  # Linear RoPE scaling
        attention_bias=False,           # No attention bias for efficiency
        attention_dropout=0.1           # Increased dropout for regularization
    ):
        # Initialize model architecture parameters with optimized values
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.intermediate_size = intermediate_size
        self.moe_intermediate_size = moe_intermediate_size
        self.num_hidden_layers = num_hidden_layers
        self.num_attention_heads = num_attention_heads
        self.num_key_value_heads = num_key_value_heads
        self.n_shared_experts = n_shared_experts
        self.n_routed_experts = n_routed_experts
        self.num_experts_per_tok = num_experts_per_tok
        self.moe_layer_freq = moe_layer_freq
        self.first_k_dense_replace = first_k_dense_replace
        self.norm_topk_prob = norm_topk_prob
        self.scoring_func = scoring_func
        self.aux_loss_alpha = aux_loss_alpha
        self.seq_aux = seq_aux
        self.hidden_act = hidden_act
        self.max_position_embeddings = max_position_embeddings
        self.initializer_range = initializer_range
        self.rms_norm_eps = rms_norm_eps
        self.use_cache = use_cache
        self.rope_theta = rope_theta
        self.rope_scaling = rope_scaling
        self.attention_bias = attention_bias
        self.attention_dropout = attention_dropout

class DeepSeekModel(tf.keras.Model):
    """Core DeepSeek transformer model for chatbot, provide raw hidden states"""
    def __init__(self, config, **kwargs):
        super().__init__(**kwargs)
        self.config = config
        self.padding_idx = 0
        
        # Initialize embedding layer for token inputs
        self.embed_tokens = layers.Embedding(
            config.vocab_size,
            config.hidden_size,
            embeddings_initializer=tf.keras.initializers.RandomNormal(
                stddev=config.initializer_range
            ),
            name="embed_tokens"
        )
        
        # Initialize decoder layers
        self.decoder_layers = [
            DeepSeekDecoderLayer(config, layer_idx=i, name=f"layer_{i}")
            for i in range(config.num_hidden_layers)
        ]
        
        # Initialize final normalization layer
        self.norm = DeepSeekRMSNorm(
            config.hidden_size, eps=config.rms_norm_eps, name="norm"
        )
        
        # Initialize expert assignment counter for MoE layers
        self.expert_assignment_counts = tf.Variable(
            tf.zeros([config.n_routed_experts], dtype=tf.int32),
            trainable=False,
            name="expert_assignment_counts"
        ) if config.n_routed_experts else None
        
    def call(self, input_ids, attention_mask=None, position_ids=None, training=False):
        # Embed input tokens
        hidden_states = self.embed_tokens(input_ids)
        
        # Create causal mask if not provided
        if attention_mask is None:
            seq_len = tf.shape(input_ids)[1]
            attention_mask = self._create_causal_mask(seq_len)
            
        # Prepare attention mask for causal attention
        attention_mask = tf.cast(attention_mask, hidden_states.dtype)
        if len(attention_mask.shape) == 2:
            attention_mask = tf.expand_dims(tf.expand_dims(attention_mask, 1), 1)
        elif len(attention_mask.shape) == 3:
            attention_mask = tf.expand_dims(attention_mask, 1)
        attention_mask = (tf.cast(1.0, hidden_states.dtype) - attention_mask) * tf.cast(-1e4, hidden_states.dtype)
        
        # Process through decoder layers and aggregate expert counts
        for layer in self.decoder_layers:
            hidden_states = layer(
                hidden_states,
                attention_mask=attention_mask,
                position_ids=position_ids,
                training=training
            )
            # Update expert assignment counts for MoE layers during training
            if training and hasattr(layer, 'mlp') and isinstance(layer.mlp, DeepSeekMoE):
                expert_counts = layer.mlp.gate.expert_counts
                if self.expert_assignment_counts is not None:
                    self.expert_assignment_counts.assign_add(expert_counts)
            
        # Apply final normalization
        return self.norm(hidden_states)
        
    def _create_causal_mask(self, seq_len):
        """Create causal attention mask to prevent attending to future tokens"""
        mask = tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
        return tf.cast(mask, tf.float16)

class DeepSeekForCausalLM(tf.keras.Model):
    """Causal language model for chatbot text generation"""
    def __init__(self, config, **kwargs):
        super().__init__(**kwargs)
        self.config = config
        self.transformer = DeepSeekModel(config, name="transformer")
        # Initialize language model head
        self.lm_head = layers.Dense(
            config.vocab_size,
            kernel_initializer=tf.keras.initializers.RandomNormal(
                stddev=config.initializer_range
            ),
            use_bias=False,
            name="lm_head"
        )
        
    def call(self, inputs, attention_mask=None, position_ids=None, training=False):
        # Handle dictionary or tensor inputs
        if isinstance(inputs, dict):
            input_ids = inputs.get('input_ids')
            attention_mask = inputs.get('attention_mask', attention_mask)
        else:
            input_ids = inputs
        
        # Process through transformer
        hidden_states = self.transformer(
            input_ids,
            attention_mask=attention_mask,
            position_ids=position_ids,
            training=training
        )
        # Generate logits for token prediction
        return self.lm_head(hidden_states)

# METRICS AND CALLBACKS

class ResourceMonitor(tf.keras.callbacks.Callback):
    """Callback to monitor system resources (GPU, CPU, memory) during training"""
    def __init__(self, batch_size=2):
        super().__init__()
        self.batch_size = batch_size
        self.resource_metrics = []  # Store resource usage metrics
        self.start_time = time.time()
        self.total_tokens_processed = 0
        
    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()
        self.epoch_tokens = 0
        
    def on_batch_end(self, batch, logs=None):
        # Update token count for throughput calculation
        self.total_tokens_processed += self.batch_size * max_seq_len
        self.epoch_tokens += self.batch_size * max_seq_len
        
    def on_epoch_end(self, epoch, logs=None):
        try:
            # Collect GPU and system resource usage
            gpus = GPUtil.getGPUs()
            gpu_memory = sum([gpu.memoryUsed for gpu in gpus]) / len(gpus) if gpus else 0
            gpu_usage = sum([gpu.load * 100 for gpu in gpus]) / len(gpus) if gpus else 0
            cpu_usage = psutil.cpu_percent()
            memory_info = psutil.virtual_memory()
            system_memory = memory_info.used / (1024 ** 3)
            epoch_time = time.time() - self.epoch_start_time
            tokens_per_second = self.epoch_tokens / epoch_time if epoch_time > 0 else 0
            
            # Store resource metrics
            resource_stats = {
                'epoch': epoch + 1,
                'avg_gpu_memory_gb': float(gpu_memory / 1024),
                'avg_system_memory_gb': float(system_memory),
                'avg_gpu_usage_percent': float(gpu_usage),
                'avg_cpu_usage_percent': float(cpu_usage),
                'tokens_per_second': float(tokens_per_second),
                'throughput_tps': float(tokens_per_second)
            }
            
            self.resource_metrics.append(resource_stats)
            
            # Print resource usage summary
            print(f"Epoch {epoch+1} Resources: GPU Mem: {gpu_memory/1024:.1f}GB, "
                  f"System Mem: {system_memory:.1f}GB, GPU: {gpu_usage:.1f}%, "
                  f"CPU: {cpu_usage:.1f}%, TPS: {tokens_per_second:.0f}")
                  
        except Exception as e:
            print(f"Resource monitoring error: {e}")

class DeepSeekMetrics(tf.keras.callbacks.Callback):
    """Callback to track specific metrics, including MaxVIO for expert balancing"""
    def __init__(self, validation_data, tokenizer):
        super().__init__()
        self.validation_data = validation_data
        self.tokenizer = tokenizer
        self.metrics_history = []
        self.start_time = time.time()
        self.max_vio_values = []
        
    def on_epoch_begin(self, epoch, logs=None):
        # Reset expert assignment counts for accurate per-epoch metrics
        if hasattr(self.model.transformer, 'expert_assignment_counts') and self.model.transformer.expert_assignment_counts is not None:
            self.model.transformer.expert_assignment_counts.assign(tf.zeros([self.model.config.n_routed_experts], dtype=tf.int32))
        for layer in self.model.transformer.decoder_layers:
            if hasattr(layer, 'mlp') and isinstance(layer.mlp, DeepSeekMoE):
                layer.mlp.gate.expert_counts.assign(tf.zeros([layer.mlp.n_routed_experts], dtype=tf.int32))
        
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        
        # Estimate FLOPs for computational efficiency tracking
        total_params = self.model.count_params()
        flops_per_token = total_params * 2
        total_flops = flops_per_token * batch_size * max_seq_len
        total_gflops = total_flops / 1e9
        
        logs['gflops'] = float(total_gflops)
        
        # Calculate MaxVIO for expert load balancing
        if self.validation_data:
            try:
                max_vio = self._calculate_max_vio()
                logs['max_vio'] = float(max_vio)
                self.max_vio_values.append(max_vio)
                
            except Exception as e:
                print(f"Metrics calculation error: {e}")
                logs['max_vio'] = 0.0
                
        self.metrics_history.append(logs.copy())
        
        # Print simplified metrics summary
        print(f"Epoch {epoch+1}: Loss={logs.get('loss', 0):.4f}, "
              f"Val_Loss={logs.get('val_loss', 0):.4f}, "
              f"GFLOPs={total_gflops:.2f}, "
              f"MaxVIO={logs.get('max_vio', 0):.4f}")
              
    def _calculate_max_vio(self):
        """Calculate Maximum Violation (MaxVIO) for expert load balancing"""
        total_counts = np.zeros(self.model.config.n_routed_experts, dtype=np.float32)
        moe_layers = 0
        # Aggregate expert counts across MoE layers
        for layer in self.model.transformer.decoder_layers:
            if hasattr(layer, 'mlp') and isinstance(layer.mlp, DeepSeekMoE):
                counts = layer.mlp.gate.expert_counts.numpy()
                total_counts += counts
                moe_layers += 1
        
        # Compute MaxVIO based on aggregated counts
        if moe_layers > 0 and np.sum(total_counts) > 0:
            normalized_counts = total_counts / (np.sum(total_counts) + 1e-10)
            expected_uniform = 1.0 / len(total_counts)
            max_vio = np.max(np.abs(normalized_counts - expected_uniform)) / expected_uniform
            return max_vio
        return 0.0

class ComprehensiveModelCheckpoint(tf.keras.callbacks.Callback):
    """Callback for saving model checkpoints based on monitored metric"""
    def __init__(self, filepath, monitor='val_loss', save_best_only=True, mode='min'):
        super().__init__()
        self.filepath = filepath
        self.monitor = monitor
        self.save_best_only = save_best_only
        self.mode = mode
        self.best_value = float('inf') if mode == 'min' else -float('inf')
        
    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            return
            
        current_value = logs.get(self.monitor)
        if current_value is None:
            return
            
        # Determine if current model should be saved
        if self.mode == 'min':
            should_save = current_value < self.best_value
        else:
            should_save = current_value > self.best_value
            
        if should_save or not self.save_best_only:
            self.best_value = current_value
            self.model.save_weights(self.filepath)
            print(f"Model checkpoint saved with {self.monitor}: {current_value:.4f}")

# Measure Perplexity

# Function to calculate perplexity on validation set
def calculate_perplexity(model, dataset, tokenizer, max_batches=100):          # Sample batch for calculate perplexity
    """Calculate perplexity on validation dataset to evaluate model performance"""
    total_loss = 0.0
    total_tokens = 0
    PAD_TOKEN_ID = tokenizer.token_to_id(PAD_TOKEN)
    for i, batch in enumerate(dataset.take(max_batches)):
        inputs, labels = batch
        logits = model(inputs, training=False)
        # Clip logits for numerical stability
        logits = tf.clip_by_value(logits, -50.0, 50.0)
        log_probs = tf.nn.log_softmax(logits, axis=-1)
        # Compute per-token loss
        per_token_loss = -tf.gather(log_probs, labels, batch_dims=2)
        if tf.reduce_any(tf.math.is_nan(per_token_loss)):
            continue
        # Mask padding tokens
        mask = tf.cast(labels != PAD_TOKEN_ID, per_token_loss.dtype)
        batch_loss = tf.reduce_sum(per_token_loss * mask)
        batch_tokens = tf.reduce_sum(mask)
        if tf.math.is_nan(batch_loss) or batch_tokens == 0:
            continue
        total_loss += batch_loss.numpy()
        total_tokens += batch_tokens.numpy()
    # Compute perplexity as exponential of average loss
    if total_tokens > 0:
        avg_loss = total_loss / total_tokens
        return math.exp(min(avg_loss, 20.0))
    return float('inf')

Physical devices cannot be modified after being initialized
TensorFlow version: 2.15.0
Mixed precision policy: mixed_float16
GPU available: True


2025-10-17 12:34:45.309900: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


## Visualize the model

In [21]:
# Visualize the model
# Create model with specified vocabulary size using direct config
vocab_size = tokenizer.get_vocab_size()
# Use the optimized config directly with the same hyperparameters from create_deepseek_chatbot_model
config = DeepSeekConfig(vocab_size=vocab_size)
model = DeepSeekForCausalLM(config)


# Build model
if tf_train_dataset:
    sample_batch = next(iter(tf_train_dataset.take(1)))
    model(sample_batch[0])

# Configure optimizer with more conservative settings
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4,  # Reduced learning rate
    decay_steps=1000,
    decay_rate=0.96
)

optimizer = tf.keras.optimizers.AdamW(
    learning_rate=lr_schedule,
    beta_1=0.9,
    beta_2=0.95,
    weight_decay=0.1,
    clipnorm=1.0
)

model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, 
        ignore_class=PAD_TOKEN_ID,  # Use correct PAD token ID
        reduction=tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE
    ),
    metrics=['accuracy']
)

print("\nModel Summary")
model.summary()


Model Summary
Model: "deep_seek_for_causal_lm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (DeepSeekModel  multiple                  3375624   
 )                                                               
                                                                 
 lm_head (Dense)             multiple                  292096    
                                                                 
Total params: 3667720 (13.99 MB)
Trainable params: 3536640 (13.49 MB)
Non-trainable params: 131080 (512.03 KB)
_________________________________________________________________


## Train Model

In [35]:
# Collect training and results storage
results_table = {
    'Average TPS': {},
    'Final Training Loss': {},
    'Final Validation Loss': {},
    'Computational Resource Usage': {},
    'Average CPU Usage (percent)': {},
    'Average GPU Usage (percent)': {},
    'Average Memory (GB)': {},
    'Average GPU Memory (GB)': {},
    'Average FLOPS Estimate (GFLOPS)': {},
    'Final Validation Perplexity': {},
    'Average MaxVIO_global': {},
    'Lowest MaxVIO_global': {},
    'Highest MaxVIO_global': {},
}

best_perplexity = float('inf')
BEST_MODEL_PATH = 'best_deepseek_chatbot_model'

# Create model with specified vocabulary size using direct config
vocab_size = tokenizer.get_vocab_size()
# Use the optimized config directly with the same hyperparameters from create_deepseek_chatbot_model
config = DeepSeekConfig(vocab_size=vocab_size)
model = DeepSeekForCausalLM(config)

# Build model with sample batch
if tf_train_dataset:
    sample_batch = next(iter(tf_train_dataset.take(1)))
    model(sample_batch[0])

# Configure optimizer with exponential decay learning rate
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=1000,
    decay_rate=0.9
)

optimizer = tf.keras.optimizers.AdamW(
    learning_rate=lr_schedule,
    beta_1=0.9,
    beta_2=0.95,
    weight_decay=0.01,
    clipnorm=1.0
)

PAD_TOKEN_ID = tokenizer.token_to_id(PAD_TOKEN)

# Compile model with sparse categorical crossentropy loss
model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, 
        ignore_class=PAD_TOKEN_ID
    ),
    metrics=['accuracy']
)

# Define training callbacks
metrics_callback = DeepSeekMetrics(tf_val_dataset, tokenizer) if tf_val_dataset else None
resource_monitor = ResourceMonitor(batch_size=2)

callbacks = [
    tf.keras.callbacks.EarlyStopping(
        patience=3, 
        restore_best_weights=True, 
        monitor='val_loss',
        min_delta=0.01
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        factor=0.5, 
        patience=2, 
        monitor='val_loss',
        min_delta=0.01,
        min_lr=1e-6
    ),
    tf.keras.callbacks.TerminateOnNaN(),
]

if metrics_callback:
    callbacks.append(metrics_callback)
if resource_monitor:
    callbacks.append(resource_monitor)

# Train model on GPU
print("Starting training...")
start_time = time.time()

# Force GPU for training the model
with tf.device('/GPU:0'):
    history = model.fit(
        tf_train_dataset,
        epochs=100,
        validation_data=tf_val_dataset,
        callbacks=callbacks,
        verbose=1
    )
total_training_time = time.time() - start_time

# Calculate final metrics
print("Calculating final perplexity")
final_perplexity = calculate_perplexity(model, tf_val_dataset, tokenizer, max_batches=50) if tf_val_dataset else float('inf')
final_training_loss = history.history['loss'][-1] if 'loss' in history.history else float('inf')
final_validation_loss = history.history['val_loss'][-1] if 'val_loss' in history.history else float('inf')

print(f"Final Training Loss: {final_training_loss:.4f}")
print(f"Final Validation Loss: {final_validation_loss:.4f}")
print(f"Final Perplexity: {final_perplexity:.2f}")

# Save best model based on perplexity
if final_perplexity < best_perplexity:
    best_perplexity = final_perplexity
    best_iteration = iteration + 1

    try:
        if os.path.exists(BEST_MODEL_PATH):
            shutil.rmtree(BEST_MODEL_PATH)
        model.save(BEST_MODEL_PATH, save_format='tf')
        print(f"New best model saved from iteration {iteration + 1} with perplexity: {final_perplexity:.2f}")
    except Exception as e:
        print(f"Error saving best model: {e}")

# Collect and store resource metrics
if resource_monitor.resource_metrics:
    avg_gpu_memory = np.mean([m['avg_gpu_memory_gb'] for m in resource_monitor.resource_metrics])
    avg_system_memory = np.mean([m['avg_system_memory_gb'] for m in resource_monitor.resource_metrics])
    avg_gpu_usage = np.mean([m['avg_gpu_usage_percent'] for m in resource_monitor.resource_metrics])
    avg_cpu_usage = np.mean([m['avg_cpu_usage_percent'] for m in resource_monitor.resource_metrics])
    avg_tps = np.mean([m['tokens_per_second'] for m in resource_monitor.resource_metrics])
else:
    avg_gpu_memory = avg_system_memory = avg_gpu_usage = avg_cpu_usage = avg_tps = 0

# Calculate FLOPs
total_params = model.count_params()
batch_size = 2
seq_length = max_seq_len
flops_per_token = total_params * 2
total_flops = flops_per_token * batch_size * seq_length
total_gflops = total_flops / 1e9

# Collect MaxVIO metrics
if metrics_callback and hasattr(metrics_callback, 'max_vio_values'):
    max_vio_values = metrics_callback.max_vio_values
    if max_vio_values:
        avg_max_vio = np.mean(max_vio_values)
        min_max_vio = np.min(max_vio_values)
        max_max_vio = np.max(max_vio_values)
    else:
        avg_max_vio = min_max_vio = max_max_vio = 0.0
else:
    avg_max_vio = min_max_vio = max_max_vio = 0.0

# Store results in results table
iteration_key = f'Initial Training Result'
results_table['Average TPS'][iteration_key] = avg_tps
results_table['Final Training Loss'][iteration_key] = final_training_loss
results_table['Final Validation Loss'][iteration_key] = final_validation_loss
results_table['Computational Resource Usage'][iteration_key] = avg_cpu_usage + avg_gpu_usage
results_table['Average CPU Usage (percent)'][iteration_key] = avg_cpu_usage
results_table['Average GPU Usage (percent)'][iteration_key] = avg_gpu_usage
results_table['Average Memory (GB)'][iteration_key] = avg_system_memory
results_table['Average GPU Memory (GB)'][iteration_key] = avg_gpu_memory
results_table['Average FLOPS Estimate (GFLOPS)'][iteration_key] = total_gflops
results_table['Final Validation Perplexity'][iteration_key] = final_perplexity
results_table['Average MaxVIO_global'][iteration_key] = avg_max_vio
results_table['Lowest MaxVIO_global'][iteration_key] = min_max_vio
results_table['Highest MaxVIO_global'][iteration_key] = max_max_vio

print(f"Total Training Time {total_training_time:.2f} seconds")
print(f"Average TPS: {avg_tps:.0f}")

# Create and display results DataFrame
final_result = pd.DataFrame(results_table)
final_result = final_result.T

print("\nTraining Results Summary:")
final_result

Starting training...
55/55 [==============================] - ETA: 0s - loss: 6.6069 - accuracy: 0.0168Epoch 1: Loss=6.6069, Val_Loss=6.1543, GFLOPs=2.02, MaxVIO=0.0327
Epoch 1 Resources: GPU Mem: 14.3GB, System Mem: 9.0GB, GPU: 11.0%, CPU: 29.3%, TPS: 222
55/55 [==============================] - 35s 151ms/step - loss: 6.6069 - accuracy: 0.0168 - val_loss: 6.1543 - val_accuracy: 0.0128 - lr: 9.9433e-05 - gflops: 2.0189 - max_vio: 0.0327
Calculating final perplexity
Final Training Loss: 6.6069
Final Validation Loss: 6.1543
Final Perplexity: 488.53
New best model saved from iteration 2 with perplexity: 488.53
Total Training Time 34.64 seconds
Average TPS: 222

Training Results Summary:


,Initial Training Result
Average TPS,222.385694
Final Training Loss,6.606869
Final Validation Loss,6.154297
Computational Resource Usage,40.300000
Average CPU Usage (percent),29.300000
Average GPU Usage (percent),11.000000
Average Memory (GB),9.004910
Average GPU Memory (GB),14.274414
Average FLOPS Estimate (GFLOPS),2.018874
Final Validation Perplexity,488.532622


# Hyperparameters Tuning

## Model Architecture

In [24]:
# Disable XLA (XLA JIT compilation) to prevent CUDA graph errors and ensure stability
tf.config.optimizer.set_jit(False)  # Disabled XLA to prevent CUDA graph errors

# Configure GPU memory growth to prevent memory allocation issues
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Enable mixed precision training with FP16 for improved performance and reduced memory usage
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

# Define global configuration parameters for the model and preprocessing pipeline
vocab_size = 12000
max_seq_len = 70
batch_size = 2
PAD_TOKEN = "<pad>"
BOS_TOKEN = "<bos>"
EOS_TOKEN = "<eos>"
SEP_TOKEN = "<sep>"

# Print TensorFlow version, mixed precision policy, and GPU availability for debugging
print(f"TensorFlow version: {tf.__version__}")
print(f"Mixed precision policy: {mixed_precision.global_policy().name}")
print(f"GPU available: {len(gpus) > 0}")

# DEEPSEEK MOE COMPONENTS

class DeepSeekRMSNorm(layers.Layer):
    """RMS Normalization layer"""
    def __init__(self, hidden_size, eps=1e-6, **kwargs):
        super().__init__(**kwargs)
        self.hidden_size = hidden_size  # Dimension of the hidden representations
        self.variance_epsilon = eps     # Small constant to avoid division by zero
        
    def build(self, input_shape):
        # Initialize learnable weight parameter for scaling normalized output
        self.weight = self.add_weight(
            name='weight',
            shape=(self.hidden_size,),
            initializer='ones',
            trainable=True
        )
        super().build(input_shape)
        
    def call(self, x):
        input_dtype = x.dtype
        x = tf.cast(x, tf.float32)  # Cast input to float32 for stable computation
        
        # Compute variance as mean of squared inputs along the last dimension
        variance = tf.reduce_mean(tf.square(x), axis=-1, keepdims=True)
        # Normalize input using RMS formula with epsilon for numerical stability
        x = x * tf.math.rsqrt(variance + self.variance_epsilon)
        
        # Scale normalized output with learnable weight and cast back to original dtype
        return self.weight * tf.cast(x, input_dtype)

class RotaryPositionEmbedding(layers.Layer):
    """Rotary Position Embedding (RoPE) with dynamic NTK scaling for positional encoding"""
    def __init__(self, dim, max_position_embeddings=2048, base=10000, scaling_type=None, scaling_factor=1.0, **kwargs):
        super().__init__(**kwargs)
        self.dim = dim                                          # Dimension of the head embeddings
        self.max_position_embeddings = max_position_embeddings  # Maximum sequence length
        self.base = base                                        # Base value for frequency calculation
        self.scaling_type = scaling_type                        # Type of scaling ("linear" or "dynamic")
        self.scaling_factor = scaling_factor                    # Scaling factor for RoPE
        
    def build(self, input_shape):
        # Compute inverse frequencies for rotary embeddings
        inv_freq = 1.0 / (self.base ** (tf.range(0, self.dim, 2, dtype=tf.float32) / tf.cast(self.dim, tf.float32)))
        self.inv_freq = tf.constant(inv_freq, dtype=tf.float32)
        self._compute_rotary_cache()                            # Precompute cosine and sine caches
        super().build(input_shape)
        
    def _compute_rotary_cache(self):
        """Compute cosine and sine caches for rotary embeddings with optional scaling"""
        if self.scaling_type == "linear":
            # Linear scaling: divide positions by scaling factor
            t = tf.range(self.max_position_embeddings, dtype=tf.float32) / self.scaling_factor
        elif self.scaling_type == "dynamic":
            # Dynamic NTK scaling for extended context lengths
            if self.max_position_embeddings > self.base:
                base = self.base * (
                    (self.scaling_factor * self.max_position_embeddings / self.base) - 
                    (self.scaling_factor - 1)
                ) ** (self.dim / (self.dim - 2))
                inv_freq = 1.0 / (base ** (tf.range(0, self.dim, 2, dtype=tf.float32) / tf.cast(self.dim, tf.float32)))
                self.inv_freq = tf.constant(inv_freq, dtype=tf.float32)
            t = tf.range(self.max_position_embeddings, dtype=tf.float32)
        else:
            # Default case: no scaling
            t = tf.range(self.max_position_embeddings, dtype=tf.float32)
            
        # Compute frequency matrix for rotary embeddings
        freqs = tf.einsum('i,j->ij', t, self.inv_freq)
        emb = tf.concat([freqs, freqs], axis=-1)                # Concatenate for full embedding dimension
        
        # Cache cosine and sine values as non-trainable variables
        self.cos_cached = tf.Variable(tf.cos(emb), trainable=False, name='cos_cached')
        self.sin_cached = tf.Variable(tf.sin(emb), trainable=False, name='sin_cached')
        
    def call(self, x, position_ids=None, seq_len=None):
        # Get input tensor dimensions
        batch_size, seq_len_x, num_heads, head_dim = tf.unstack(tf.shape(x))
        
        # Generate position IDs if not provided
        if position_ids is None:
            position_ids = tf.range(seq_len_x, dtype=tf.int32)
            position_ids = tf.tile(tf.reshape(position_ids, [1, -1]), [batch_size, 1])
        
        # Gather precomputed cosine and sine values for given position IDs
        cos = tf.gather(self.cos_cached, position_ids, axis=0)  # Shape: [batch_size, seq_len_x, head_dim]
        sin = tf.gather(self.sin_cached, position_ids, axis=0)  # Shape: [batch_size, seq_len_x, head_dim]
        
        # Split cosine and sine for application to x1 and x2
        cos_1, cos_2 = tf.split(cos, 2, axis=-1)
        sin_1, sin_2 = tf.split(sin, 2, axis=-1)
        
        # Reshape for broadcasting with attention heads
        cos_1 = tf.reshape(cos_1, [batch_size, seq_len_x, 1, head_dim // 2])
        sin_1 = tf.reshape(sin_1, [batch_size, seq_len_x, 1, head_dim // 2])
        cos_2 = tf.reshape(cos_2, [batch_size, seq_len_x, 1, head_dim // 2])
        sin_2 = tf.reshape(sin_2, [batch_size, seq_len_x, 1, head_dim // 2])
        
        # Repeat to match number of attention heads
        cos_1 = tf.repeat(cos_1, num_heads, axis=2)
        sin_1 = tf.repeat(sin_1, num_heads, axis=2)
        cos_2 = tf.repeat(cos_2, num_heads, axis=2)
        sin_2 = tf.repeat(sin_2, num_heads, axis=2)
        
        # Cast to input dtype for mixed precision compatibility
        x_dtype = x.dtype
        cos_1 = tf.cast(cos_1, x_dtype)
        sin_1 = tf.cast(sin_1, x_dtype)
        cos_2 = tf.cast(cos_2, x_dtype)
        sin_2 = tf.cast(sin_2, x_dtype)
        
        # Split input tensor into two parts for rotary application
        x1, x2 = tf.split(x, 2, axis=-1)
        
        # Apply rotary embeddings to compute rotated representations
        rotated_x1 = x1 * cos_1 - x2 * sin_1
        rotated_x2 = x1 * sin_2 + x2 * cos_2
        
        # Concatenate rotated parts to restore original shape
        return tf.concat([rotated_x1, rotated_x2], axis=-1)

class DeepSeekMLP(layers.Layer):
    """Multi-Layer Perceptron (MLP) with Swish activation"""
    def __init__(self, hidden_size, intermediate_size, hidden_act="swish", **kwargs):
        super().__init__(**kwargs)
        self.hidden_size = hidden_size              # Input and output dimension
        self.intermediate_size = intermediate_size  # Intermediate layer dimension
        self.hidden_act = hidden_act                # Activation function (Swish/SILU)
        
    def build(self, input_shape):
        # Define gate projection layer with reduced initializer variance for stability
        self.gate_proj = layers.Dense(
            self.intermediate_size,
            kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.001),
            use_bias=False
        )
        # Define up projection layer
        self.up_proj = layers.Dense(
            self.intermediate_size,
            kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.001),
            use_bias=False
        )
        # Define down projection layer to return to hidden size
        self.down_proj = layers.Dense(
            self.hidden_size,
            kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.001),
            use_bias=False
        )
        super().build(input_shape)
        
    def call(self, x):
        # Apply gate and up projections
        gate_output = self.gate_proj(x)
        up_output = self.up_proj(x)
        # Apply Swish (SiLU) activation to gate output and multiply with up output
        activated = tf.nn.silu(gate_output) * up_output
        # Project back to hidden size
        return self.down_proj(activated)

class MoEGate(layers.Layer):
    """Mixture of Experts (MoE) gating mechanism with load balancing auxiliary loss"""
    def __init__(self, num_experts, num_experts_per_tok, hidden_size, 
                 scoring_func='softmax', norm_topk_prob=False, 
                 aux_loss_alpha=0.001, seq_aux=True, **kwargs):
        super().__init__(**kwargs)
        self.num_experts = num_experts                  # Total number of experts
        self.num_experts_per_tok = num_experts_per_tok  # Number of experts per token
        self.hidden_size = hidden_size                  # Input dimension
        self.scoring_func = scoring_func                # Scoring function for expert selection
        self.norm_topk_prob = norm_topk_prob            # Normalize top-k probabilities
        self.aux_loss_alpha = aux_loss_alpha            # Weight for auxiliary loss
        self.seq_aux = seq_aux                          # Compute auxiliary loss per sequence
        
    def build(self, input_shape):
        # Initialize gating weights with reduced variance for stability
        self.weight = self.add_weight(
            name='gate_weight',
            shape=(self.num_experts, self.hidden_size),
            initializer=tf.keras.initializers.RandomNormal(stddev=0.001),
            trainable=True
        )
        # Initialize expert assignment counter for tracking during training
        self.expert_counts = tf.Variable(tf.zeros([self.num_experts], dtype=tf.int32), trainable=False)
        super().build(input_shape)
        
    def call(self, hidden_states, training=False):
        # Get input dimensions
        batch_size, seq_len, h = tf.unstack(tf.shape(hidden_states))
        hidden_states_flat = tf.reshape(hidden_states, [-1, h])
        # Compute gating logits
        logits = tf.matmul(hidden_states_flat, self.weight, transpose_b=True)
        
        # Clip logits for numerical stability
        logits = tf.clip_by_value(logits, -50.0, 50.0)
        
        # Apply scoring function (softmax) to compute expert scores
        if self.scoring_func == 'softmax':
            temperature = 0.5  # Temperature for scaling logits
            scaled_logits = tf.cast(logits, tf.float32) / temperature
            scores = tf.nn.softmax(scaled_logits, axis=-1)
            scores = tf.cast(scores, hidden_states.dtype)
        else:
            scores = tf.nn.softmax(tf.cast(logits, tf.float32), axis=-1)
            scores = tf.cast(scores, hidden_states.dtype)

        # Select top-k experts and their weights
        topk_weights, topk_indices = tf.math.top_k(
            scores, k=self.num_experts_per_tok, sorted=False
        )

        # Normalize top-k weights if specified
        if self.num_experts_per_tok > 1 and self.norm_topk_prob:
            denominator = tf.reduce_sum(topk_weights, axis=-1, keepdims=True) + 1e-20
            topk_weights = topk_weights / denominator

        # Update expert assignment counts during training
        if training:
            flat_topk_indices = tf.reshape(topk_indices, [-1])
            counts = tf.reduce_sum(tf.one_hot(flat_topk_indices, depth=self.num_experts), axis=0)
            self.expert_counts.assign_add(tf.cast(counts, tf.int32))

        aux_loss = None
        if training and self.aux_loss_alpha > 0.0:
            if self.seq_aux:
                # Sequence-level auxiliary loss for load balancing
                scores_seq = tf.reshape(scores, [batch_size, seq_len, -1])
                topk_indices_seq = tf.reshape(topk_indices, [batch_size, -1])
                ce = tf.zeros([batch_size, self.num_experts], dtype=tf.float32)
                mask = tf.one_hot(topk_indices_seq, depth=self.num_experts)
                ce = tf.reduce_sum(mask, axis=1)
                seq_len_float = tf.cast(seq_len, tf.float32)
                num_experts_per_tok_float = tf.cast(self.num_experts_per_tok, tf.float32)
                num_experts_float = tf.cast(self.num_experts, tf.float32)
                ce = ce / (seq_len_float * num_experts_per_tok_float / num_experts_float)
                scores_seq_mean = tf.reduce_mean(tf.cast(scores_seq, tf.float32), axis=1)
                aux_loss = tf.reduce_sum(ce * scores_seq_mean, axis=1)
                aux_loss = tf.reduce_mean(aux_loss) * self.aux_loss_alpha
            else:
                # Token-level auxiliary loss
                mask_ce = tf.one_hot(tf.reshape(topk_indices, [-1]), depth=self.num_experts)
                ce = tf.reduce_mean(tf.cast(mask_ce, tf.float32), axis=0)
                Pi = tf.reduce_mean(tf.cast(scores, tf.float32), axis=0)
                fi = ce * self.num_experts
                aux_loss = tf.reduce_sum(Pi * fi) * self.aux_loss_alpha

            # Add auxiliary loss to the layer's losses
            self.add_loss(aux_loss)

        return topk_indices, topk_weights, aux_loss

class DeepSeekMoE(layers.Layer):
    """Mixture of Experts (MoE) layer with shared and routed experts"""
    def __init__(self, hidden_size, n_routed_experts, n_shared_experts, 
                 num_experts_per_tok, moe_intermediate_size, **kwargs):
        super().__init__(**kwargs)
        self.hidden_size = hidden_size                      # Input and output dimension
        self.n_routed_experts = n_routed_experts            # Number of routed experts
        self.n_shared_experts = n_shared_experts            # Number of shared experts
        self.num_experts_per_tok = num_experts_per_tok      # Experts per token
        self.moe_intermediate_size = moe_intermediate_size  # MoE intermediate size
        
    def build(self, input_shape):
        # Initialize list of routed expert MLPs
        self.routed_experts = [
            DeepSeekMLP(
                self.hidden_size, 
                self.moe_intermediate_size,
                name=f"routed_expert_{i}"
            ) for i in range(self.n_routed_experts)
        ]
        
        # Initialize shared experts if specified
        if self.n_shared_experts is not None:
            shared_intermediate_size = self.moe_intermediate_size * self.n_shared_experts
            self.shared_experts = DeepSeekMLP(
                self.hidden_size,
                shared_intermediate_size,
                name="shared_experts"
            )
        else:
            self.shared_experts = None
            
        # Initialize MoE gating mechanism
        self.gate = MoEGate(
            self.n_routed_experts,
            self.num_experts_per_tok,
            self.hidden_size,
            name="moe_gate"
        )
        
        # Build each expert and gate
        for expert in self.routed_experts:
            expert.build(input_shape)
        if self.shared_experts:
            self.shared_experts.build(input_shape)
        self.gate.build(input_shape)
        
        super().build(input_shape)
        
    def call(self, hidden_states, training=False):
        identity = hidden_states  # Store input for residual connection
        orig_shape = tf.shape(hidden_states)
        # Get top-k expert indices and weights
        topk_idx, topk_weight, aux_loss = self.gate(hidden_states, training=training)
        hidden_states_flat = tf.reshape(hidden_states, [-1, self.hidden_size])
        
        if training:
            # Expand inputs for top-k experts during training
            hidden_states_expanded = tf.repeat(
                hidden_states_flat, self.num_experts_per_tok, axis=0
            )
            flat_topk_idx = tf.reshape(topk_idx, [-1])
            combined_output = tf.zeros_like(hidden_states_expanded)
            # Process each expert's input
            for i, expert in enumerate(self.routed_experts):
                expert_mask = tf.equal(flat_topk_idx, i)
                if tf.reduce_any(expert_mask):
                    expert_input = tf.boolean_mask(hidden_states_expanded, expert_mask)
                    expert_output = expert(expert_input)
                    update_indices = tf.where(expert_mask)
                    combined_output = tf.tensor_scatter_nd_update(
                        combined_output,
                        update_indices,
                        expert_output
                    )
                    
            # Apply expert weights and reshape output
            topk_weight_flat = tf.reshape(topk_weight, [-1, 1])
            weighted_output = combined_output * topk_weight_flat
            weighted_output = tf.reshape(
                weighted_output,
                [-1, self.num_experts_per_tok, self.hidden_size]
            )
            routed_output = tf.reduce_sum(weighted_output, axis=1)
            routed_output = tf.reshape(routed_output, orig_shape)
        else:
            # Use efficient inference path
            routed_output = self._moe_infer(hidden_states_flat, topk_idx, topk_weight)
            routed_output = tf.reshape(routed_output, orig_shape)
            
        # Add shared expert output if available
        if self.shared_experts is not None:
            shared_output = self.shared_experts(identity)
            final_output = routed_output + shared_output
        else:
            final_output = routed_output
            
        # Apply residual connection
        return final_output + identity
        
    def _moe_infer(self, x, expert_indices, expert_weights):
        """Efficient inference path for MoE layer"""
        batch_size = tf.shape(x)[0]
        expert_cache = tf.zeros_like(x)
        # Clip expert indices to valid range
        expert_indices = tf.clip_by_value(expert_indices, 0, self.n_routed_experts - 1)
        flat_expert_indices = tf.reshape(expert_indices, [-1])
        flat_expert_weights = tf.reshape(expert_weights, [-1, 1])
        total_tokens = tf.shape(x)[0]
        token_indices_base = tf.range(total_tokens, dtype=tf.int64)
        token_indices_base = tf.repeat(token_indices_base, self.num_experts_per_tok, axis=0)
        
        # Process each expert's tokens
        for i in range(self.n_routed_experts):
            expert_mask = tf.equal(flat_expert_indices, i)
            if tf.reduce_any(expert_mask):
                token_indices = tf.boolean_mask(token_indices_base, expert_mask)
                expert_tokens = tf.gather(x, token_indices)
                expert_out = self.routed_experts[i](expert_tokens)
                expert_out_weighted = expert_out * tf.gather(flat_expert_weights, tf.where(expert_mask)[:, 0])
                expert_cache = tf.tensor_scatter_nd_add(
                    expert_cache,
                    tf.expand_dims(token_indices, 1),
                    expert_out_weighted
                )
                
        return expert_cache

class DeepSeekAttention(layers.Layer):
    """Multi-head attention with Rotary Position Embeddings and Grouped Query Attention (GQA) support"""
    def __init__(self, hidden_size, num_attention_heads, num_key_value_heads=None,
                 attention_dropout=0.0, max_position_embeddings=2048,
                 rope_theta=10000.0, rope_scaling=None, attention_bias=False, **kwargs):
        super().__init__(**kwargs)
        self.hidden_size = hidden_size                                         # Input and output dimension
        self.num_attention_heads = num_attention_heads                         # Number of attention heads
        self.num_key_value_heads = num_key_value_heads or num_attention_heads  # Number of key/value heads (GQA)
        self.attention_dropout = attention_dropout                             # Dropout rate for attention weights
        self.max_position_embeddings = max_position_embeddings                 # Maximum sequence length
        self.rope_theta = rope_theta                                           # Base period for RoPE
        self.rope_scaling = rope_scaling                                       # RoPE scaling configuration
        self.attention_bias = attention_bias                                   # Whether to use bias in projections
        self.head_dim = hidden_size // num_attention_heads                     # Dimension per head
        self.num_key_value_groups = num_attention_heads // self.num_key_value_heads  # Groups for GQA
        
    def build(self, input_shape):
        # Initialize query projection layer
        self.q_proj = layers.Dense(
            self.num_attention_heads * self.head_dim,
            use_bias=self.attention_bias,
            kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.001),
            name="q_proj"
        )
        # Initialize key projection layer
        self.k_proj = layers.Dense(
            self.num_key_value_heads * self.head_dim,
            use_bias=self.attention_bias,
            kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.001),
            name="k_proj"
        )
        # Initialize value projection layer
        self.v_proj = layers.Dense(
            self.num_key_value_heads * self.head_dim,
            use_bias=self.attention_bias,
            kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.001),
            name="v_proj"
        )
        # Initialize output projection layer
        self.o_proj = layers.Dense(
            self.hidden_size,
            use_bias=self.attention_bias,
            kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.001),
            name="o_proj"
        )
        
        # Initialize RoPE with specified scaling
        scaling_type = self.rope_scaling.get("type") if self.rope_scaling else None
        scaling_factor = self.rope_scaling.get("factor", 1.0) if self.rope_scaling else 1.0
        self.rotary_emb = RotaryPositionEmbedding(
            self.head_dim,
            self.max_position_embeddings,
            self.rope_theta,
            scaling_type,
            scaling_factor,
            name="rotary_emb"
        )
        self.dropout_layer = layers.Dropout(self.attention_dropout)
        self.rotary_emb.build(input_shape)
        super().build(input_shape)
        
    def call(self, hidden_states, attention_mask=None, position_ids=None, training=False):
        # Get input dimensions
        batch_size, seq_len, _ = tf.unstack(tf.shape(hidden_states))
        
        # Project inputs to query, key, and value
        q = self.q_proj(hidden_states)
        k = self.k_proj(hidden_states)
        v = self.v_proj(hidden_states)
        
        # Reshape for multi-head attention
        q = tf.reshape(q, [batch_size, seq_len, self.num_attention_heads, self.head_dim])
        k = tf.reshape(k, [batch_size, seq_len, self.num_key_value_heads, self.head_dim])
        v = tf.reshape(v, [batch_size, seq_len, self.num_key_value_heads, self.head_dim])
        
        # Apply rotary embeddings to query and key
        q_rotated = self.rotary_emb(q, position_ids=position_ids)
        k_rotated = self.rotary_emb(k, position_ids=position_ids)
        
        # Transpose for attention computation
        q_rotated = tf.transpose(q_rotated, [0, 2, 1, 3])
        k_rotated = tf.transpose(k_rotated, [0, 2, 1, 3])
        v = tf.transpose(v, [0, 2, 1, 3])
        
        # Repeat key and value for GQA if necessary
        if self.num_key_value_groups > 1:
            k_rotated = tf.repeat(k_rotated, self.num_key_value_groups, axis=1)
            v = tf.repeat(v, self.num_key_value_groups, axis=1)
            
        # Compute scaled dot-product attention
        scale = tf.math.rsqrt(tf.cast(self.head_dim, tf.float32))
        scale = tf.cast(scale, q_rotated.dtype)
        attn_weights = tf.matmul(q_rotated, k_rotated, transpose_b=True) * scale
        
        # Apply attention mask if provided
        if attention_mask is not None:
            attention_mask = tf.cast(attention_mask, attn_weights.dtype)
            if len(attention_mask.shape) == 2:
                attention_mask = tf.expand_dims(tf.expand_dims(attention_mask, 1), 1)
            elif len(attention_mask.shape) == 3:
                attention_mask = tf.expand_dims(attention_mask, 1)
            attn_weights += attention_mask
            
        # Clip attention weights for numerical stability
        attn_weights = tf.clip_by_value(attn_weights, -50.0, 50.0)
        attn_weights = tf.cast(attn_weights, tf.float32)
        # Apply softmax with max subtraction for stability
        attn_weights = attn_weights - tf.reduce_max(attn_weights, axis=-1, keepdims=True)
        attn_weights = tf.nn.softmax(attn_weights, axis=-1)
        attn_weights = tf.cast(attn_weights, q_rotated.dtype)
        
        # Apply dropout to attention weights during training
        attn_weights = self.dropout_layer(attn_weights, training=training)
        # Compute attention output
        attn_output = tf.matmul(attn_weights, v)
        attn_output = tf.transpose(attn_output, [0, 2, 1, 3])
        attn_output = tf.reshape(attn_output, [batch_size, seq_len, self.hidden_size])
        
        # Project attention output to hidden size
        return self.o_proj(attn_output)

class DeepSeekDecoderLayer(layers.Layer):
    """Transformer decoder layer combining attention and MoE/MLP with residual connections"""
    def __init__(self, config, layer_idx, **kwargs):
        super().__init__(**kwargs)
        self.config = config
        self.layer_idx = layer_idx
        
        # Initialize self-attention with GQA and RoPE
        self.self_attn = DeepSeekAttention(
            hidden_size=config.hidden_size,
            num_attention_heads=config.num_attention_heads,
            num_key_value_heads=config.num_key_value_heads,
            attention_dropout=config.attention_dropout,
            max_position_embeddings=config.max_position_embeddings,
            rope_theta=config.rope_theta,
            rope_scaling=config.rope_scaling,
            attention_bias=config.attention_bias,
            name="self_attn"
        )
        
        # Determine whether to use MoE or dense MLP based on layer index and config
        self.use_moe = (
            config.n_routed_experts is not None and
            layer_idx >= config.first_k_dense_replace and
            layer_idx % config.moe_layer_freq == 0
        )
        
        if self.use_moe:
            # Initialize MoE layer for routed experts
            self.mlp = DeepSeekMoE(
                hidden_size=config.hidden_size,
                n_routed_experts=config.n_routed_experts,
                n_shared_experts=config.n_shared_experts,
                num_experts_per_tok=config.num_experts_per_tok,
                moe_intermediate_size=config.moe_intermediate_size,
                name="moe"
            )
        else:
            # Initialize dense MLP layer
            self.mlp = DeepSeekMLP(
                hidden_size=config.hidden_size,
                intermediate_size=config.intermediate_size,
                hidden_act=config.hidden_act,
                name="mlp"
            )
            
        # Initialize input and post-attention normalization layers
        self.input_layernorm = DeepSeekRMSNorm(
            config.hidden_size, eps=config.rms_norm_eps, name="input_layernorm"
        )
        self.post_attention_layernorm = DeepSeekRMSNorm(
            config.hidden_size, eps=config.rms_norm_eps, name="post_attention_layernorm"
        )
        
    def call(self, hidden_states, attention_mask=None, position_ids=None, training=False):
        # Apply input normalization and self-attention with residual connection
        residual = hidden_states
        hidden_states = self.input_layernorm(hidden_states)
        attn_output = self.self_attn(
            hidden_states,
            attention_mask=attention_mask,
            position_ids=position_ids,
            training=training
        )
        hidden_states = residual + attn_output
        
        # Apply post-attention normalization and MLP/MoE with residual connection
        residual = hidden_states
        hidden_states = self.post_attention_layernorm(hidden_states)
        mlp_output = self.mlp(hidden_states, training=training)
        hidden_states = residual + mlp_output
        
        return hidden_states

# MODEL CONFIGURATION

class DeepSeekConfig:
    """Configuration class for DeepSeek model parameters with optimized chatbot hyperparameters"""
    def __init__(
        self,
        vocab_size=vocab_size,  # Default vocabulary size for chatbot
        hidden_size=256,        # Optimized hidden size for stability and performance
        intermediate_size=512,  # Balanced intermediate size for MLP layers
        moe_intermediate_size=256,  # MoE-specific intermediate dimension
        num_hidden_layers=4,    # Optimal layer count for chatbot tasks
        num_attention_heads=4,  # Balanced attention heads for computation
        num_key_value_heads=2,  # GQA for memory efficiency
        n_shared_experts=1,     # Single shared expert for parameter efficiency
        n_routed_experts=4,     # Four routed experts for specialization
        num_experts_per_tok=2,  # Two experts per token for balanced load
        moe_layer_freq=2,       # MoE every other layer for computational balance
        first_k_dense_replace=1,  # First layer dense for stability
        norm_topk_prob=False,   # No normalization for expert weights
        scoring_func='softmax', # Standard softmax for expert selection
        aux_loss_alpha=0.001,   # Default auxiliary loss weight
        seq_aux=True,           # Sequence-level auxiliary loss
        hidden_act="swish",     # Swish activation for better gradients
        max_position_embeddings=256,  # Sufficient context length for chatbot
        initializer_range=0.001,  # Reduced initializer range for stability
        rms_norm_eps=1e-6,      # Standard RMS norm epsilon
        use_cache=True,         # Enable caching for inference
        rope_theta=10000.0,     # Standard RoPE base period
        rope_scaling={"type": "linear", "factor": 2.0},  # Linear RoPE scaling
        attention_bias=False,   # No attention bias for efficiency
        attention_dropout=0.1   # Increased dropout for regularization
    ):
        # Initialize model architecture parameters with optimized values
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.intermediate_size = intermediate_size
        self.moe_intermediate_size = moe_intermediate_size
        self.num_hidden_layers = num_hidden_layers
        self.num_attention_heads = num_attention_heads
        self.num_key_value_heads = num_key_value_heads
        self.n_shared_experts = n_shared_experts
        self.n_routed_experts = n_routed_experts
        self.num_experts_per_tok = num_experts_per_tok
        self.moe_layer_freq = moe_layer_freq
        self.first_k_dense_replace = first_k_dense_replace
        self.norm_topk_prob = norm_topk_prob
        self.scoring_func = scoring_func
        self.aux_loss_alpha = aux_loss_alpha
        self.seq_aux = seq_aux
        self.hidden_act = hidden_act
        self.max_position_embeddings = max_position_embeddings
        self.initializer_range = initializer_range
        self.rms_norm_eps = rms_norm_eps
        self.use_cache = use_cache
        self.rope_theta = rope_theta
        self.rope_scaling = rope_scaling
        self.attention_bias = attention_bias
        self.attention_dropout = attention_dropout

class DeepSeekModel(tf.keras.Model):
    """Core DeepSeek transformer model for chatbot, provide raw hidden states"""
    def __init__(self, config, **kwargs):
        super().__init__(**kwargs)
        self.config = config
        self.padding_idx = 0
        
        # Initialize embedding layer for token inputs
        self.embed_tokens = layers.Embedding(
            config.vocab_size,
            config.hidden_size,
            embeddings_initializer=tf.keras.initializers.RandomNormal(
                stddev=config.initializer_range
            ),
            name="embed_tokens"
        )
        
        # Initialize decoder layers
        self.decoder_layers = [
            DeepSeekDecoderLayer(config, layer_idx=i, name=f"layer_{i}")
            for i in range(config.num_hidden_layers)
        ]
        
        # Initialize final normalization layer
        self.norm = DeepSeekRMSNorm(
            config.hidden_size, eps=config.rms_norm_eps, name="norm"
        )
        
        # Initialize expert assignment counter for MoE layers
        self.expert_assignment_counts = tf.Variable(
            tf.zeros([config.n_routed_experts], dtype=tf.int32),
            trainable=False,
            name="expert_assignment_counts"
        ) if config.n_routed_experts else None
        
    def call(self, input_ids, attention_mask=None, position_ids=None, training=False):
        # Embed input tokens
        hidden_states = self.embed_tokens(input_ids)
        
        # Create causal mask if not provided
        if attention_mask is None:
            seq_len = tf.shape(input_ids)[1]
            attention_mask = self._create_causal_mask(seq_len)
            
        # Prepare attention mask for causal attention
        attention_mask = tf.cast(attention_mask, hidden_states.dtype)
        if len(attention_mask.shape) == 2:
            attention_mask = tf.expand_dims(tf.expand_dims(attention_mask, 1), 1)
        elif len(attention_mask.shape) == 3:
            attention_mask = tf.expand_dims(attention_mask, 1)
        attention_mask = (tf.cast(1.0, hidden_states.dtype) - attention_mask) * tf.cast(-1e4, hidden_states.dtype)
        
        # Process through decoder layers and aggregate expert counts
        for layer in self.decoder_layers:
            hidden_states = layer(
                hidden_states,
                attention_mask=attention_mask,
                position_ids=position_ids,
                training=training
            )
            # Update expert assignment counts for MoE layers during training
            if training and hasattr(layer, 'mlp') and isinstance(layer.mlp, DeepSeekMoE):
                expert_counts = layer.mlp.gate.expert_counts
                if self.expert_assignment_counts is not None:
                    self.expert_assignment_counts.assign_add(expert_counts)
            
        # Apply final normalization
        return self.norm(hidden_states)
        
    def _create_causal_mask(self, seq_len):
        """Create causal attention mask to prevent attending to future tokens"""
        mask = tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
        return tf.cast(mask, tf.float16)

class DeepSeekForCausalLM(tf.keras.Model):
    """Causal language model for chatbot text generation"""
    def __init__(self, config, **kwargs):
        super().__init__(**kwargs)
        self.config = config
        self.transformer = DeepSeekModel(config, name="transformer")
        # Initialize language model head
        self.lm_head = layers.Dense(
            config.vocab_size,
            kernel_initializer=tf.keras.initializers.RandomNormal(
                stddev=config.initializer_range
            ),
            use_bias=False,
            name="lm_head"
        )
        
    def call(self, inputs, attention_mask=None, position_ids=None, training=False):
        # Handle dictionary or tensor inputs
        if isinstance(inputs, dict):
            input_ids = inputs.get('input_ids')
            attention_mask = inputs.get('attention_mask', attention_mask)
        else:
            input_ids = inputs
        
        # Process through transformer
        hidden_states = self.transformer(
            input_ids,
            attention_mask=attention_mask,
            position_ids=position_ids,
            training=training
        )
        # Generate logits for token prediction
        return self.lm_head(hidden_states)

# TRAINING METRICS AND CALLBACKS 
class ResourceMonitor(tf.keras.callbacks.Callback):
    """Callback to monitor system resources (GPU, CPU, memory) during training"""
    def __init__(self, batch_size=2):
        super().__init__()
        self.batch_size = batch_size
        self.resource_metrics = []  # Store resource usage metrics
        self.start_time = time.time()
        self.total_tokens_processed = 0
        
    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()
        self.epoch_tokens = 0
        
    def on_batch_end(self, batch, logs=None):
        # Update token count for throughput calculation
        self.total_tokens_processed += self.batch_size * max_seq_len
        self.epoch_tokens += self.batch_size * max_seq_len
        
    def on_epoch_end(self, epoch, logs=None):
        try:
            # Collect GPU and system resource usage
            gpus = GPUtil.getGPUs()
            gpu_memory = sum([gpu.memoryUsed for gpu in gpus]) / len(gpus) if gpus else 0
            gpu_usage = sum([gpu.load * 100 for gpu in gpus]) / len(gpus) if gpus else 0
            cpu_usage = psutil.cpu_percent()
            memory_info = psutil.virtual_memory()
            system_memory = memory_info.used / (1024 ** 3)
            epoch_time = time.time() - self.epoch_start_time
            tokens_per_second = self.epoch_tokens / epoch_time if epoch_time > 0 else 0
            
            # Store resource metrics
            resource_stats = {
                'epoch': epoch + 1,
                'avg_gpu_memory_gb': float(gpu_memory / 1024),
                'avg_system_memory_gb': float(system_memory),
                'avg_gpu_usage_percent': float(gpu_usage),
                'avg_cpu_usage_percent': float(cpu_usage),
                'tokens_per_second': float(tokens_per_second),
                'throughput_tps': float(tokens_per_second)
            }
            
            self.resource_metrics.append(resource_stats)
            
            # Print resource usage summary
            print(f"Epoch {epoch+1} Resources: GPU Mem: {gpu_memory/1024:.1f}GB, "
                  f"System Mem: {system_memory:.1f}GB, GPU: {gpu_usage:.1f}%, "
                  f"CPU: {cpu_usage:.1f}%, TPS: {tokens_per_second:.0f}")
                  
        except Exception as e:
            print(f"Resource monitoring error: {e}")

class DeepSeekMetrics(tf.keras.callbacks.Callback):
    """Callback to track specific metrics, including MaxVIO for expert balancing"""
    def __init__(self, validation_data, tokenizer):
        super().__init__()
        self.validation_data = validation_data
        self.tokenizer = tokenizer
        self.metrics_history = []
        self.start_time = time.time()
        self.max_vio_values = []
        
    def on_epoch_begin(self, epoch, logs=None):
        # Reset expert assignment counts for accurate per-epoch metrics
        if hasattr(self.model.transformer, 'expert_assignment_counts') and self.model.transformer.expert_assignment_counts is not None:
            self.model.transformer.expert_assignment_counts.assign(tf.zeros([self.model.config.n_routed_experts], dtype=tf.int32))
        for layer in self.model.transformer.decoder_layers:
            if hasattr(layer, 'mlp') and isinstance(layer.mlp, DeepSeekMoE):
                layer.mlp.gate.expert_counts.assign(tf.zeros([layer.mlp.n_routed_experts], dtype=tf.int32))
        
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        
        # Estimate FLOPs for computational efficiency tracking
        total_params = self.model.count_params()
        flops_per_token = total_params * 2
        total_flops = flops_per_token * batch_size * max_seq_len
        total_gflops = total_flops / 1e9
        
        logs['gflops'] = float(total_gflops)
        
        # Calculate MaxVIO for expert load balancing if validation data is available
        if self.validation_data:
            try:
                max_vio = self._calculate_max_vio()
                logs['max_vio'] = float(max_vio)
                self.max_vio_values.append(max_vio)
                
            except Exception as e:
                print(f"Metrics calculation error: {e}")
                logs['max_vio'] = 0.0
                
        self.metrics_history.append(logs.copy())
        
        # Print simplified metrics summary
        print(f"Epoch {epoch+1}: Loss={logs.get('loss', 0):.4f}, "
              f"Val_Loss={logs.get('val_loss', 0):.4f}, "
              f"GFLOPs={total_gflops:.2f}, "
              f"MaxVIO={logs.get('max_vio', 0):.4f}")
              
    def _calculate_max_vio(self):
        """Calculate Maximum Violation (MaxVIO) for expert load balancing"""
        total_counts = np.zeros(self.model.config.n_routed_experts, dtype=np.float32)
        moe_layers = 0
        # Aggregate expert counts across MoE layers
        for layer in self.model.transformer.decoder_layers:
            if hasattr(layer, 'mlp') and isinstance(layer.mlp, DeepSeekMoE):
                counts = layer.mlp.gate.expert_counts.numpy()
                total_counts += counts
                moe_layers += 1
        
        # Compute MaxVIO based on aggregated counts
        if moe_layers > 0 and np.sum(total_counts) > 0:
            normalized_counts = total_counts / (np.sum(total_counts) + 1e-10)
            expected_uniform = 1.0 / len(total_counts)
            max_vio = np.max(np.abs(normalized_counts - expected_uniform)) / expected_uniform
            return max_vio
        return 0.0

class ComprehensiveModelCheckpoint(tf.keras.callbacks.Callback):
    """Callback for saving model checkpoints based on monitored metric"""
    def __init__(self, filepath, monitor='val_loss', save_best_only=True, mode='min'):
        super().__init__()
        self.filepath = filepath
        self.monitor = monitor
        self.save_best_only = save_best_only
        self.mode = mode
        self.best_value = float('inf') if mode == 'min' else -float('inf')
        
    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            return
            
        current_value = logs.get(self.monitor)
        if current_value is None:
            return
            
        # Determine if current model should be saved
        if self.mode == 'min':
            should_save = current_value < self.best_value
        else:
            should_save = current_value > self.best_value
            
        if should_save or not self.save_best_only:
            self.best_value = current_value
            self.model.save_weights(self.filepath)
            print(f"Model checkpoint saved with {self.monitor}: {current_value:.4f}")

# Function to calculate perplexity on validation set
def calculate_perplexity(model, dataset, tokenizer, max_batches=50):
    """Calculate perplexity on validation dataset to evaluate model performance"""
    total_loss = 0.0
    total_tokens = 0
    PAD_TOKEN_ID = tokenizer.token_to_id(PAD_TOKEN)
    for i, batch in enumerate(dataset.take(max_batches)):
        inputs, labels = batch
        logits = model(inputs, training=False)
        # Clip logits for numerical stability
        logits = tf.clip_by_value(logits, -50.0, 50.0)
        log_probs = tf.nn.log_softmax(logits, axis=-1)
        # Compute per-token loss
        per_token_loss = -tf.gather(log_probs, labels, batch_dims=2)
        if tf.reduce_any(tf.math.is_nan(per_token_loss)):
            continue
        # Mask padding tokens
        mask = tf.cast(labels != PAD_TOKEN_ID, per_token_loss.dtype)
        batch_loss = tf.reduce_sum(per_token_loss * mask)
        batch_tokens = tf.reduce_sum(mask)
        if tf.math.is_nan(batch_loss) or batch_tokens == 0:
            continue
        total_loss += batch_loss.numpy()
        total_tokens += batch_tokens.numpy()
    # Compute perplexity as exponential of average loss
    if total_tokens > 0:
        avg_loss = total_loss / total_tokens
        return math.exp(min(avg_loss, 20.0))
    return float('inf')

Physical devices cannot be modified after being initialized
TensorFlow version: 2.15.0
Mixed precision policy: mixed_float16
GPU available: True


## Define suggest hyperparameters

In [25]:
def suggest_routing_hyperparameters(trial):
    """
    Suggest hyperparameters related ONLY to routing algorithm (Top-K).
    Other parameters (hidden_size, num_attention_heads, etc.) are kept fixed.
    """
    params = {}
    
    n_shared_experts = trial.suggest_int('n_shared_experts', 1, 4)
    params['n_shared_experts'] = n_shared_experts
    
    n_routed_experts = trial.suggest_int('n_routed_experts', 2, 8)
    params['n_routed_experts'] = n_routed_experts
    
    # Ensure num_experts_per_tok doesn't exceed n_routed_experts
    max_experts_per_tok = min(4, n_routed_experts)
    num_experts_per_tok = trial.suggest_int('num_experts_per_tok', 1, max_experts_per_tok)
    params['num_experts_per_tok'] = num_experts_per_tok
    
    # MoE layer frequency constraints
    # Using fixed num_hidden_layers from default config for constraint calculation
    num_hidden_layers = 4                              # Fixed from default config
    max_moe_freq = max(2, num_hidden_layers // 2)
    moe_layer_freq = trial.suggest_int('moe_layer_freq', 1, max_moe_freq)
    params['moe_layer_freq'] = moe_layer_freq
    
    # First K dense replace constraints
    max_first_k = max(1, num_hidden_layers // 2)
    first_k_dense_replace = trial.suggest_int('first_k_dense_replace', 0, max_first_k)
    params['first_k_dense_replace'] = first_k_dense_replace
    
    # MoE gating and balancing parameters - core routing algorithm parameters
    norm_topk_prob = trial.suggest_categorical('norm_topk_prob', [True, False])
    params['norm_topk_prob'] = norm_topk_prob
    
    scoring_func = trial.suggest_categorical('scoring_func', ['softmax', 'sigmoid'])
    params['scoring_func'] = scoring_func
    
    aux_loss_alpha = trial.suggest_float('aux_loss_alpha', 1e-4, 0.1, log=True)
    params['aux_loss_alpha'] = aux_loss_alpha
    
    seq_aux = trial.suggest_categorical('seq_aux', [True, False])
    params['seq_aux'] = seq_aux
    
    # Keep all non-routing parameters fixed at default values
    # These are not tuned to maintain model architecture stability
    params['hidden_size'] = 256  
    params['intermediate_size'] = 512  
    params['moe_intermediate_size'] = 256 
    params['num_hidden_layers'] = 4  
    params['num_attention_heads'] = 4  
    params['num_key_value_heads'] = 2 
    params['hidden_act'] = 'swish' 
    params['max_position_embeddings'] = 256 
    params['initializer_range'] = 0.001  
    params['rms_norm_eps'] = 1e-6 
    params['use_cache'] = True  
    params['rope_theta'] = 10000.0  
    params['rope_scaling_type'] = 'linear'  
    params['rope_scaling_factor'] = 2.0  
    params['attention_bias'] = False  
    params['attention_dropout'] = 0.1  
    
    return params

## Tuned Hyperparameters

In [26]:
def multi_objective(trial):
    """
    Multi-objective optimization function focusing ONLY on routing parameters.
    This prevents dimensional mismatches and maintains stable hyperparameter model architecture.
    """
    # Get validated parameters with constraints
    params = suggest_routing_hyperparameters(trial)
    
    # Create rope_scaling dictionary with fixed parameters
    rope_scaling = {
        "type": params['rope_scaling_type'], 
        "factor": params['rope_scaling_factor']
    }
    
    # Create config with fixed architecture parameters and tuned routing parameters
    config = DeepSeekConfig(
        # Fixed parameters
        vocab_size=vocab_size,  
        hidden_size=params['hidden_size'],  
        intermediate_size=params['intermediate_size'], 
        moe_intermediate_size=params['moe_intermediate_size'],  
        num_hidden_layers=params['num_hidden_layers'], 
        num_attention_heads=params['num_attention_heads'], 
        num_key_value_heads=params['num_key_value_heads'],  
        hidden_act=params['hidden_act'],  # Fixed
        max_position_embeddings=params['max_position_embeddings'],  
        initializer_range=params['initializer_range'],  
        rms_norm_eps=params['rms_norm_eps'], 
        use_cache=params['use_cache'],  
        rope_theta=params['rope_theta'],  
        rope_scaling=rope_scaling, 
        attention_bias=params['attention_bias'], 
        attention_dropout=params['attention_dropout'],
        
        # Tuned Routing Parameters
        n_shared_experts=params['n_shared_experts'],  
        n_routed_experts=params['n_routed_experts'],
        num_experts_per_tok=params['num_experts_per_tok'], 
        moe_layer_freq=params['moe_layer_freq'], 
        first_k_dense_replace=params['first_k_dense_replace'], 
        norm_topk_prob=params['norm_topk_prob'],  
        scoring_func=params['scoring_func'],  
        aux_loss_alpha=params['aux_loss_alpha'], 
        seq_aux=params['seq_aux'],  
    )
    
    # Clear TensorFlow session and memory
    tf.keras.backend.clear_session()
    gc.collect()

    # Create model with the tuned config using DeepSeekForCausalLM directly
    model = DeepSeekForCausalLM(config)

    # Build model with sample batch
    sample_batch = next(iter(tf_train_dataset.take(1)))
    model(sample_batch[0])

    # Configure optimizer with exponential decay learning rate
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=1e-4,
        decay_steps=1000,
        decay_rate=0.9
    )

    optimizer = tf.keras.optimizers.AdamW(
        learning_rate=lr_schedule,
        beta_1=0.9,
        beta_2=0.95,
        weight_decay=0.01,
        clipnorm=1.0
    )

    # Get PAD_TOKEN_ID 
    PAD_TOKEN_ID = tokenizer.token_to_id(PAD_TOKEN)

    # Compile model 
    model.compile(
        optimizer=optimizer,
        loss=tf.keras.losses.SparseCategoricalCrossentropy(
            from_logits=True, 
            ignore_class=PAD_TOKEN_ID
        ),
        metrics=['accuracy']
    )

    # Define training callbacks
    metrics_callback = DeepSeekMetrics(tf_val_dataset, tokenizer)
    resource_monitor = ResourceMonitor(batch_size=2)

    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            patience=3,
            restore_best_weights=True, 
            monitor='val_loss',
            min_delta=0.01
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            factor=0.5, 
            patience=2,
            monitor='val_loss',
            min_delta=0.01,
            min_lr=1e-6
        ),
        tf.keras.callbacks.TerminateOnNaN(),
        metrics_callback,
        resource_monitor
    ]

    # Train model on GPU
    print("Starting training...")
    start_time = time.time()

    # Force GPU for training the model
    with tf.device('/GPU:0'):
        history = model.fit(
            tf_train_dataset,
            epochs=100,                         # Number of epoch for each hyperparameter tuning iteration
            validation_data=tf_val_dataset,
            callbacks=callbacks,
            verbose=0,
        )
            
    total_training_time = time.time() - start_time

    # Calculate final metrics
    print("Calculating final metrics...")
    final_perplexity = calculate_perplexity(model, tf_val_dataset, tokenizer, max_batches=100)
            
    final_training_loss = history.history['loss'][-1]
    final_validation_loss = history.history['val_loss'][-1]

    # Calculate MaxVIO metrics - this is our second objective
    max_vio_values = metrics_callback.max_vio_values
    final_max_vio = max_vio_values[-1]
    min_max_vio = min(max_vio_values)
    avg_max_vio = np.mean(max_vio_values)

    print(f"Final Training Loss: {final_training_loss:.4f}")
    print(f"Final Validation Loss: {final_validation_loss:.4f}")
    print(f"Final Perplexity: {final_perplexity:.2f}")
    print(f"Final MaxVIO: {final_max_vio:.4f}")
    print(f"Minimum MaxVIO: {min_max_vio:.4f}")
    print(f"Average MaxVIO: {avg_max_vio:.4f}")

    # Store additional metrics in trial user attributes for analysis
    trial.set_user_attr("final_training_loss", float(final_training_loss))
    trial.set_user_attr("final_perplexity", float(final_perplexity))
    trial.set_user_attr("min_max_vio", float(min_max_vio))
    trial.set_user_attr("avg_max_vio", float(avg_max_vio))
    trial.set_user_attr("training_time", total_training_time)

    # Return both validation loss and final MaxVIO as objectives to minimize
    return final_validation_loss, final_max_vio

# Create multi-objective study for Pareto front analysis
multi_study = optuna.create_study(
    study_name="deepseek_routing_tuning_V1",
    directions=['minimize', 'minimize'],                    # Minimize both validation loss and MaxVIO
    storage='sqlite:///deepseek_routing_tuning_study_V1.db',# Save hyperparameter tuning for continue tuning
    load_if_exists=True                                     # Set to False if firstime
)

# Tuned Hyperparameters
multi_study.optimize(multi_objective, n_trials=20)  # Number of iteration to do hyperparameters tuning

[I 2025-10-17 12:38:39,704] Using an existing study with name 'deepseek_routing_tuning_V1' instead of creating a new one.


Starting training...
Epoch 1: Loss=8.7774, Val_Loss=7.9827, GFLOPs=2.75, MaxVIO=0.0660
Epoch 1 Resources: GPU Mem: 14.3GB, System Mem: 6.1GB, GPU: 27.0%, CPU: 13.1%, TPS: 311
Calculating final metrics...


[I 2025-10-17 12:39:09,428] Trial 36 finished with values: [7.982700824737549, 0.06597402691841125] and parameters: {'n_shared_experts': 2, 'n_routed_experts': 6, 'num_experts_per_tok': 3, 'moe_layer_freq': 2, 'first_k_dense_replace': 2, 'norm_topk_prob': False, 'scoring_func': 'softmax', 'aux_loss_alpha': 0.0003182236318183972, 'seq_aux': False}.


Final Training Loss: 8.7774
Final Validation Loss: 7.9827
Final Perplexity: 2921.71
Final MaxVIO: 0.0660
Minimum MaxVIO: 0.0660
Average MaxVIO: 0.0660
Starting training...
Epoch 1: Loss=8.7912, Val_Loss=8.0240, GFLOPs=2.80, MaxVIO=0.1166
Epoch 1 Resources: GPU Mem: 14.3GB, System Mem: 6.4GB, GPU: 23.0%, CPU: 24.1%, TPS: 327
Calculating final metrics...
Final Training Loss: 8.7912
Final Validation Loss: 8.0240
Final Perplexity: 3074.14
Final MaxVIO: 0.1166
Minimum MaxVIO: 0.1166
Average MaxVIO: 0.1166


[I 2025-10-17 12:39:38,215] Trial 37 finished with values: [8.023995399475098, 0.11662334203720093] and parameters: {'n_shared_experts': 3, 'n_routed_experts': 6, 'num_experts_per_tok': 1, 'moe_layer_freq': 2, 'first_k_dense_replace': 2, 'norm_topk_prob': False, 'scoring_func': 'softmax', 'aux_loss_alpha': 0.0003786647150736817, 'seq_aux': False}.


## Pareto Front Analysis

In [27]:
def analyze_pareto_front(study):
    """Use Pareto front for multi-objective optimization with zero MaxVIO filtering.
       Multi-objective are lowest validation loss and lowest MaxViO .
       By, that MaxViO must not be 0 (ideal = errors) and 1 (model cannot achieve spare model).
    """
    print("\nPareto Front Analysis...")
    
    # Get Pareto front trials
    pareto_trials = study.best_trials
    
    # Filter out trials with zero MaxVIO to avoid unrealistic solutions
    filtered_trials = [trial for trial in pareto_trials 
                      if trial.values and len(trial.values) >= 2 
                      and trial.values[1] > 0.0]                       # Exclude zero MaxVIO
    
    print(f"Found {len(pareto_trials)} Pareto-optimal solutions before filtering...")
    print(f"Found {len(filtered_trials)} Pareto-optimal solutions after filtering out zero MaxVIO...")
    
    # Find the solution with lowest validation loss (excluding zero MaxVIO)
    lowest_val_loss_trial = None
    lowest_val_loss = float('inf')
    
    # Find the solution with lowest MaxVIO (excluding zero MaxVIO)
    lowest_maxvio_trial = None
    lowest_maxvio = float('inf')
    
    for trial in filtered_trials:
        if trial.values and len(trial.values) >= 2:
            val_loss = trial.values[0]
            max_vio = trial.values[1]
            
            # Track lowest validation loss
            if val_loss < lowest_val_loss:
                lowest_val_loss = val_loss
                lowest_val_loss_trial = trial
                
            # Track lowest MaxVIO
            if max_vio < lowest_maxvio:
                lowest_maxvio = max_vio
                lowest_maxvio_trial = trial
    
    # Display all filtered Pareto-optimal solutions
    for i, trial in enumerate(filtered_trials):
        val_loss = trial.values[0]
        max_vio = trial.values[1]
        print(f"Solution {i+1}: Val Loss = {val_loss:.4f}, MaxVIO = {max_vio:.4f}")
        
        # Display key routing parameters for this solution
        key_params = {
            'n_routed_experts': trial.params.get('n_routed_experts'),
            'num_experts_per_tok': trial.params.get('num_experts_per_tok'),
            'n_shared_experts': trial.params.get('n_shared_experts'),
            'aux_loss_alpha': trial.params.get('aux_loss_alpha'),
            'moe_layer_freq': trial.params.get('moe_layer_freq'),
            'scoring_func': trial.params.get('scoring_func'),
            'norm_topk_prob': trial.params.get('norm_topk_prob')
        }
        print(f"    Routing params: {key_params}")
    
    # Display the best solutions for each objective
    if lowest_val_loss_trial:
        print(f"\n------Best Solution for Validation Loss------")
        print(f"Lowest Validation Loss: {lowest_val_loss:.4f}")
        print(f"Corresponding MaxVIO: {lowest_val_loss_trial.values[1]:.4f}")
        print(f"Parameters: {lowest_val_loss_trial.params}")
    
    if lowest_maxvio_trial:
        print(f"\n-------Best Solution for MaxVIO------")
        print(f"Lowest MaxVIO: {lowest_maxvio:.4f}")
        print(f"Corresponding Validation Loss: {lowest_maxvio_trial.values[0]:.4f}")
        print(f"Parameters: {lowest_maxvio_trial.params}")
    
    return filtered_trials, lowest_val_loss_trial, lowest_maxvio_trial

def find_best_balanced_solution(study):
    """
    Find the best balanced solution that considers both validation loss and MaxVIO.
    This identifies solutions that are good on both objectives rather than extreme on one obejective.
    Filters out zero MaxVIO solutions before calculating balanced scores.
    """
    print("\nFinding Best Balanced Solution")
    
    # Get filtered trials excluding zero MaxVIO
    filtered_trials = [trial for trial in study.best_trials 
                      if trial.values and len(trial.values) >= 2 
                      and trial.values[1] > 0.0]
        
    print(f"Evaluating {len(filtered_trials)} solutions for balanced scoring (excluding zero MaxVIO)")
        
    # Calculate balanced scores for each filtered Pareto-optimal solution
    balanced_scores = []
    for trial in filtered_trials:
        val_loss = trial.values[0]
        max_vio = trial.values[1]
        
        # Use weighted geometric mean as balanced score
        # This penalizes extreme values in both objectives
        if val_loss > 0 and max_vio > 0:
            # Geometric mean gives equal importance to both objectives
            balanced_score = (val_loss * max_vio) ** 0.5
            balanced_scores.append((balanced_score, trial, val_loss, max_vio))
        else:
            # Skip invalid values
            continue
    
    # Find the solution with the best (lowest) balanced score
    best_balanced_score, best_balanced_trial, best_val_loss, best_max_vio = min(
        balanced_scores, key=lambda x: x[0]
    )
    
    print(f"Best Balanced Solution (after filtering zero MaxVIO):")
    print(f"\n  Validation Loss: {best_val_loss:.4f}")
    print(f"  MaxVIO: {best_max_vio:.4f}")
    print(f"  Balanced Score: {best_balanced_score:.4f}")
    print(f"  Routing Parameters: {best_balanced_trial.params}")
    
    # Show the range of balanced scores for context
    if len(balanced_scores) > 1:
        scores_only = [score[0] for score in balanced_scores]
        print(f"  Balanced Score Range: {min(scores_only):.4f} - {max(scores_only):.4f}")
    
    return best_balanced_trial

# Analyze results
analyze_pareto_front(multi_study)
best_balanced_trial = find_best_balanced_solution(multi_study)

# Display final results summary
print(f"\nFinal Best Balanced Routing Selected")
print(f"Validation Loss: {best_balanced_trial.values[0]:.4f}")
print(f"MaxVIO: {best_balanced_trial.values[1]:.4f}")

print(f"Optimized Routing Hyperparameters:")
routing_params = {
    'n_routed_experts': best_balanced_trial.params.get('n_routed_experts'),
    'n_shared_experts': best_balanced_trial.params.get('n_shared_experts'),
    'num_experts_per_tok': best_balanced_trial.params.get('num_experts_per_tok'),
    'moe_layer_freq': best_balanced_trial.params.get('moe_layer_freq'),
    'first_k_dense_replace': best_balanced_trial.params.get('first_k_dense_replace'),
    'scoring_func': best_balanced_trial.params.get('scoring_func'),
    'norm_topk_prob': best_balanced_trial.params.get('norm_topk_prob'),
    'aux_loss_alpha': best_balanced_trial.params.get('aux_loss_alpha'),
    'seq_aux': best_balanced_trial.params.get('seq_aux')
}
for key, value in routing_params.items():
    print(f"  {key}: {value}")

print("\nHyperparameter Tuning Complete")


Pareto Front Analysis...
Found 2 Pareto-optimal solutions before filtering...
Found 1 Pareto-optimal solutions after filtering out zero MaxVIO...
Solution 1: Val Loss = 0.5756, MaxVIO = 0.0481
    Routing params: {'n_routed_experts': 6, 'num_experts_per_tok': 2, 'n_shared_experts': 3, 'aux_loss_alpha': 0.0022997166087842954, 'moe_layer_freq': 1, 'scoring_func': 'softmax', 'norm_topk_prob': True}

------Best Solution for Validation Loss------
Lowest Validation Loss: 0.5756
Corresponding MaxVIO: 0.0481
Parameters: {'n_shared_experts': 3, 'n_routed_experts': 6, 'num_experts_per_tok': 2, 'moe_layer_freq': 1, 'first_k_dense_replace': 1, 'norm_topk_prob': True, 'scoring_func': 'softmax', 'aux_loss_alpha': 0.0022997166087842954, 'seq_aux': False}

-------Best Solution for MaxVIO------
Lowest MaxVIO: 0.0481
Corresponding Validation Loss: 0.5756
Parameters: {'n_shared_experts': 3, 'n_routed_experts': 6, 'num_experts_per_tok': 2, 'moe_layer_freq': 1, 'first_k_dense_replace': 1, 'norm_topk_prob'

# Model Training and Experiment

### Model training and experiment with the best hyperparameters

In [28]:
# Disable XLA (XLA JIT compilation) to prevent CUDA graph errors and ensure stability
tf.config.optimizer.set_jit(False)  # Disabled XLA to prevent CUDA graph errors

# Configure GPU memory growth to prevent memory allocation issues
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Enable mixed precision training with FP16 for improved performance and reduced memory usage
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

# Configuration parameters from the model and preprocessing pipeline
vocab_size = 12000
max_seq_len = 70
batch_size = 2
PAD_TOKEN = "<pad>"
BOS_TOKEN = "<bos>"
EOS_TOKEN = "<eos>"
SEP_TOKEN = "<sep>"

# Print TensorFlow version, mixed precision policy, and GPU availability for debugging
print(f"TensorFlow version: {tf.__version__}")
print(f"Mixed precision policy: {mixed_precision.global_policy().name}")
print(f"GPU available: {len(gpus) > 0}")

# DEEPSEEK MOE COMPONENTS

class DeepSeekRMSNorm(layers.Layer):
    """RMS Normalization layer """
    def __init__(self, hidden_size, eps=1e-6, **kwargs):
        super().__init__(**kwargs)
        self.hidden_size = hidden_size  # Dimension of the hidden representations
        self.variance_epsilon = eps     # Small constant to avoid division by zero
        
    def build(self, input_shape):
        # Initialize learnable weight parameter for scaling normalized output
        self.weight = self.add_weight(
            name='weight',
            shape=(self.hidden_size,),
            initializer='ones',
            trainable=True
        )
        super().build(input_shape)
        
    def call(self, x):
        input_dtype = x.dtype
        x = tf.cast(x, tf.float32)  # Cast input to float32 for stable computation
        
        # Compute variance as mean of squared inputs along the last dimension
        variance = tf.reduce_mean(tf.square(x), axis=-1, keepdims=True)
        # Normalize input using RMS formula with epsilon for numerical stability
        x = x * tf.math.rsqrt(variance + self.variance_epsilon)
        
        # Scale normalized output with learnable weight and cast back to original dtype
        return self.weight * tf.cast(x, input_dtype)

class RotaryPositionEmbedding(layers.Layer):
    """Rotary Position Embedding (RoPE) with dynamic NTK scaling for positional encoding"""
    def __init__(self, dim, max_position_embeddings=2048, base=10000, scaling_type=None, scaling_factor=1.0, **kwargs):
        super().__init__(**kwargs)
        self.dim = dim                              # Dimension of the head embeddings
        self.max_position_embeddings = max_position_embeddings  # Maximum sequence length
        self.base = base                           # Base value for frequency calculation
        self.scaling_type = scaling_type           # Type of scaling ("linear" or "dynamic")
        self.scaling_factor = scaling_factor       # Scaling factor for RoPE
        
    def build(self, input_shape):
        # Compute inverse frequencies for rotary embeddings
        inv_freq = 1.0 / (self.base ** (tf.range(0, self.dim, 2, dtype=tf.float32) / tf.cast(self.dim, tf.float32)))
        self.inv_freq = tf.constant(inv_freq, dtype=tf.float32)
        self._compute_rotary_cache()  # Precompute cosine and sine caches
        super().build(input_shape)
        
    def _compute_rotary_cache(self):
        """Compute cosine and sine caches for rotary embeddings with optional scaling"""
        if self.scaling_type == "linear":
            # Linear scaling: divide positions by scaling factor
            t = tf.range(self.max_position_embeddings, dtype=tf.float32) / self.scaling_factor
        elif self.scaling_type == "dynamic":
            # Dynamic NTK scaling for extended context lengths
            if self.max_position_embeddings > self.base:
                base = self.base * (
                    (self.scaling_factor * self.max_position_embeddings / self.base) - 
                    (self.scaling_factor - 1)
                ) ** (self.dim / (self.dim - 2))
                inv_freq = 1.0 / (base ** (tf.range(0, self.dim, 2, dtype=tf.float32) / tf.cast(self.dim, tf.float32)))
                self.inv_freq = tf.constant(inv_freq, dtype=tf.float32)
            t = tf.range(self.max_position_embeddings, dtype=tf.float32)
        else:
            # Default case: no scaling
            t = tf.range(self.max_position_embeddings, dtype=tf.float32)
            
        # Compute frequency matrix for rotary embeddings
        freqs = tf.einsum('i,j->ij', t, self.inv_freq)
        emb = tf.concat([freqs, freqs], axis=-1)  # Concatenate for full embedding dimension
        
        # Cache cosine and sine values as non-trainable variables
        self.cos_cached = tf.Variable(tf.cos(emb), trainable=False, name='cos_cached')
        self.sin_cached = tf.Variable(tf.sin(emb), trainable=False, name='sin_cached')
        
    def call(self, x, position_ids=None, seq_len=None):
        # Get input tensor dimensions
        batch_size, seq_len_x, num_heads, head_dim = tf.unstack(tf.shape(x))
        
        # Generate position IDs if not provided
        if position_ids is None:
            position_ids = tf.range(seq_len_x, dtype=tf.int32)
            position_ids = tf.tile(tf.reshape(position_ids, [1, -1]), [batch_size, 1])
        
        # Gather precomputed cosine and sine values for given position IDs
        cos = tf.gather(self.cos_cached, position_ids, axis=0)  # Shape: [batch_size, seq_len_x, head_dim]
        sin = tf.gather(self.sin_cached, position_ids, axis=0)  # Shape: [batch_size, seq_len_x, head_dim]
        
        # Split cosine and sine for application to x1 and x2
        cos_1, cos_2 = tf.split(cos, 2, axis=-1)
        sin_1, sin_2 = tf.split(sin, 2, axis=-1)
        
        # Reshape for broadcasting with attention heads
        cos_1 = tf.reshape(cos_1, [batch_size, seq_len_x, 1, head_dim // 2])
        sin_1 = tf.reshape(sin_1, [batch_size, seq_len_x, 1, head_dim // 2])
        cos_2 = tf.reshape(cos_2, [batch_size, seq_len_x, 1, head_dim // 2])
        sin_2 = tf.reshape(sin_2, [batch_size, seq_len_x, 1, head_dim // 2])
        
        # Repeat to match number of attention heads
        cos_1 = tf.repeat(cos_1, num_heads, axis=2)
        sin_1 = tf.repeat(sin_1, num_heads, axis=2)
        cos_2 = tf.repeat(cos_2, num_heads, axis=2)
        sin_2 = tf.repeat(sin_2, num_heads, axis=2)
        
        # Cast to input dtype for mixed precision compatibility
        x_dtype = x.dtype
        cos_1 = tf.cast(cos_1, x_dtype)
        sin_1 = tf.cast(sin_1, x_dtype)
        cos_2 = tf.cast(cos_2, x_dtype)
        sin_2 = tf.cast(sin_2, x_dtype)
        
        # Split input tensor into two parts for rotary application
        x1, x2 = tf.split(x, 2, axis=-1)
        
        # Apply rotary embeddings to compute rotated representations
        rotated_x1 = x1 * cos_1 - x2 * sin_1
        rotated_x2 = x1 * sin_2 + x2 * cos_2
        
        # Concatenate rotated parts to restore original shape
        return tf.concat([rotated_x1, rotated_x2], axis=-1)

class DeepSeekMLP(layers.Layer):
    """Multi-Layer Perceptron (MLP) with Swish activation"""
    def __init__(self, hidden_size, intermediate_size, hidden_act="swish", **kwargs):
        super().__init__(**kwargs)
        self.hidden_size = hidden_size           # Input and output dimension
        self.intermediate_size = intermediate_size  # Intermediate layer dimension
        self.hidden_act = hidden_act            # Activation function (Swish/SILU)
        
    def build(self, input_shape):
        # Define gate projection layer with reduced initializer variance for stability
        self.gate_proj = layers.Dense(
            self.intermediate_size,
            kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.001),
            use_bias=False
        )
        # Define up projection layer
        self.up_proj = layers.Dense(
            self.intermediate_size,
            kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.001),
            use_bias=False
        )
        # Define down projection layer to return to hidden size
        self.down_proj = layers.Dense(
            self.hidden_size,
            kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.001),
            use_bias=False
        )
        super().build(input_shape)
        
    def call(self, x):
        # Apply gate and up projections
        gate_output = self.gate_proj(x)
        up_output = self.up_proj(x)
        # Apply Swish (SiLU) activation to gate output and multiply with up output
        activated = tf.nn.silu(gate_output) * up_output
        # Project back to hidden size
        return self.down_proj(activated)

class MoEGate(layers.Layer):
    """Mixture of Experts (MoE) gating mechanism with load balancing auxiliary loss"""
    def __init__(self, num_experts, num_experts_per_tok, hidden_size, 
                 scoring_func='softmax', norm_topk_prob=False, 
                 aux_loss_alpha=0.001, seq_aux=True, **kwargs):
        super().__init__(**kwargs)
        self.num_experts = num_experts                  # Total number of experts
        self.num_experts_per_tok = num_experts_per_tok  # Number of experts per token
        self.hidden_size = hidden_size                  # Input dimension
        self.scoring_func = scoring_func                # Scoring function for expert selection
        self.norm_topk_prob = norm_topk_prob            # Normalize top-k probabilities
        self.aux_loss_alpha = aux_loss_alpha            # Weight for auxiliary loss
        self.seq_aux = seq_aux                          # Compute auxiliary loss per sequence
        
    def build(self, input_shape):
        # Initialize gating weights with reduced variance for stability
        self.weight = self.add_weight(
            name='gate_weight',
            shape=(self.num_experts, self.hidden_size),
            initializer=tf.keras.initializers.RandomNormal(stddev=0.001),
            trainable=True
        )
        # Initialize expert assignment counter for tracking during training
        self.expert_counts = tf.Variable(tf.zeros([self.num_experts], dtype=tf.int32), trainable=False)
        super().build(input_shape)
        
    def call(self, hidden_states, training=False):
        # Get input dimensions
        batch_size, seq_len, h = tf.unstack(tf.shape(hidden_states))
        hidden_states_flat = tf.reshape(hidden_states, [-1, h])
        # Compute gating logits
        logits = tf.matmul(hidden_states_flat, self.weight, transpose_b=True)
        
        # Clip logits for numerical stability
        logits = tf.clip_by_value(logits, -50.0, 50.0)
        
        # Apply scoring function (softmax) to compute expert scores
        if self.scoring_func == 'softmax':
            temperature = 0.5                                         # Temperature for scaling logits
            scaled_logits = tf.cast(logits, tf.float32) / temperature
            scores = tf.nn.softmax(scaled_logits, axis=-1)
            scores = tf.cast(scores, hidden_states.dtype)
        else:
            scores = tf.nn.softmax(tf.cast(logits, tf.float32), axis=-1)
            scores = tf.cast(scores, hidden_states.dtype)

        # Select top-k experts and their weights
        topk_weights, topk_indices = tf.math.top_k(
            scores, k=self.num_experts_per_tok, sorted=False
        )

        # Normalize top-k weights if specified
        if self.num_experts_per_tok > 1 and self.norm_topk_prob:
            denominator = tf.reduce_sum(topk_weights, axis=-1, keepdims=True) + 1e-20
            topk_weights = topk_weights / denominator

        # Update expert assignment counts during training
        if training:
            flat_topk_indices = tf.reshape(topk_indices, [-1])
            counts = tf.reduce_sum(tf.one_hot(flat_topk_indices, depth=self.num_experts), axis=0)
            self.expert_counts.assign_add(tf.cast(counts, tf.int32))

        aux_loss = None
        if training and self.aux_loss_alpha > 0.0:
            if self.seq_aux:
                # Sequence-level auxiliary loss for load balancing
                scores_seq = tf.reshape(scores, [batch_size, seq_len, -1])
                topk_indices_seq = tf.reshape(topk_indices, [batch_size, -1])
                ce = tf.zeros([batch_size, self.num_experts], dtype=tf.float32)
                mask = tf.one_hot(topk_indices_seq, depth=self.num_experts)
                ce = tf.reduce_sum(mask, axis=1)
                seq_len_float = tf.cast(seq_len, tf.float32)
                num_experts_per_tok_float = tf.cast(self.num_experts_per_tok, tf.float32)
                num_experts_float = tf.cast(self.num_experts, tf.float32)
                ce = ce / (seq_len_float * num_experts_per_tok_float / num_experts_float)
                scores_seq_mean = tf.reduce_mean(tf.cast(scores_seq, tf.float32), axis=1)
                aux_loss = tf.reduce_sum(ce * scores_seq_mean, axis=1)
                aux_loss = tf.reduce_mean(aux_loss) * self.aux_loss_alpha
            else:
                # Token-level auxiliary loss
                mask_ce = tf.one_hot(tf.reshape(topk_indices, [-1]), depth=self.num_experts)
                ce = tf.reduce_mean(tf.cast(mask_ce, tf.float32), axis=0)
                Pi = tf.reduce_mean(tf.cast(scores, tf.float32), axis=0)
                fi = ce * self.num_experts
                aux_loss = tf.reduce_sum(Pi * fi) * self.aux_loss_alpha

            # Add auxiliary loss to the layer's losses
            self.add_loss(aux_loss)

        return topk_indices, topk_weights, aux_loss

class DeepSeekMoE(layers.Layer):
    """Mixture of Experts (MoE) layer with shared and routed experts"""
    def __init__(self, hidden_size, n_routed_experts, n_shared_experts, 
                 num_experts_per_tok, moe_intermediate_size, **kwargs):
        super().__init__(**kwargs)
        self.hidden_size = hidden_size                      # Input and output dimension
        self.n_routed_experts = n_routed_experts            # Number of routed experts
        self.n_shared_experts = n_shared_experts            # Number of shared experts
        self.num_experts_per_tok = num_experts_per_tok      # Experts per token
        self.moe_intermediate_size = moe_intermediate_size  # MoE intermediate size
        
    def build(self, input_shape):
        # Initialize list of routed expert MLPs
        self.routed_experts = [
            DeepSeekMLP(
                self.hidden_size, 
                self.moe_intermediate_size,
                name=f"routed_expert_{i}"
            ) for i in range(self.n_routed_experts)
        ]
        
        # Initialize shared experts if specified
        if self.n_shared_experts is not None:
            shared_intermediate_size = self.moe_intermediate_size * self.n_shared_experts
            self.shared_experts = DeepSeekMLP(
                self.hidden_size,
                shared_intermediate_size,
                name="shared_experts"
            )
        else:
            self.shared_experts = None
            
        # Initialize MoE gating mechanism
        self.gate = MoEGate(
            self.n_routed_experts,
            self.num_experts_per_tok,
            self.hidden_size,
            name="moe_gate"
        )
        
        # Build each expert and gate
        for expert in self.routed_experts:
            expert.build(input_shape)
        if self.shared_experts:
            self.shared_experts.build(input_shape)
        self.gate.build(input_shape)
        
        super().build(input_shape)
        
    def call(self, hidden_states, training=False):
        identity = hidden_states  # Store input for residual connection
        orig_shape = tf.shape(hidden_states)
        # Get top-k expert indices and weights
        topk_idx, topk_weight, aux_loss = self.gate(hidden_states, training=training)
        hidden_states_flat = tf.reshape(hidden_states, [-1, self.hidden_size])
        
        if training:
            # Expand inputs for top-k experts during training
            hidden_states_expanded = tf.repeat(
                hidden_states_flat, self.num_experts_per_tok, axis=0
            )
            flat_topk_idx = tf.reshape(topk_idx, [-1])
            combined_output = tf.zeros_like(hidden_states_expanded)
            # Process each expert's input
            for i, expert in enumerate(self.routed_experts):
                expert_mask = tf.equal(flat_topk_idx, i)
                if tf.reduce_any(expert_mask):
                    expert_input = tf.boolean_mask(hidden_states_expanded, expert_mask)
                    expert_output = expert(expert_input)
                    update_indices = tf.where(expert_mask)
                    combined_output = tf.tensor_scatter_nd_update(
                        combined_output,
                        update_indices,
                        expert_output
                    )
                    
            # Apply expert weights and reshape output
            topk_weight_flat = tf.reshape(topk_weight, [-1, 1])
            weighted_output = combined_output * topk_weight_flat
            weighted_output = tf.reshape(
                weighted_output,
                [-1, self.num_experts_per_tok, self.hidden_size]
            )
            routed_output = tf.reduce_sum(weighted_output, axis=1)
            routed_output = tf.reshape(routed_output, orig_shape)
        else:
            # Use efficient inference path
            routed_output = self._moe_infer(hidden_states_flat, topk_idx, topk_weight)
            routed_output = tf.reshape(routed_output, orig_shape)
            
        # Add shared expert output if available
        if self.shared_experts is not None:
            shared_output = self.shared_experts(identity)
            final_output = routed_output + shared_output
        else:
            final_output = routed_output
            
        # Apply residual connection
        return final_output + identity
        
    def _moe_infer(self, x, expert_indices, expert_weights):
        """Efficient inference path for MoE layer"""
        batch_size = tf.shape(x)[0]
        expert_cache = tf.zeros_like(x)
        # Clip expert indices to valid range
        expert_indices = tf.clip_by_value(expert_indices, 0, self.n_routed_experts - 1)
        flat_expert_indices = tf.reshape(expert_indices, [-1])
        flat_expert_weights = tf.reshape(expert_weights, [-1, 1])
        total_tokens = tf.shape(x)[0]
        token_indices_base = tf.range(total_tokens, dtype=tf.int64)
        token_indices_base = tf.repeat(token_indices_base, self.num_experts_per_tok, axis=0)
        
        # Process each expert's tokens
        for i in range(self.n_routed_experts):
            expert_mask = tf.equal(flat_expert_indices, i)
            if tf.reduce_any(expert_mask):
                token_indices = tf.boolean_mask(token_indices_base, expert_mask)
                expert_tokens = tf.gather(x, token_indices)
                expert_out = self.routed_experts[i](expert_tokens)
                expert_out_weighted = expert_out * tf.gather(flat_expert_weights, tf.where(expert_mask)[:, 0])
                expert_cache = tf.tensor_scatter_nd_add(
                    expert_cache,
                    tf.expand_dims(token_indices, 1),
                    expert_out_weighted
                )
                
        return expert_cache

class DeepSeekAttention(layers.Layer):
    """Multi-head attention with Rotary Position Embeddings and Grouped Query Attention (GQA) support"""
    def __init__(self, hidden_size, num_attention_heads, num_key_value_heads=None,
                 attention_dropout=0.0, max_position_embeddings=2048,
                 rope_theta=10000.0, rope_scaling=None, attention_bias=False, **kwargs):
        super().__init__(**kwargs)
        self.hidden_size = hidden_size                                         # Input and output dimension
        self.num_attention_heads = num_attention_heads                         # Number of attention heads
        self.num_key_value_heads = num_key_value_heads or num_attention_heads  # Number of key/value heads (GQA)
        self.attention_dropout = attention_dropout                             # Dropout rate for attention weights
        self.max_position_embeddings = max_position_embeddings                 # Maximum sequence length
        self.rope_theta = rope_theta                                           # Base period for RoPE
        self.rope_scaling = rope_scaling                                       # RoPE scaling configuration
        self.attention_bias = attention_bias                                   # Whether to use bias in projections
        self.head_dim = hidden_size // num_attention_heads                     # Dimension per head
        self.num_key_value_groups = num_attention_heads // self.num_key_value_heads  # Groups for GQA
        
    def build(self, input_shape):
        # Initialize query projection layer
        self.q_proj = layers.Dense(
            self.num_attention_heads * self.head_dim,
            use_bias=self.attention_bias,
            kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.001),
            name="q_proj"
        )
        # Initialize key projection layer
        self.k_proj = layers.Dense(
            self.num_key_value_heads * self.head_dim,
            use_bias=self.attention_bias,
            kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.001),
            name="k_proj"
        )
        # Initialize value projection layer
        self.v_proj = layers.Dense(
            self.num_key_value_heads * self.head_dim,
            use_bias=self.attention_bias,
            kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.001),
            name="v_proj"
        )
        # Initialize output projection layer
        self.o_proj = layers.Dense(
            self.hidden_size,
            use_bias=self.attention_bias,
            kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.001),
            name="o_proj"
        )
        
        # Initialize RoPE with specified scaling
        scaling_type = self.rope_scaling.get("type") if self.rope_scaling else None
        scaling_factor = self.rope_scaling.get("factor", 1.0) if self.rope_scaling else 1.0
        self.rotary_emb = RotaryPositionEmbedding(
            self.head_dim,
            self.max_position_embeddings,
            self.rope_theta,
            scaling_type,
            scaling_factor,
            name="rotary_emb"
        )
        self.dropout_layer = layers.Dropout(self.attention_dropout)
        self.rotary_emb.build(input_shape)
        super().build(input_shape)
        
    def call(self, hidden_states, attention_mask=None, position_ids=None, training=False):
        # Get input dimensions
        batch_size, seq_len, _ = tf.unstack(tf.shape(hidden_states))
        
        # Project inputs to query, key, and value
        q = self.q_proj(hidden_states)
        k = self.k_proj(hidden_states)
        v = self.v_proj(hidden_states)
        
        # Reshape for multi-head attention
        q = tf.reshape(q, [batch_size, seq_len, self.num_attention_heads, self.head_dim])
        k = tf.reshape(k, [batch_size, seq_len, self.num_key_value_heads, self.head_dim])
        v = tf.reshape(v, [batch_size, seq_len, self.num_key_value_heads, self.head_dim])
        
        # Apply rotary embeddings to query and key
        q_rotated = self.rotary_emb(q, position_ids=position_ids)
        k_rotated = self.rotary_emb(k, position_ids=position_ids)
        
        # Transpose for attention computation
        q_rotated = tf.transpose(q_rotated, [0, 2, 1, 3])
        k_rotated = tf.transpose(k_rotated, [0, 2, 1, 3])
        v = tf.transpose(v, [0, 2, 1, 3])
        
        # Repeat key and value for GQA if necessary
        if self.num_key_value_groups > 1:
            k_rotated = tf.repeat(k_rotated, self.num_key_value_groups, axis=1)
            v = tf.repeat(v, self.num_key_value_groups, axis=1)
            
        # Compute scaled dot-product attention
        scale = tf.math.rsqrt(tf.cast(self.head_dim, tf.float32))
        scale = tf.cast(scale, q_rotated.dtype)
        attn_weights = tf.matmul(q_rotated, k_rotated, transpose_b=True) * scale
        
        # Apply attention mask if provided
        if attention_mask is not None:
            attention_mask = tf.cast(attention_mask, attn_weights.dtype)
            if len(attention_mask.shape) == 2:
                attention_mask = tf.expand_dims(tf.expand_dims(attention_mask, 1), 1)
            elif len(attention_mask.shape) == 3:
                attention_mask = tf.expand_dims(attention_mask, 1)
            attn_weights += attention_mask
            
        # Clip attention weights for numerical stability
        attn_weights = tf.clip_by_value(attn_weights, -50.0, 50.0)
        attn_weights = tf.cast(attn_weights, tf.float32)
        # Apply softmax with max subtraction for stability
        attn_weights = attn_weights - tf.reduce_max(attn_weights, axis=-1, keepdims=True)
        attn_weights = tf.nn.softmax(attn_weights, axis=-1)
        attn_weights = tf.cast(attn_weights, q_rotated.dtype)
        
        # Apply dropout to attention weights during training
        attn_weights = self.dropout_layer(attn_weights, training=training)
        # Compute attention output
        attn_output = tf.matmul(attn_weights, v)
        attn_output = tf.transpose(attn_output, [0, 2, 1, 3])
        attn_output = tf.reshape(attn_output, [batch_size, seq_len, self.hidden_size])
        
        # Project attention output to hidden size
        return self.o_proj(attn_output)

class DeepSeekDecoderLayer(layers.Layer):
    """Transformer decoder layer combining attention and MoE/MLP with residual connections"""
    def __init__(self, config, layer_idx, **kwargs):
        super().__init__(**kwargs)
        self.config = config
        self.layer_idx = layer_idx
        
        # Initialize self-attention with GQA and RoPE
        self.self_attn = DeepSeekAttention(
            hidden_size=config.hidden_size,
            num_attention_heads=config.num_attention_heads,
            num_key_value_heads=config.num_key_value_heads,
            attention_dropout=config.attention_dropout,
            max_position_embeddings=config.max_position_embeddings,
            rope_theta=config.rope_theta,
            rope_scaling=config.rope_scaling,
            attention_bias=config.attention_bias,
            name="self_attn"
        )
        
        # Determine whether to use MoE or dense MLP based on layer index and config
        self.use_moe = (
            config.n_routed_experts is not None and
            layer_idx >= config.first_k_dense_replace and
            layer_idx % config.moe_layer_freq == 0
        )
        
        if self.use_moe:
            # Initialize MoE layer for routed experts
            self.mlp = DeepSeekMoE(
                hidden_size=config.hidden_size,
                n_routed_experts=config.n_routed_experts,
                n_shared_experts=config.n_shared_experts,
                num_experts_per_tok=config.num_experts_per_tok,
                moe_intermediate_size=config.moe_intermediate_size,
                name="moe"
            )
        else:
            # Initialize dense MLP layer
            self.mlp = DeepSeekMLP(
                hidden_size=config.hidden_size,
                intermediate_size=config.intermediate_size,
                hidden_act=config.hidden_act,
                name="mlp"
            )
            
        # Initialize input and post-attention normalization layers
        self.input_layernorm = DeepSeekRMSNorm(
            config.hidden_size, eps=config.rms_norm_eps, name="input_layernorm"
        )
        self.post_attention_layernorm = DeepSeekRMSNorm(
            config.hidden_size, eps=config.rms_norm_eps, name="post_attention_layernorm"
        )
        
    def call(self, hidden_states, attention_mask=None, position_ids=None, training=False):
        # Apply input normalization and self-attention with residual connection
        residual = hidden_states
        hidden_states = self.input_layernorm(hidden_states)
        attn_output = self.self_attn(
            hidden_states,
            attention_mask=attention_mask,
            position_ids=position_ids,
            training=training
        )
        hidden_states = residual + attn_output
        
        # Apply post-attention normalization and MLP/MoE with residual connection
        residual = hidden_states
        hidden_states = self.post_attention_layernorm(hidden_states)
        mlp_output = self.mlp(hidden_states, training=training)
        hidden_states = residual + mlp_output
        
        return hidden_states

# MODEL CONFIGURATION 

class DeepSeekConfig:
    """
    This is the configuration class to store the configuration of a [`DeepseekMoEModel`]. It is used to instantiate an DeepSeek
    model according to the specified arguments, defining the model architecture. Instantiating a configuration with the
    defaults below will yield a configuration for a small, stable, and computationally efficient MoE chatbot model.

    Args:
        vocab_size (`int`, *optional*, defaults to 12000):
            Vocabulary size of the Deep model. Defines the number of different tokens that can be represented by the
            `inputs_ids` passed when calling [`DeepseekModel`]. (Optimized for chatbot use case).
            
        hidden_size (`int`, *optional*, defaults to 256):
            Dimension of the hidden representations. (Optimized hidden size for stability and performance).
            
        intermediate_size (`int`, *optional*, defaults to 512):
            Dimension of the MLP representations. (Balanced intermediate size for MLP layers).
            
        moe_intermediate_size (`int`, *optional*, defaults to 256):
            Dimension of the MoE representations. (MoE-specific intermediate dimension).
            
        num_hidden_layers (`int`, *optional*, defaults to 4):
            Number of hidden layers in the Transformer decoder. (Optimal layer count for chatbot tasks).
            
        num_attention_heads (`int`, *optional*, defaults to 4):
            Number of attention heads for each attention layer in the Transformer decoder. (Balanced attention heads for computation).
        
        n_shared_experts (`int`, *optional*, defaults to 1):
            Number of shared experts, None means dense model. (Set to 1, a single shared expert for parameter efficiency).
        
        n_routed_experts (`int`, *optional*, defaults to 4):
            Number of routed experts, None means dense model. (Set to 4 for specialization).
            
        num_experts_per_tok (`int`, *optional*, defaults to 2):
            Number of selected experts, None means dense model. (Set to 2 experts per token for balanced load).
            
        moe_layer_freq (`int`, *optional*, defaults to 2):
            The frequency of the MoE layer: one expert layer for every `moe_layer_freq - 1` dense layers.
            (MoE every other layer for computational balance).
            
        first_k_dense_replace (`int`, *optional*, defaults to 1):
            - Number of dense layers in shallow layers(embed->dense->dense->...->dense->moe->moe...->lm_head).
            - k dense layers (First layer dense for stability).
        norm_topk_prob (`bool`, *optional*, defaults to False):
            Whether to normalize the weights of the routed experts. (No normalization for expert weights).
            
        scoring_func (`str`, *optional*, defaults to 'softmax'):
            Method of computing expert weights. (Standard softmax for expert selection).
            
        aux_loss_alpha (`float`, *optional*, defaults to 0.001):
            Auxiliary loss weight coefficient. (Default auxiliary loss weight).
            
        seq_aux = (`bool`, *optional*, defaults to True):
            Whether to compute the auxiliary loss for each individual sample. (Sequence-level auxiliary loss).
            
        num_key_value_heads (`int`, *optional*, defaults to 2):
            This is the number of key_value heads that should be used to implement Grouped Query Attention. If
            `num_key_value_heads=num_attention_heads`, the model will use Multi Head Attention (MHA), if
            `num_key_value_heads=1` the model will use Multi Query Attention (MQA) otherwise GQA is used.
            (Set to 2 for GQA and memory efficiency).
            
        hidden_act (`str` or `function`, *optional*, defaults to "swish"):
            The non-linear activation function (function or string) in the decoder. (Swish activation for better gradients).
        
        max_position_embeddings (`int`, *optional*, defaults to 256):
            The maximum sequence length that this model might ever be used with. (Sufficient context length for chatbot).
        
        initializer_range (`float`, *optional*, defaults to 0.001):
            The standard deviation of the truncated_normal_initializer for initializing all weight matrices. (Reduced initializer range for stability).
        
        rms_norm_eps (`float`, *optional*, defaults to 1e-6):
            The epsilon used by the rms normalization layers. (Standard RMS norm epsilon).
        
        use_cache (`bool`, *optional*, defaults to True):
            Whether or not the model should return the last key/values attentions (not used by all models). Only
            relevant if `config.is_decoder=True`. (Enable caching for inference).
        
        pad_token_id (`int`, *optional*):
            Padding token id.
        bos_token_id (`int`, *optional*, defaults to 1):
            Beginning of stream token id.
        eos_token_id (`int`, *optional*, defaults to 2):
            End of stream token id.
        
        pretraining_tp (`int`, *optional*, defaults to 1):
            Experimental feature. Tensor parallelism rank used during pretraining. Please refer to [this
            document](https://huggingface.co/docs/transformers/parallelism) to understand more about it. This value is
            necessary to ensure exact reproducibility of the pretraining results. Please refer to [this
            issue](https://github.com/pytorch/pytorch/issues/76232).
        
        tie_word_embeddings (`bool`, *optional*, defaults to False):
            Whether to tie weight embeddings
        
        rope_theta (`float`, *optional*, defaults to 10000.0):
            The base period of the RoPE embeddings. (Standard RoPE base period).
        
        rope_scaling (`Dict`, *optional*, defaults to {"type": "linear", "factor": 2.0}):
            Dictionary containing the scaling configuration for the RoPE embeddings. Currently supports two scaling
            strategies: linear and dynamic. Their scaling factor must be a float greater than 1. The expected format is
            `{"type": strategy name, "factor": scaling factor}`. When using this flag, don't update
            `max_position_embeddings` to the expected new maximum. (Linear RoPE scaling enabled).
        
        attention_bias (`bool`, defaults to False, *optional*, defaults to False):
            Whether to use a bias in the query, key, value and output projection layers during self-attention. (No attention bias for efficiency).
        
        attention_dropout (`float`, *optional*, defaults to 0.1):
            The dropout ratio for the attention probabilities. (Increased dropout for regularization).
    """

    def __init__(
        self,
        vocab_size=vocab_size,      # Default vocabulary size optimized for chatbot
        hidden_size=256,            # Optimized hidden size for stability and performance
        intermediate_size=512,      # Balanced intermediate size for MLP layers
        moe_intermediate_size=256,  # MoE-specific intermediate dimension
        num_hidden_layers=4,        # Optimal layer count for chatbot tasks
        num_attention_heads=4,      # Balanced attention heads for computation
        num_key_value_heads=2,      # GQA for memory efficiency
        n_shared_experts=3,         # Three shared experts for parameter efficiency (REVISED)
        n_routed_experts=6,         # Six routed experts for specialization     (REVISED)
        num_experts_per_tok=2,      # Two experts per token for balanced load
        moe_layer_freq=1,           # MoE every layer for computational balance (REVISED)
        first_k_dense_replace=1,    # First layer dense for stability
        norm_topk_prob=True,        # Normalization for expert weights          (REVISED)
        scoring_func='softmax',     # Standard softmax for expert selection
        aux_loss_alpha=0.0022997166087842954,  # Auxiliary loss weight          (REVISED)
        seq_aux=False,              # Token-level auxiliary loss                (REVISED)
        hidden_act="swish",         # Swish activation for better gradients
        max_position_embeddings=256,  # Sufficient context length for chatbot
        initializer_range=0.001,    # Reduced initializer range for stability
        rms_norm_eps=1e-6,          # Standard RMS norm epsilon
        use_cache=True,             # Enable caching for inference
        rope_theta=10000.0,         # Standard RoPE base period
        rope_scaling={"type": "linear", "factor": 2.0},  # Linear RoPE scaling
        attention_bias=False,       # No attention bias for efficiency
        attention_dropout=0.1       # Increased dropout for regularization
    ):
        # Initialize model architecture parameters with optimized values
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.intermediate_size = intermediate_size
        self.moe_intermediate_size = moe_intermediate_size
        self.num_hidden_layers = num_hidden_layers
        self.num_attention_heads = num_attention_heads
        self.num_key_value_heads = num_key_value_heads
        self.n_shared_experts = n_shared_experts
        self.n_routed_experts = n_routed_experts
        self.num_experts_per_tok = num_experts_per_tok
        self.moe_layer_freq = moe_layer_freq
        self.first_k_dense_replace = first_k_dense_replace
        self.norm_topk_prob = norm_topk_prob
        self.scoring_func = scoring_func
        self.aux_loss_alpha = aux_loss_alpha
        self.seq_aux = seq_aux
        self.hidden_act = hidden_act
        self.max_position_embeddings = max_position_embeddings
        self.initializer_range = initializer_range
        self.rms_norm_eps = rms_norm_eps
        self.use_cache = use_cache
        self.rope_theta = rope_theta
        self.rope_scaling = rope_scaling
        self.attention_bias = attention_bias
        self.attention_dropout = attention_dropout

class DeepSeekModel(tf.keras.Model):
    """Core DeepSeek transformer model for chatbot, provide raw hidden states"""
    def __init__(self, config, **kwargs):
        super().__init__(**kwargs)
        self.config = config
        self.padding_idx = 0
        
        # Initialize embedding layer for token inputs
        self.embed_tokens = layers.Embedding(
            config.vocab_size,
            config.hidden_size,
            embeddings_initializer=tf.keras.initializers.RandomNormal(
                stddev=config.initializer_range
            ),
            name="embed_tokens"
        )
        
        # Initialize decoder layers
        self.decoder_layers = [
            DeepSeekDecoderLayer(config, layer_idx=i, name=f"layer_{i}")
            for i in range(config.num_hidden_layers)
        ]
        
        # Initialize final normalization layer
        self.norm = DeepSeekRMSNorm(
            config.hidden_size, eps=config.rms_norm_eps, name="norm"
        )
        
        # Initialize expert assignment counter for MoE layers
        self.expert_assignment_counts = tf.Variable(
            tf.zeros([config.n_routed_experts], dtype=tf.int32),
            trainable=False,
            name="expert_assignment_counts"
        ) if config.n_routed_experts else None
        
    def call(self, input_ids, attention_mask=None, position_ids=None, training=False):
        # Embed input tokens
        hidden_states = self.embed_tokens(input_ids)
        
        # Create causal mask if not provided
        if attention_mask is None:
            seq_len = tf.shape(input_ids)[1]
            attention_mask = self._create_causal_mask(seq_len)
            
        # Prepare attention mask for causal attention
        attention_mask = tf.cast(attention_mask, hidden_states.dtype)
        if len(attention_mask.shape) == 2:
            attention_mask = tf.expand_dims(tf.expand_dims(attention_mask, 1), 1)
        elif len(attention_mask.shape) == 3:
            attention_mask = tf.expand_dims(attention_mask, 1)
        attention_mask = (tf.cast(1.0, hidden_states.dtype) - attention_mask) * tf.cast(-1e4, hidden_states.dtype)
        
        # Process through decoder layers and aggregate expert counts
        for layer in self.decoder_layers:
            hidden_states = layer(
                hidden_states,
                attention_mask=attention_mask,
                position_ids=position_ids,
                training=training
            )
            # Update expert assignment counts for MoE layers during training
            if training and hasattr(layer, 'mlp') and isinstance(layer.mlp, DeepSeekMoE):
                expert_counts = layer.mlp.gate.expert_counts
                if self.expert_assignment_counts is not None:
                    self.expert_assignment_counts.assign_add(expert_counts)
            
        # Apply final normalization
        return self.norm(hidden_states)
        
    def _create_causal_mask(self, seq_len):
        """Create causal attention mask to prevent attending to future tokens"""
        mask = tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
        return tf.cast(mask, tf.float16)

class DeepSeekForCausalLM(tf.keras.Model):
    """Causal language model for chatbot text generation"""
    def __init__(self, config, **kwargs):
        super().__init__(**kwargs)
        self.config = config
        self.transformer = DeepSeekModel(config, name="transformer")
        # Initialize language model head
        self.lm_head = layers.Dense(
            config.vocab_size,
            kernel_initializer=tf.keras.initializers.RandomNormal(
                stddev=config.initializer_range
            ),
            use_bias=False,
            name="lm_head"
        )
        
    def call(self, inputs, attention_mask=None, position_ids=None, training=False):
        # Handle dictionary or tensor inputs
        if isinstance(inputs, dict):
            input_ids = inputs.get('input_ids')
            attention_mask = inputs.get('attention_mask', attention_mask)
        else:
            input_ids = inputs
        
        # Process through transformer
        hidden_states = self.transformer(
            input_ids,
            attention_mask=attention_mask,
            position_ids=position_ids,
            training=training
        )
        # Generate logits for token prediction
        return self.lm_head(hidden_states)

# METRICS AND CALLBACKS
class ResourceMonitor(tf.keras.callbacks.Callback):
    """Callback to monitor system resources (GPU, CPU, memory) during training"""
    def __init__(self, batch_size=2):
        super().__init__()
        self.batch_size = batch_size
        self.resource_metrics = []  # Store resource usage metrics
        self.start_time = time.time()
        self.total_tokens_processed = 0
        
    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()
        self.epoch_tokens = 0
        
    def on_batch_end(self, batch, logs=None):
        # Update token count for throughput calculation
        self.total_tokens_processed += self.batch_size * max_seq_len
        self.epoch_tokens += self.batch_size * max_seq_len
        
    def on_epoch_end(self, epoch, logs=None):
        try:
            # Collect GPU and system resource usage
            gpus = GPUtil.getGPUs()
            gpu_memory = sum([gpu.memoryUsed for gpu in gpus]) / len(gpus) if gpus else 0
            gpu_usage = sum([gpu.load * 100 for gpu in gpus]) / len(gpus) if gpus else 0
            cpu_usage = psutil.cpu_percent()
            memory_info = psutil.virtual_memory()
            system_memory = memory_info.used / (1024 ** 3)
            epoch_time = time.time() - self.epoch_start_time
            tokens_per_second = self.epoch_tokens / epoch_time if epoch_time > 0 else 0
            
            # Store resource metrics
            resource_stats = {
                'epoch': epoch + 1,
                'avg_gpu_memory_gb': float(gpu_memory / 1024),
                'avg_system_memory_gb': float(system_memory),
                'avg_gpu_usage_percent': float(gpu_usage),
                'avg_cpu_usage_percent': float(cpu_usage),
                'tokens_per_second': float(tokens_per_second),
                'throughput_tps': float(tokens_per_second)
            }
            
            self.resource_metrics.append(resource_stats)
            
            # Print resource usage summary
            print(f"Epoch {epoch+1} Resources: GPU Mem: {gpu_memory/1024:.1f}GB, "
                  f"System Mem: {system_memory:.1f}GB, GPU: {gpu_usage:.1f}%, "
                  f"CPU: {cpu_usage:.1f}%, TPS: {tokens_per_second:.0f}")
                  
        except Exception as e:
            print(f"Resource monitoring error: {e}")

class DeepSeekMetrics(tf.keras.callbacks.Callback):
    """Callback to track specific metrics, including MaxVIO for expert balancing"""
    def __init__(self, validation_data, tokenizer):
        super().__init__()
        self.validation_data = validation_data
        self.tokenizer = tokenizer
        self.metrics_history = []
        self.start_time = time.time()
        self.max_vio_values = []
        
    def on_epoch_begin(self, epoch, logs=None):
        # Reset expert assignment counts for accurate per-epoch metrics
        if hasattr(self.model.transformer, 'expert_assignment_counts') and self.model.transformer.expert_assignment_counts is not None:
            self.model.transformer.expert_assignment_counts.assign(tf.zeros([self.model.config.n_routed_experts], dtype=tf.int32))
        for layer in self.model.transformer.decoder_layers:
            if hasattr(layer, 'mlp') and isinstance(layer.mlp, DeepSeekMoE):
                layer.mlp.gate.expert_counts.assign(tf.zeros([layer.mlp.n_routed_experts], dtype=tf.int32))
        
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        
        # Estimate FLOPs for computational efficiency tracking
        total_params = self.model.count_params()
        flops_per_token = total_params * 2
        total_flops = flops_per_token * batch_size * max_seq_len
        total_gflops = total_flops / 1e9
        
        logs['gflops'] = float(total_gflops)
        
        # Calculate MaxVIO for expert load balancing
        if self.validation_data:
            try:
                max_vio = self._calculate_max_vio()
                logs['max_vio'] = float(max_vio)
                self.max_vio_values.append(max_vio)
                
            except Exception as e:
                print(f"Metrics calculation error: {e}")
                logs['max_vio'] = 0.0
                
        self.metrics_history.append(logs.copy())
        
        # Print simplified metrics summary
        print(f"Epoch {epoch+1}: Loss={logs.get('loss', 0):.4f}, "
              f"Val_Loss={logs.get('val_loss', 0):.4f}, "
              f"GFLOPs={total_gflops:.2f}, "
              f"MaxVIO={logs.get('max_vio', 0):.4f}")
              
    def _calculate_max_vio(self):
        """Calculate Maximum Violation (MaxVIO) for expert load balancing"""
        total_counts = np.zeros(self.model.config.n_routed_experts, dtype=np.float32)
        moe_layers = 0
        # Aggregate expert counts across MoE layers
        for layer in self.model.transformer.decoder_layers:
            if hasattr(layer, 'mlp') and isinstance(layer.mlp, DeepSeekMoE):
                counts = layer.mlp.gate.expert_counts.numpy()
                total_counts += counts
                moe_layers += 1
        
        # Compute MaxVIO based on aggregated counts
        if moe_layers > 0 and np.sum(total_counts) > 0:
            normalized_counts = total_counts / (np.sum(total_counts) + 1e-10)
            expected_uniform = 1.0 / len(total_counts)
            max_vio = np.max(np.abs(normalized_counts - expected_uniform)) / expected_uniform
            return max_vio
        return 0.0

class ComprehensiveModelCheckpoint(tf.keras.callbacks.Callback):
    """Callback for saving model checkpoints based on monitored metric"""
    def __init__(self, filepath, monitor='val_loss', save_best_only=True, mode='min'):
        super().__init__()
        self.filepath = filepath
        self.monitor = monitor
        self.save_best_only = save_best_only
        self.mode = mode
        self.best_value = float('inf') if mode == 'min' else -float('inf')
        
    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            return
            
        current_value = logs.get(self.monitor)
        if current_value is None:
            return
            
        # Determine if current model should be saved
        if self.mode == 'min':
            should_save = current_value < self.best_value
        else:
            should_save = current_value > self.best_value
            
        if should_save or not self.save_best_only:
            self.best_value = current_value
            self.model.save_weights(self.filepath)
            print(f"Model checkpoint saved with {self.monitor}: {current_value:.4f}")


# Function to calculate perplexity on validation set
def calculate_perplexity(model, dataset, tokenizer, max_batches=100):          # Sample batch for calculate perplexity
    """Calculate perplexity on validation dataset to evaluate model performance"""
    total_loss = 0.0
    total_tokens = 0
    PAD_TOKEN_ID = tokenizer.token_to_id(PAD_TOKEN)
    
    for i, batch in enumerate(dataset.take(max_batches)):
        inputs, labels = batch
        logits = model(inputs, training=False)
        # Clip logits for numerical stability
        logits = tf.clip_by_value(logits, -50.0, 50.0)
        log_probs = tf.nn.log_softmax(logits, axis=-1)
        # Compute per-token loss
        per_token_loss = -tf.gather(log_probs, labels, batch_dims=2)
        if tf.reduce_any(tf.math.is_nan(per_token_loss)):
            continue
        # Mask padding tokens
        mask = tf.cast(labels != PAD_TOKEN_ID, per_token_loss.dtype)
        batch_loss = tf.reduce_sum(per_token_loss * mask)
        batch_tokens = tf.reduce_sum(mask)
        if tf.math.is_nan(batch_loss) or batch_tokens == 0:
            continue
        total_loss += batch_loss.numpy()
        total_tokens += batch_tokens.numpy()
        
    # Compute perplexity as exponential of average loss
    if total_tokens > 0:
        avg_loss = total_loss / total_tokens
        return math.exp(min(avg_loss, 20.0))
    return float('inf')

Physical devices cannot be modified after being initialized
TensorFlow version: 2.15.0
Mixed precision policy: mixed_float16
GPU available: True


In [29]:
# Visualize the model
# Create model with specified vocabulary size using direct config
vocab_size = tokenizer.get_vocab_size()
# Use the optimized config directly with the same hyperparameters from create_deepseek_chatbot_model
config = DeepSeekConfig(vocab_size=vocab_size)
model = DeepSeekForCausalLM(config)


# Build model
if tf_train_dataset:
    sample_batch = next(iter(tf_train_dataset.take(1)))
    model(sample_batch[0])

# Configure optimizer with more conservative settings
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4,  # Reduced learning rate
    decay_steps=1000,
    decay_rate=0.96
)

optimizer = tf.keras.optimizers.AdamW(
    learning_rate=lr_schedule,
    beta_1=0.9,
    beta_2=0.95,
    weight_decay=0.1,
    clipnorm=1.0
)

model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, 
        ignore_class=PAD_TOKEN_ID,  # Use correct PAD token ID
        reduction=tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE
    ),
    metrics=['accuracy']
)

print("\nModel Summary")
model.summary()


Model Summary
Model: "deep_seek_for_causal_lm_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (DeepSeekModel  multiple                  6918168   
 )                                                               
                                                                 
 lm_head (Dense)             multiple                  292096    
                                                                 
Total params: 7210264 (27.50 MB)
Trainable params: 7079168 (27.00 MB)
Non-trainable params: 131096 (512.09 KB)
_________________________________________________________________


## Model Training Experiment Loop

In [30]:
# Collect training and results storage
results_table = {
    'Average TPS': {},
    'Final Training Loss': {},
    'Final Validation Loss': {},
    'Computational Resource Usage': {},
    'Average CPU Usage (percent)': {},
    'Average GPU Usage (percent)': {},
    'Average Memory (GB)': {},
    'Average GPU Memory (GB)': {},
    'Average FLOPS Estimate (GFLOPS)': {},
    'Final Validation Perplexity': {},
    'Average MaxVIO_global': {},
    'Lowest MaxVIO_global': {},
    'Highest MaxVIO_global': {},
}

best_perplexity = float('inf')
# best_iteration = 0
BEST_MODEL_PATH = 'best_deepseek_chatbot_model'

# number training model iteration
num_iteration = 5

for iteration in range(num_iteration):
    print(f"\nStarting Training Iteration {iteration + 1}")
    # Clear TensorFlow session and memory
    tf.keras.backend.clear_session()
    gc.collect()

    # Create model with specified vocabulary size using direct config
    vocab_size = tokenizer.get_vocab_size()
    # Use the optimized config directly with the same hyperparameters from create_deepseek_chatbot_model
    config = DeepSeekConfig(vocab_size=vocab_size)
    model = DeepSeekForCausalLM(config)

    # Build model with sample batch
    if tf_train_dataset:
        sample_batch = next(iter(tf_train_dataset.take(1)))
        model(sample_batch[0])

    # Configure optimizer with exponential decay learning rate
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=1e-4,
        decay_steps=1000,
        decay_rate=0.9
    )

    optimizer = tf.keras.optimizers.AdamW(
        learning_rate=lr_schedule,
        beta_1=0.9,
        beta_2=0.95,
        weight_decay=0.01,
        clipnorm=1.0
    )

    PAD_TOKEN_ID = tokenizer.token_to_id(PAD_TOKEN)

    # Compile model with sparse categorical crossentropy loss
    model.compile(
        optimizer=optimizer,
        loss=tf.keras.losses.SparseCategoricalCrossentropy(
            from_logits=True, 
            ignore_class=PAD_TOKEN_ID
        ),
        metrics=['accuracy']
    )

    # Define training callbacks
    metrics_callback = DeepSeekMetrics(tf_val_dataset, tokenizer) if tf_val_dataset else None
    resource_monitor = ResourceMonitor(batch_size=2)

    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            patience=3, 
            restore_best_weights=True, 
            monitor='val_loss',
            min_delta=0.01
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            factor=0.5, 
            patience=2, 
            monitor='val_loss',
            min_delta=0.01,
            min_lr=1e-6
        ),
        tf.keras.callbacks.TerminateOnNaN(),
    ]

    if metrics_callback:
        callbacks.append(metrics_callback)
    if resource_monitor:
        callbacks.append(resource_monitor)

    # Train model on GPU
    print("Starting training...")
    start_time = time.time()

    # Force GPU for training the model
    with tf.device('/GPU:0'):
        history = model.fit(
            tf_train_dataset,
            epochs=100,
            validation_data=tf_val_dataset,
            callbacks=callbacks,
            verbose=1
        )
    total_training_time = time.time() - start_time

    # Calculate final metrics
    print("Calculating final perplexity")
    final_perplexity = calculate_perplexity(model, tf_val_dataset, tokenizer, max_batches=50) if tf_val_dataset else float('inf')
    final_training_loss = history.history['loss'][-1] if 'loss' in history.history else float('inf')
    final_validation_loss = history.history['val_loss'][-1] if 'val_loss' in history.history else float('inf')

    print(f"Final Training Loss: {final_training_loss:.4f}")
    print(f"Final Validation Loss: {final_validation_loss:.4f}")
    print(f"Final Perplexity: {final_perplexity:.2f}")

    # Save best model based on perplexity
    if final_perplexity < best_perplexity:
        best_perplexity = final_perplexity
        best_iteration = iteration + 1

        try:
            if os.path.exists(BEST_MODEL_PATH):
                shutil.rmtree(BEST_MODEL_PATH)
            model.save(BEST_MODEL_PATH, save_format='tf')
            print(f"New best model saved from iteration {iteration + 1} with perplexity: {final_perplexity:.2f}")
        except Exception as e:
            print(f"Error saving best model: {e}")

    # Collect and store resource metrics
    if resource_monitor.resource_metrics:
        avg_gpu_memory = np.mean([m['avg_gpu_memory_gb'] for m in resource_monitor.resource_metrics])
        avg_system_memory = np.mean([m['avg_system_memory_gb'] for m in resource_monitor.resource_metrics])
        avg_gpu_usage = np.mean([m['avg_gpu_usage_percent'] for m in resource_monitor.resource_metrics])
        avg_cpu_usage = np.mean([m['avg_cpu_usage_percent'] for m in resource_monitor.resource_metrics])
        avg_tps = np.mean([m['tokens_per_second'] for m in resource_monitor.resource_metrics])
    else:
        avg_gpu_memory = avg_system_memory = avg_gpu_usage = avg_cpu_usage = avg_tps = 0

    # Calculate FLOPs
    total_params = model.count_params()
    batch_size = 2
    seq_length = max_seq_len
    flops_per_token = total_params * 2
    total_flops = flops_per_token * batch_size * seq_length
    total_gflops = total_flops / 1e9

    # Collect MaxVIO metrics
    if metrics_callback and hasattr(metrics_callback, 'max_vio_values'):
        max_vio_values = metrics_callback.max_vio_values
        if max_vio_values:
            avg_max_vio = np.mean(max_vio_values)
            min_max_vio = np.min(max_vio_values)
            max_max_vio = np.max(max_vio_values)
        else:
            avg_max_vio = min_max_vio = max_max_vio = 0.0
    else:
        avg_max_vio = min_max_vio = max_max_vio = 0.0

    # Store results in results table
    iteration_key = f'Iteration {iteration + 1}'
    results_table['Average TPS'][iteration_key] = avg_tps
    results_table['Final Training Loss'][iteration_key] = final_training_loss
    results_table['Final Validation Loss'][iteration_key] = final_validation_loss
    results_table['Computational Resource Usage'][iteration_key] = avg_cpu_usage + avg_gpu_usage
    results_table['Average CPU Usage (percent)'][iteration_key] = avg_cpu_usage
    results_table['Average GPU Usage (percent)'][iteration_key] = avg_gpu_usage
    results_table['Average Memory (GB)'][iteration_key] = avg_system_memory
    results_table['Average GPU Memory (GB)'][iteration_key] = avg_gpu_memory
    results_table['Average FLOPS Estimate (GFLOPS)'][iteration_key] = total_gflops
    results_table['Final Validation Perplexity'][iteration_key] = final_perplexity
    results_table['Average MaxVIO_global'][iteration_key] = avg_max_vio
    results_table['Lowest MaxVIO_global'][iteration_key] = min_max_vio
    results_table['Highest MaxVIO_global'][iteration_key] = max_max_vio

    print(f"Result {iteration + 1} completed in {total_training_time:.2f} seconds")
    print(f"Average TPS: {avg_tps:.0f}")

# Calculate average metrics across iterations
for key in results_table:
    values = [results_table[key][f'Iteration {i+1}'] for i in range(num_iteration) if f'Iteration {i+1}' in results_table[key]]
    results_table[key]['Average'] = np.mean(values) if values else 0

# Create and display results DataFrame
final_result = pd.DataFrame(results_table)
final_result = final_result.T

# Save final model weights
model.save_weights('final_deepseek_chatbot_weights.h5')

print("\nTraining Results Summary:")
final_result


Starting Training Iteration 1
Starting training...
55/55 [==============================] - ETA: 0s - loss: 6.6434 - accuracy: 0.0187Epoch 1: Loss=6.6434, Val_Loss=6.1649, GFLOPs=2.02, MaxVIO=0.0149
Epoch 1 Resources: GPU Mem: 14.3GB, System Mem: 6.9GB, GPU: 10.0%, CPU: 28.1%, TPS: 201
55/55 [==============================] - 38s 172ms/step - loss: 6.6434 - accuracy: 0.0187 - val_loss: 6.1649 - val_accuracy: 0.0209 - lr: 9.9433e-05 - gflops: 2.0189 - max_vio: 0.0149
Calculating final perplexity
Final Training Loss: 6.6434
Final Validation Loss: 6.1649
Final Perplexity: 479.32
New best model saved from iteration 1 with perplexity: 479.32
Result 1 completed in 38.27 seconds
Average TPS: 201

Starting Training Iteration 2
Starting training...
55/55 [==============================] - ETA: 0s - loss: 6.6407 - accuracy: 0.0208Epoch 1: Loss=6.6407, Val_Loss=6.1621, GFLOPs=2.02, MaxVIO=0.0332
Epoch 1 Resources: GPU Mem: 14.3GB, System Mem: 7.8GB, GPU: 15.0%, CPU: 22.7%, TPS: 213
55/55 [======

,Iteration 1,Iteration 2,Average
Average TPS,201.341918,212.828149,207.085033
Final Training Loss,6.643372,6.640678,6.642025
Final Validation Loss,6.164899,6.162109,6.163504
Computational Resource Usage,38.100000,37.700000,37.900000
Average CPU Usage (percent),28.100000,22.700000,25.400000
Average GPU Usage (percent),10.000000,15.000000,12.500000
Average Memory (GB),6.876156,7.781719,7.328938
Average GPU Memory (GB),14.272461,14.272461,14.272461
Average FLOPS Estimate (GFLOPS),2.018874,2.018874,2.018874
Final Validation Perplexity,479.324046,483.965125,481.644585


# Model Evaluation

## Model Loading

In [31]:
# Disable XLA (XLA JIT compilation) to prevent CUDA graph errors and ensure stability
tf.config.optimizer.set_jit(False)  # Disabled XLA to prevent CUDA graph errors

# Configure GPU memory growth to prevent memory allocation issues
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Enable mixed precision training with FP16 for improved performance and reduced memory usage
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

# Configuration parameters from the model and preprocessing pipeline
vocab_size = 12000
max_seq_len = 70
batch_size = 2
PAD_TOKEN = "<pad>"
BOS_TOKEN = "<bos>"
EOS_TOKEN = "<eos>"
SEP_TOKEN = "<sep>"

# Print TensorFlow version, mixed precision policy, and GPU availability for debugging
print(f"TensorFlow version: {tf.__version__}")
print(f"Mixed precision policy: {mixed_precision.global_policy().name}")
print(f"GPU available: {len(gpus) > 0}")

# DEEPSEEK MOE COMPONENTS

class DeepSeekRMSNorm(layers.Layer):
    """RMS Normalization layer """
    def __init__(self, hidden_size, eps=1e-6, **kwargs):
        super().__init__(**kwargs)
        self.hidden_size = hidden_size  # Dimension of the hidden representations
        self.variance_epsilon = eps     # Small constant to avoid division by zero
        
    def build(self, input_shape):
        # Initialize learnable weight parameter for scaling normalized output
        self.weight = self.add_weight(
            name='weight',
            shape=(self.hidden_size,),
            initializer='ones',
            trainable=True
        )
        super().build(input_shape)
        
    def call(self, x):
        input_dtype = x.dtype
        x = tf.cast(x, tf.float32)  # Cast input to float32 for stable computation
        
        # Compute variance as mean of squared inputs along the last dimension
        variance = tf.reduce_mean(tf.square(x), axis=-1, keepdims=True)
        # Normalize input using RMS formula with epsilon for numerical stability
        x = x * tf.math.rsqrt(variance + self.variance_epsilon)
        
        # Scale normalized output with learnable weight and cast back to original dtype
        return self.weight * tf.cast(x, input_dtype)

class RotaryPositionEmbedding(layers.Layer):
    """Rotary Position Embedding (RoPE) with dynamic NTK scaling for positional encoding"""
    def __init__(self, dim, max_position_embeddings=2048, base=10000, scaling_type=None, scaling_factor=1.0, **kwargs):
        super().__init__(**kwargs)
        self.dim = dim                              # Dimension of the head embeddings
        self.max_position_embeddings = max_position_embeddings  # Maximum sequence length
        self.base = base                           # Base value for frequency calculation
        self.scaling_type = scaling_type           # Type of scaling ("linear" or "dynamic")
        self.scaling_factor = scaling_factor       # Scaling factor for RoPE
        
    def build(self, input_shape):
        # Compute inverse frequencies for rotary embeddings
        inv_freq = 1.0 / (self.base ** (tf.range(0, self.dim, 2, dtype=tf.float32) / tf.cast(self.dim, tf.float32)))
        self.inv_freq = tf.constant(inv_freq, dtype=tf.float32)
        self._compute_rotary_cache()  # Precompute cosine and sine caches
        super().build(input_shape)
        
    def _compute_rotary_cache(self):
        """Compute cosine and sine caches for rotary embeddings with optional scaling"""
        if self.scaling_type == "linear":
            # Linear scaling: divide positions by scaling factor
            t = tf.range(self.max_position_embeddings, dtype=tf.float32) / self.scaling_factor
        elif self.scaling_type == "dynamic":
            # Dynamic NTK scaling for extended context lengths
            if self.max_position_embeddings > self.base:
                base = self.base * (
                    (self.scaling_factor * self.max_position_embeddings / self.base) - 
                    (self.scaling_factor - 1)
                ) ** (self.dim / (self.dim - 2))
                inv_freq = 1.0 / (base ** (tf.range(0, self.dim, 2, dtype=tf.float32) / tf.cast(self.dim, tf.float32)))
                self.inv_freq = tf.constant(inv_freq, dtype=tf.float32)
            t = tf.range(self.max_position_embeddings, dtype=tf.float32)
        else:
            # Default case: no scaling
            t = tf.range(self.max_position_embeddings, dtype=tf.float32)
            
        # Compute frequency matrix for rotary embeddings
        freqs = tf.einsum('i,j->ij', t, self.inv_freq)
        emb = tf.concat([freqs, freqs], axis=-1)  # Concatenate for full embedding dimension
        
        # Cache cosine and sine values as non-trainable variables
        self.cos_cached = tf.Variable(tf.cos(emb), trainable=False, name='cos_cached')
        self.sin_cached = tf.Variable(tf.sin(emb), trainable=False, name='sin_cached')
        
    def call(self, x, position_ids=None, seq_len=None):
        # Get input tensor dimensions
        batch_size, seq_len_x, num_heads, head_dim = tf.unstack(tf.shape(x))
        
        # Generate position IDs if not provided
        if position_ids is None:
            position_ids = tf.range(seq_len_x, dtype=tf.int32)
            position_ids = tf.tile(tf.reshape(position_ids, [1, -1]), [batch_size, 1])
        
        # Gather precomputed cosine and sine values for given position IDs
        cos = tf.gather(self.cos_cached, position_ids, axis=0)  # Shape: [batch_size, seq_len_x, head_dim]
        sin = tf.gather(self.sin_cached, position_ids, axis=0)  # Shape: [batch_size, seq_len_x, head_dim]
        
        # Split cosine and sine for application to x1 and x2
        cos_1, cos_2 = tf.split(cos, 2, axis=-1)
        sin_1, sin_2 = tf.split(sin, 2, axis=-1)
        
        # Reshape for broadcasting with attention heads
        cos_1 = tf.reshape(cos_1, [batch_size, seq_len_x, 1, head_dim // 2])
        sin_1 = tf.reshape(sin_1, [batch_size, seq_len_x, 1, head_dim // 2])
        cos_2 = tf.reshape(cos_2, [batch_size, seq_len_x, 1, head_dim // 2])
        sin_2 = tf.reshape(sin_2, [batch_size, seq_len_x, 1, head_dim // 2])
        
        # Repeat to match number of attention heads
        cos_1 = tf.repeat(cos_1, num_heads, axis=2)
        sin_1 = tf.repeat(sin_1, num_heads, axis=2)
        cos_2 = tf.repeat(cos_2, num_heads, axis=2)
        sin_2 = tf.repeat(sin_2, num_heads, axis=2)
        
        # Cast to input dtype for mixed precision compatibility
        x_dtype = x.dtype
        cos_1 = tf.cast(cos_1, x_dtype)
        sin_1 = tf.cast(sin_1, x_dtype)
        cos_2 = tf.cast(cos_2, x_dtype)
        sin_2 = tf.cast(sin_2, x_dtype)
        
        # Split input tensor into two parts for rotary application
        x1, x2 = tf.split(x, 2, axis=-1)
        
        # Apply rotary embeddings to compute rotated representations
        rotated_x1 = x1 * cos_1 - x2 * sin_1
        rotated_x2 = x1 * sin_2 + x2 * cos_2
        
        # Concatenate rotated parts to restore original shape
        return tf.concat([rotated_x1, rotated_x2], axis=-1)

class DeepSeekMLP(layers.Layer):
    """Multi-Layer Perceptron (MLP) with Swish activation"""
    def __init__(self, hidden_size, intermediate_size, hidden_act="swish", **kwargs):
        super().__init__(**kwargs)
        self.hidden_size = hidden_size           # Input and output dimension
        self.intermediate_size = intermediate_size  # Intermediate layer dimension
        self.hidden_act = hidden_act            # Activation function (Swish/SILU)
        
    def build(self, input_shape):
        # Define gate projection layer with reduced initializer variance for stability
        self.gate_proj = layers.Dense(
            self.intermediate_size,
            kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.001),
            use_bias=False
        )
        # Define up projection layer
        self.up_proj = layers.Dense(
            self.intermediate_size,
            kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.001),
            use_bias=False
        )
        # Define down projection layer to return to hidden size
        self.down_proj = layers.Dense(
            self.hidden_size,
            kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.001),
            use_bias=False
        )
        super().build(input_shape)
        
    def call(self, x):
        # Apply gate and up projections
        gate_output = self.gate_proj(x)
        up_output = self.up_proj(x)
        # Apply Swish (SiLU) activation to gate output and multiply with up output
        activated = tf.nn.silu(gate_output) * up_output
        # Project back to hidden size
        return self.down_proj(activated)

class MoEGate(layers.Layer):
    """Mixture of Experts (MoE) gating mechanism with load balancing auxiliary loss"""
    def __init__(self, num_experts, num_experts_per_tok, hidden_size, 
                 scoring_func='softmax', norm_topk_prob=False, 
                 aux_loss_alpha=0.001, seq_aux=True, **kwargs):
        super().__init__(**kwargs)
        self.num_experts = num_experts                  # Total number of experts
        self.num_experts_per_tok = num_experts_per_tok  # Number of experts per token
        self.hidden_size = hidden_size                  # Input dimension
        self.scoring_func = scoring_func                # Scoring function for expert selection
        self.norm_topk_prob = norm_topk_prob            # Normalize top-k probabilities
        self.aux_loss_alpha = aux_loss_alpha            # Weight for auxiliary loss
        self.seq_aux = seq_aux                          # Compute auxiliary loss per sequence
        
    def build(self, input_shape):
        # Initialize gating weights with reduced variance for stability
        self.weight = self.add_weight(
            name='gate_weight',
            shape=(self.num_experts, self.hidden_size),
            initializer=tf.keras.initializers.RandomNormal(stddev=0.001),
            trainable=True
        )
        # Initialize expert assignment counter for tracking during training
        self.expert_counts = tf.Variable(tf.zeros([self.num_experts], dtype=tf.int32), trainable=False)
        super().build(input_shape)
        
    def call(self, hidden_states, training=False):
        # Get input dimensions
        batch_size, seq_len, h = tf.unstack(tf.shape(hidden_states))
        hidden_states_flat = tf.reshape(hidden_states, [-1, h])
        # Compute gating logits
        logits = tf.matmul(hidden_states_flat, self.weight, transpose_b=True)
        
        # Clip logits for numerical stability
        logits = tf.clip_by_value(logits, -50.0, 50.0)
        
        # Apply scoring function (softmax) to compute expert scores
        if self.scoring_func == 'softmax':
            temperature = 0.5                                         # Temperature for scaling logits
            scaled_logits = tf.cast(logits, tf.float32) / temperature
            scores = tf.nn.softmax(scaled_logits, axis=-1)
            scores = tf.cast(scores, hidden_states.dtype)
        else:
            scores = tf.nn.softmax(tf.cast(logits, tf.float32), axis=-1)
            scores = tf.cast(scores, hidden_states.dtype)

        # Select top-k experts and their weights
        topk_weights, topk_indices = tf.math.top_k(
            scores, k=self.num_experts_per_tok, sorted=False
        )

        # Normalize top-k weights if specified
        if self.num_experts_per_tok > 1 and self.norm_topk_prob:
            denominator = tf.reduce_sum(topk_weights, axis=-1, keepdims=True) + 1e-20
            topk_weights = topk_weights / denominator

        # Update expert assignment counts during training
        if training:
            flat_topk_indices = tf.reshape(topk_indices, [-1])
            counts = tf.reduce_sum(tf.one_hot(flat_topk_indices, depth=self.num_experts), axis=0)
            self.expert_counts.assign_add(tf.cast(counts, tf.int32))

        aux_loss = None
        if training and self.aux_loss_alpha > 0.0:
            if self.seq_aux:
                # Sequence-level auxiliary loss for load balancing
                scores_seq = tf.reshape(scores, [batch_size, seq_len, -1])
                topk_indices_seq = tf.reshape(topk_indices, [batch_size, -1])
                ce = tf.zeros([batch_size, self.num_experts], dtype=tf.float32)
                mask = tf.one_hot(topk_indices_seq, depth=self.num_experts)
                ce = tf.reduce_sum(mask, axis=1)
                seq_len_float = tf.cast(seq_len, tf.float32)
                num_experts_per_tok_float = tf.cast(self.num_experts_per_tok, tf.float32)
                num_experts_float = tf.cast(self.num_experts, tf.float32)
                ce = ce / (seq_len_float * num_experts_per_tok_float / num_experts_float)
                scores_seq_mean = tf.reduce_mean(tf.cast(scores_seq, tf.float32), axis=1)
                aux_loss = tf.reduce_sum(ce * scores_seq_mean, axis=1)
                aux_loss = tf.reduce_mean(aux_loss) * self.aux_loss_alpha
            else:
                # Token-level auxiliary loss
                mask_ce = tf.one_hot(tf.reshape(topk_indices, [-1]), depth=self.num_experts)
                ce = tf.reduce_mean(tf.cast(mask_ce, tf.float32), axis=0)
                Pi = tf.reduce_mean(tf.cast(scores, tf.float32), axis=0)
                fi = ce * self.num_experts
                aux_loss = tf.reduce_sum(Pi * fi) * self.aux_loss_alpha

            # Add auxiliary loss to the layer's losses
            self.add_loss(aux_loss)

        return topk_indices, topk_weights, aux_loss

class DeepSeekMoE(layers.Layer):
    """Mixture of Experts (MoE) layer with shared and routed experts"""
    def __init__(self, hidden_size, n_routed_experts, n_shared_experts, 
                 num_experts_per_tok, moe_intermediate_size, **kwargs):
        super().__init__(**kwargs)
        self.hidden_size = hidden_size                      # Input and output dimension
        self.n_routed_experts = n_routed_experts            # Number of routed experts
        self.n_shared_experts = n_shared_experts            # Number of shared experts
        self.num_experts_per_tok = num_experts_per_tok      # Experts per token
        self.moe_intermediate_size = moe_intermediate_size  # MoE intermediate size
        
    def build(self, input_shape):
        # Initialize list of routed expert MLPs
        self.routed_experts = [
            DeepSeekMLP(
                self.hidden_size, 
                self.moe_intermediate_size,
                name=f"routed_expert_{i}"
            ) for i in range(self.n_routed_experts)
        ]
        
        # Initialize shared experts if specified
        if self.n_shared_experts is not None:
            shared_intermediate_size = self.moe_intermediate_size * self.n_shared_experts
            self.shared_experts = DeepSeekMLP(
                self.hidden_size,
                shared_intermediate_size,
                name="shared_experts"
            )
        else:
            self.shared_experts = None
            
        # Initialize MoE gating mechanism
        self.gate = MoEGate(
            self.n_routed_experts,
            self.num_experts_per_tok,
            self.hidden_size,
            name="moe_gate"
        )
        
        # Build each expert and gate
        for expert in self.routed_experts:
            expert.build(input_shape)
        if self.shared_experts:
            self.shared_experts.build(input_shape)
        self.gate.build(input_shape)
        
        super().build(input_shape)
        
    def call(self, hidden_states, training=False):
        identity = hidden_states  # Store input for residual connection
        orig_shape = tf.shape(hidden_states)
        # Get top-k expert indices and weights
        topk_idx, topk_weight, aux_loss = self.gate(hidden_states, training=training)
        hidden_states_flat = tf.reshape(hidden_states, [-1, self.hidden_size])
        
        if training:
            # Expand inputs for top-k experts during training
            hidden_states_expanded = tf.repeat(
                hidden_states_flat, self.num_experts_per_tok, axis=0
            )
            flat_topk_idx = tf.reshape(topk_idx, [-1])
            combined_output = tf.zeros_like(hidden_states_expanded)
            # Process each expert's input
            for i, expert in enumerate(self.routed_experts):
                expert_mask = tf.equal(flat_topk_idx, i)
                if tf.reduce_any(expert_mask):
                    expert_input = tf.boolean_mask(hidden_states_expanded, expert_mask)
                    expert_output = expert(expert_input)
                    update_indices = tf.where(expert_mask)
                    combined_output = tf.tensor_scatter_nd_update(
                        combined_output,
                        update_indices,
                        expert_output
                    )
                    
            # Apply expert weights and reshape output
            topk_weight_flat = tf.reshape(topk_weight, [-1, 1])
            weighted_output = combined_output * topk_weight_flat
            weighted_output = tf.reshape(
                weighted_output,
                [-1, self.num_experts_per_tok, self.hidden_size]
            )
            routed_output = tf.reduce_sum(weighted_output, axis=1)
            routed_output = tf.reshape(routed_output, orig_shape)
        else:
            # Use efficient inference path
            routed_output = self._moe_infer(hidden_states_flat, topk_idx, topk_weight)
            routed_output = tf.reshape(routed_output, orig_shape)
            
        # Add shared expert output if available
        if self.shared_experts is not None:
            shared_output = self.shared_experts(identity)
            final_output = routed_output + shared_output
        else:
            final_output = routed_output
            
        # Apply residual connection
        return final_output + identity
        
    def _moe_infer(self, x, expert_indices, expert_weights):
        """Efficient inference path for MoE layer"""
        batch_size = tf.shape(x)[0]
        expert_cache = tf.zeros_like(x)
        # Clip expert indices to valid range
        expert_indices = tf.clip_by_value(expert_indices, 0, self.n_routed_experts - 1)
        flat_expert_indices = tf.reshape(expert_indices, [-1])
        flat_expert_weights = tf.reshape(expert_weights, [-1, 1])
        total_tokens = tf.shape(x)[0]
        token_indices_base = tf.range(total_tokens, dtype=tf.int64)
        token_indices_base = tf.repeat(token_indices_base, self.num_experts_per_tok, axis=0)
        
        # Process each expert's tokens
        for i in range(self.n_routed_experts):
            expert_mask = tf.equal(flat_expert_indices, i)
            if tf.reduce_any(expert_mask):
                token_indices = tf.boolean_mask(token_indices_base, expert_mask)
                expert_tokens = tf.gather(x, token_indices)
                expert_out = self.routed_experts[i](expert_tokens)
                expert_out_weighted = expert_out * tf.gather(flat_expert_weights, tf.where(expert_mask)[:, 0])
                expert_cache = tf.tensor_scatter_nd_add(
                    expert_cache,
                    tf.expand_dims(token_indices, 1),
                    expert_out_weighted
                )
                
        return expert_cache

class DeepSeekAttention(layers.Layer):
    """Multi-head attention with Rotary Position Embeddings and Grouped Query Attention (GQA) support"""
    def __init__(self, hidden_size, num_attention_heads, num_key_value_heads=None,
                 attention_dropout=0.0, max_position_embeddings=2048,
                 rope_theta=10000.0, rope_scaling=None, attention_bias=False, **kwargs):
        super().__init__(**kwargs)
        self.hidden_size = hidden_size                                         # Input and output dimension
        self.num_attention_heads = num_attention_heads                         # Number of attention heads
        self.num_key_value_heads = num_key_value_heads or num_attention_heads  # Number of key/value heads (GQA)
        self.attention_dropout = attention_dropout                             # Dropout rate for attention weights
        self.max_position_embeddings = max_position_embeddings                 # Maximum sequence length
        self.rope_theta = rope_theta                                           # Base period for RoPE
        self.rope_scaling = rope_scaling                                       # RoPE scaling configuration
        self.attention_bias = attention_bias                                   # Whether to use bias in projections
        self.head_dim = hidden_size // num_attention_heads                     # Dimension per head
        self.num_key_value_groups = num_attention_heads // self.num_key_value_heads  # Groups for GQA
        
    def build(self, input_shape):
        # Initialize query projection layer
        self.q_proj = layers.Dense(
            self.num_attention_heads * self.head_dim,
            use_bias=self.attention_bias,
            kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.001),
            name="q_proj"
        )
        # Initialize key projection layer
        self.k_proj = layers.Dense(
            self.num_key_value_heads * self.head_dim,
            use_bias=self.attention_bias,
            kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.001),
            name="k_proj"
        )
        # Initialize value projection layer
        self.v_proj = layers.Dense(
            self.num_key_value_heads * self.head_dim,
            use_bias=self.attention_bias,
            kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.001),
            name="v_proj"
        )
        # Initialize output projection layer
        self.o_proj = layers.Dense(
            self.hidden_size,
            use_bias=self.attention_bias,
            kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.001),
            name="o_proj"
        )
        
        # Initialize RoPE with specified scaling
        scaling_type = self.rope_scaling.get("type") if self.rope_scaling else None
        scaling_factor = self.rope_scaling.get("factor", 1.0) if self.rope_scaling else 1.0
        self.rotary_emb = RotaryPositionEmbedding(
            self.head_dim,
            self.max_position_embeddings,
            self.rope_theta,
            scaling_type,
            scaling_factor,
            name="rotary_emb"
        )
        self.dropout_layer = layers.Dropout(self.attention_dropout)
        self.rotary_emb.build(input_shape)
        super().build(input_shape)
        
    def call(self, hidden_states, attention_mask=None, position_ids=None, training=False):
        # Get input dimensions
        batch_size, seq_len, _ = tf.unstack(tf.shape(hidden_states))
        
        # Project inputs to query, key, and value
        q = self.q_proj(hidden_states)
        k = self.k_proj(hidden_states)
        v = self.v_proj(hidden_states)
        
        # Reshape for multi-head attention
        q = tf.reshape(q, [batch_size, seq_len, self.num_attention_heads, self.head_dim])
        k = tf.reshape(k, [batch_size, seq_len, self.num_key_value_heads, self.head_dim])
        v = tf.reshape(v, [batch_size, seq_len, self.num_key_value_heads, self.head_dim])
        
        # Apply rotary embeddings to query and key
        q_rotated = self.rotary_emb(q, position_ids=position_ids)
        k_rotated = self.rotary_emb(k, position_ids=position_ids)
        
        # Transpose for attention computation
        q_rotated = tf.transpose(q_rotated, [0, 2, 1, 3])
        k_rotated = tf.transpose(k_rotated, [0, 2, 1, 3])
        v = tf.transpose(v, [0, 2, 1, 3])
        
        # Repeat key and value for GQA if necessary
        if self.num_key_value_groups > 1:
            k_rotated = tf.repeat(k_rotated, self.num_key_value_groups, axis=1)
            v = tf.repeat(v, self.num_key_value_groups, axis=1)
            
        # Compute scaled dot-product attention
        scale = tf.math.rsqrt(tf.cast(self.head_dim, tf.float32))
        scale = tf.cast(scale, q_rotated.dtype)
        attn_weights = tf.matmul(q_rotated, k_rotated, transpose_b=True) * scale
        
        # Apply attention mask if provided
        if attention_mask is not None:
            attention_mask = tf.cast(attention_mask, attn_weights.dtype)
            if len(attention_mask.shape) == 2:
                attention_mask = tf.expand_dims(tf.expand_dims(attention_mask, 1), 1)
            elif len(attention_mask.shape) == 3:
                attention_mask = tf.expand_dims(attention_mask, 1)
            attn_weights += attention_mask
            
        # Clip attention weights for numerical stability
        attn_weights = tf.clip_by_value(attn_weights, -50.0, 50.0)
        attn_weights = tf.cast(attn_weights, tf.float32)
        # Apply softmax with max subtraction for stability
        attn_weights = attn_weights - tf.reduce_max(attn_weights, axis=-1, keepdims=True)
        attn_weights = tf.nn.softmax(attn_weights, axis=-1)
        attn_weights = tf.cast(attn_weights, q_rotated.dtype)
        
        # Apply dropout to attention weights during training
        attn_weights = self.dropout_layer(attn_weights, training=training)
        # Compute attention output
        attn_output = tf.matmul(attn_weights, v)
        attn_output = tf.transpose(attn_output, [0, 2, 1, 3])
        attn_output = tf.reshape(attn_output, [batch_size, seq_len, self.hidden_size])
        
        # Project attention output to hidden size
        return self.o_proj(attn_output)

class DeepSeekDecoderLayer(layers.Layer):
    """Transformer decoder layer combining attention and MoE/MLP with residual connections"""
    def __init__(self, config, layer_idx, **kwargs):
        super().__init__(**kwargs)
        self.config = config
        self.layer_idx = layer_idx
        
        # Initialize self-attention with GQA and RoPE
        self.self_attn = DeepSeekAttention(
            hidden_size=config.hidden_size,
            num_attention_heads=config.num_attention_heads,
            num_key_value_heads=config.num_key_value_heads,
            attention_dropout=config.attention_dropout,
            max_position_embeddings=config.max_position_embeddings,
            rope_theta=config.rope_theta,
            rope_scaling=config.rope_scaling,
            attention_bias=config.attention_bias,
            name="self_attn"
        )
        
        # Determine whether to use MoE or dense MLP based on layer index and config
        self.use_moe = (
            config.n_routed_experts is not None and
            layer_idx >= config.first_k_dense_replace and
            layer_idx % config.moe_layer_freq == 0
        )
        
        if self.use_moe:
            # Initialize MoE layer for routed experts
            self.mlp = DeepSeekMoE(
                hidden_size=config.hidden_size,
                n_routed_experts=config.n_routed_experts,
                n_shared_experts=config.n_shared_experts,
                num_experts_per_tok=config.num_experts_per_tok,
                moe_intermediate_size=config.moe_intermediate_size,
                name="moe"
            )
        else:
            # Initialize dense MLP layer
            self.mlp = DeepSeekMLP(
                hidden_size=config.hidden_size,
                intermediate_size=config.intermediate_size,
                hidden_act=config.hidden_act,
                name="mlp"
            )
            
        # Initialize input and post-attention normalization layers
        self.input_layernorm = DeepSeekRMSNorm(
            config.hidden_size, eps=config.rms_norm_eps, name="input_layernorm"
        )
        self.post_attention_layernorm = DeepSeekRMSNorm(
            config.hidden_size, eps=config.rms_norm_eps, name="post_attention_layernorm"
        )
        
    def call(self, hidden_states, attention_mask=None, position_ids=None, training=False):
        # Apply input normalization and self-attention with residual connection
        residual = hidden_states
        hidden_states = self.input_layernorm(hidden_states)
        attn_output = self.self_attn(
            hidden_states,
            attention_mask=attention_mask,
            position_ids=position_ids,
            training=training
        )
        hidden_states = residual + attn_output
        
        # Apply post-attention normalization and MLP/MoE with residual connection
        residual = hidden_states
        hidden_states = self.post_attention_layernorm(hidden_states)
        mlp_output = self.mlp(hidden_states, training=training)
        hidden_states = residual + mlp_output
        
        return hidden_states

# MODEL CONFIGURATION 

class DeepSeekConfig:
    """
    This is the configuration class to store the configuration of a [`DeepseekMoEModel`]. It is used to instantiate an DeepSeek
    model according to the specified arguments, defining the model architecture. Instantiating a configuration with the
    defaults below will yield a configuration for a small, stable, and computationally efficient MoE chatbot model.

    Args:
        vocab_size (`int`, *optional*, defaults to 12000):
            Vocabulary size of the Deep model. Defines the number of different tokens that can be represented by the
            `inputs_ids` passed when calling [`DeepseekModel`]. (Optimized for chatbot use case).
            
        hidden_size (`int`, *optional*, defaults to 256):
            Dimension of the hidden representations. (Optimized hidden size for stability and performance).
            
        intermediate_size (`int`, *optional*, defaults to 512):
            Dimension of the MLP representations. (Balanced intermediate size for MLP layers).
            
        moe_intermediate_size (`int`, *optional*, defaults to 256):
            Dimension of the MoE representations. (MoE-specific intermediate dimension).
            
        num_hidden_layers (`int`, *optional*, defaults to 4):
            Number of hidden layers in the Transformer decoder. (Optimal layer count for chatbot tasks).
            
        num_attention_heads (`int`, *optional*, defaults to 4):
            Number of attention heads for each attention layer in the Transformer decoder. (Balanced attention heads for computation).
        
        n_shared_experts (`int`, *optional*, defaults to 1):
            Number of shared experts, None means dense model. (Set to 1, a single shared expert for parameter efficiency).
        
        n_routed_experts (`int`, *optional*, defaults to 4):
            Number of routed experts, None means dense model. (Set to 4 for specialization).
            
        num_experts_per_tok (`int`, *optional*, defaults to 2):
            Number of selected experts, None means dense model. (Set to 2 experts per token for balanced load).
            
        moe_layer_freq (`int`, *optional*, defaults to 2):
            The frequency of the MoE layer: one expert layer for every `moe_layer_freq - 1` dense layers.
            (MoE every other layer for computational balance).
            
        first_k_dense_replace (`int`, *optional*, defaults to 1):
            - Number of dense layers in shallow layers(embed->dense->dense->...->dense->moe->moe...->lm_head).
            - k dense layers (First layer dense for stability).
        norm_topk_prob (`bool`, *optional*, defaults to False):
            Whether to normalize the weights of the routed experts. (No normalization for expert weights).
            
        scoring_func (`str`, *optional*, defaults to 'softmax'):
            Method of computing expert weights. (Standard softmax for expert selection).
            
        aux_loss_alpha (`float`, *optional*, defaults to 0.001):
            Auxiliary loss weight coefficient. (Default auxiliary loss weight).
            
        seq_aux = (`bool`, *optional*, defaults to True):
            Whether to compute the auxiliary loss for each individual sample. (Sequence-level auxiliary loss).
            
        num_key_value_heads (`int`, *optional*, defaults to 2):
            This is the number of key_value heads that should be used to implement Grouped Query Attention. If
            `num_key_value_heads=num_attention_heads`, the model will use Multi Head Attention (MHA), if
            `num_key_value_heads=1` the model will use Multi Query Attention (MQA) otherwise GQA is used.
            (Set to 2 for GQA and memory efficiency).
            
        hidden_act (`str` or `function`, *optional*, defaults to "swish"):
            The non-linear activation function (function or string) in the decoder. (Swish activation for better gradients).
        
        max_position_embeddings (`int`, *optional*, defaults to 256):
            The maximum sequence length that this model might ever be used with. (Sufficient context length for chatbot).
        
        initializer_range (`float`, *optional*, defaults to 0.001):
            The standard deviation of the truncated_normal_initializer for initializing all weight matrices. (Reduced initializer range for stability).
        
        rms_norm_eps (`float`, *optional*, defaults to 1e-6):
            The epsilon used by the rms normalization layers. (Standard RMS norm epsilon).
        
        use_cache (`bool`, *optional*, defaults to True):
            Whether or not the model should return the last key/values attentions (not used by all models). Only
            relevant if `config.is_decoder=True`. (Enable caching for inference).
        
        pad_token_id (`int`, *optional*):
            Padding token id.
        bos_token_id (`int`, *optional*, defaults to 1):
            Beginning of stream token id.
        eos_token_id (`int`, *optional*, defaults to 2):
            End of stream token id.
        
        pretraining_tp (`int`, *optional*, defaults to 1):
            Experimental feature. Tensor parallelism rank used during pretraining. Please refer to [this
            document](https://huggingface.co/docs/transformers/parallelism) to understand more about it. This value is
            necessary to ensure exact reproducibility of the pretraining results. Please refer to [this
            issue](https://github.com/pytorch/pytorch/issues/76232).
        
        tie_word_embeddings (`bool`, *optional*, defaults to False):
            Whether to tie weight embeddings
        
        rope_theta (`float`, *optional*, defaults to 10000.0):
            The base period of the RoPE embeddings. (Standard RoPE base period).
        
        rope_scaling (`Dict`, *optional*, defaults to {"type": "linear", "factor": 2.0}):
            Dictionary containing the scaling configuration for the RoPE embeddings. Currently supports two scaling
            strategies: linear and dynamic. Their scaling factor must be a float greater than 1. The expected format is
            `{"type": strategy name, "factor": scaling factor}`. When using this flag, don't update
            `max_position_embeddings` to the expected new maximum. (Linear RoPE scaling enabled).
        
        attention_bias (`bool`, defaults to False, *optional*, defaults to False):
            Whether to use a bias in the query, key, value and output projection layers during self-attention. (No attention bias for efficiency).
        
        attention_dropout (`float`, *optional*, defaults to 0.1):
            The dropout ratio for the attention probabilities. (Increased dropout for regularization).
    """

    def __init__(
        self,
        vocab_size=vocab_size,      # Default vocabulary size optimized for chatbot
        hidden_size=256,            # Optimized hidden size for stability and performance
        intermediate_size=512,      # Balanced intermediate size for MLP layers
        moe_intermediate_size=256,  # MoE-specific intermediate dimension
        num_hidden_layers=4,        # Optimal layer count for chatbot tasks
        num_attention_heads=4,      # Balanced attention heads for computation
        num_key_value_heads=2,      # GQA for memory efficiency
        n_shared_experts=3,         # Three shared experts for parameter efficiency (REVISED)
        n_routed_experts=6,         # Six routed experts for specialization     (REVISED)
        num_experts_per_tok=2,      # Two experts per token for balanced load
        moe_layer_freq=1,           # MoE every layer for computational balance (REVISED)
        first_k_dense_replace=1,    # First layer dense for stability
        norm_topk_prob=True,        # Normalization for expert weights          (REVISED)
        scoring_func='softmax',     # Standard softmax for expert selection
        aux_loss_alpha=0.0022997166087842954,  # Auxiliary loss weight          (REVISED)
        seq_aux=False,              # Token-level auxiliary loss                (REVISED)
        hidden_act="swish",         # Swish activation for better gradients
        max_position_embeddings=256,  # Sufficient context length for chatbot
        initializer_range=0.001,    # Reduced initializer range for stability
        rms_norm_eps=1e-6,          # Standard RMS norm epsilon
        use_cache=True,             # Enable caching for inference
        rope_theta=10000.0,         # Standard RoPE base period
        rope_scaling={"type": "linear", "factor": 2.0},  # Linear RoPE scaling
        attention_bias=False,       # No attention bias for efficiency
        attention_dropout=0.1       # Increased dropout for regularization
    ):
        # Initialize model architecture parameters with optimized values
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.intermediate_size = intermediate_size
        self.moe_intermediate_size = moe_intermediate_size
        self.num_hidden_layers = num_hidden_layers
        self.num_attention_heads = num_attention_heads
        self.num_key_value_heads = num_key_value_heads
        self.n_shared_experts = n_shared_experts
        self.n_routed_experts = n_routed_experts
        self.num_experts_per_tok = num_experts_per_tok
        self.moe_layer_freq = moe_layer_freq
        self.first_k_dense_replace = first_k_dense_replace
        self.norm_topk_prob = norm_topk_prob
        self.scoring_func = scoring_func
        self.aux_loss_alpha = aux_loss_alpha
        self.seq_aux = seq_aux
        self.hidden_act = hidden_act
        self.max_position_embeddings = max_position_embeddings
        self.initializer_range = initializer_range
        self.rms_norm_eps = rms_norm_eps
        self.use_cache = use_cache
        self.rope_theta = rope_theta
        self.rope_scaling = rope_scaling
        self.attention_bias = attention_bias
        self.attention_dropout = attention_dropout

class DeepSeekModel(tf.keras.Model):
    """Core DeepSeek transformer model for chatbot, provide raw hidden states"""
    def __init__(self, config, **kwargs):
        super().__init__(**kwargs)
        self.config = config
        self.padding_idx = 0
        
        # Initialize embedding layer for token inputs
        self.embed_tokens = layers.Embedding(
            config.vocab_size,
            config.hidden_size,
            embeddings_initializer=tf.keras.initializers.RandomNormal(
                stddev=config.initializer_range
            ),
            name="embed_tokens"
        )
        
        # Initialize decoder layers
        self.decoder_layers = [
            DeepSeekDecoderLayer(config, layer_idx=i, name=f"layer_{i}")
            for i in range(config.num_hidden_layers)
        ]
        
        # Initialize final normalization layer
        self.norm = DeepSeekRMSNorm(
            config.hidden_size, eps=config.rms_norm_eps, name="norm"
        )
        
        # Initialize expert assignment counter for MoE layers
        self.expert_assignment_counts = tf.Variable(
            tf.zeros([config.n_routed_experts], dtype=tf.int32),
            trainable=False,
            name="expert_assignment_counts"
        ) if config.n_routed_experts else None
        
    def call(self, input_ids, attention_mask=None, position_ids=None, training=False):
        # Embed input tokens
        hidden_states = self.embed_tokens(input_ids)
        
        # Create causal mask if not provided
        if attention_mask is None:
            seq_len = tf.shape(input_ids)[1]
            attention_mask = self._create_causal_mask(seq_len)
            
        # Prepare attention mask for causal attention
        attention_mask = tf.cast(attention_mask, hidden_states.dtype)
        if len(attention_mask.shape) == 2:
            attention_mask = tf.expand_dims(tf.expand_dims(attention_mask, 1), 1)
        elif len(attention_mask.shape) == 3:
            attention_mask = tf.expand_dims(attention_mask, 1)
        attention_mask = (tf.cast(1.0, hidden_states.dtype) - attention_mask) * tf.cast(-1e4, hidden_states.dtype)
        
        # Process through decoder layers and aggregate expert counts
        for layer in self.decoder_layers:
            hidden_states = layer(
                hidden_states,
                attention_mask=attention_mask,
                position_ids=position_ids,
                training=training
            )
            # Update expert assignment counts for MoE layers during training
            if training and hasattr(layer, 'mlp') and isinstance(layer.mlp, DeepSeekMoE):
                expert_counts = layer.mlp.gate.expert_counts
                if self.expert_assignment_counts is not None:
                    self.expert_assignment_counts.assign_add(expert_counts)
            
        # Apply final normalization
        return self.norm(hidden_states)
        
    def _create_causal_mask(self, seq_len):
        """Create causal attention mask to prevent attending to future tokens"""
        mask = tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
        return tf.cast(mask, tf.float16)

class DeepSeekForCausalLM(tf.keras.Model):
    """Causal language model for chatbot text generation"""
    def __init__(self, config, **kwargs):
        super().__init__(**kwargs)
        self.config = config
        self.transformer = DeepSeekModel(config, name="transformer")
        # Initialize language model head
        self.lm_head = layers.Dense(
            config.vocab_size,
            kernel_initializer=tf.keras.initializers.RandomNormal(
                stddev=config.initializer_range
            ),
            use_bias=False,
            name="lm_head"
        )
        
    def call(self, inputs, attention_mask=None, position_ids=None, training=False):
        # Handle dictionary or tensor inputs
        if isinstance(inputs, dict):
            input_ids = inputs.get('input_ids')
            attention_mask = inputs.get('attention_mask', attention_mask)
        else:
            input_ids = inputs
        
        # Process through transformer
        hidden_states = self.transformer(
            input_ids,
            attention_mask=attention_mask,
            position_ids=position_ids,
            training=training
        )
        # Generate logits for token prediction
        return self.lm_head(hidden_states)

# METRICS AND CALLBACKS

class ResourceMonitor(tf.keras.callbacks.Callback):
    """Callback to monitor system resources (GPU, CPU, memory) during training"""
    def __init__(self, batch_size=2):
        super().__init__()
        self.batch_size = batch_size
        self.resource_metrics = []  # Store resource usage metrics
        self.start_time = time.time()
        self.total_tokens_processed = 0
        
    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()
        self.epoch_tokens = 0
        
    def on_batch_end(self, batch, logs=None):
        # Update token count for throughput calculation
        self.total_tokens_processed += self.batch_size * max_seq_len
        self.epoch_tokens += self.batch_size * max_seq_len
        
    def on_epoch_end(self, epoch, logs=None):
        try:
            # Collect GPU and system resource usage
            gpus = GPUtil.getGPUs()
            gpu_memory = sum([gpu.memoryUsed for gpu in gpus]) / len(gpus) if gpus else 0
            gpu_usage = sum([gpu.load * 100 for gpu in gpus]) / len(gpus) if gpus else 0
            cpu_usage = psutil.cpu_percent()
            memory_info = psutil.virtual_memory()
            system_memory = memory_info.used / (1024 ** 3)
            epoch_time = time.time() - self.epoch_start_time
            tokens_per_second = self.epoch_tokens / epoch_time if epoch_time > 0 else 0
            
            # Store resource metrics
            resource_stats = {
                'epoch': epoch + 1,
                'avg_gpu_memory_gb': float(gpu_memory / 1024),
                'avg_system_memory_gb': float(system_memory),
                'avg_gpu_usage_percent': float(gpu_usage),
                'avg_cpu_usage_percent': float(cpu_usage),
                'tokens_per_second': float(tokens_per_second),
                'throughput_tps': float(tokens_per_second)
            }
            
            self.resource_metrics.append(resource_stats)
            
            # Print resource usage summary
            print(f"Epoch {epoch+1} Resources: GPU Mem: {gpu_memory/1024:.1f}GB, "
                  f"System Mem: {system_memory:.1f}GB, GPU: {gpu_usage:.1f}%, "
                  f"CPU: {cpu_usage:.1f}%, TPS: {tokens_per_second:.0f}")
                  
        except Exception as e:
            print(f"Resource monitoring error: {e}")

class DeepSeekMetrics(tf.keras.callbacks.Callback):
    """Callback to track specific metrics, including MaxVIO for expert balancing"""
    def __init__(self, validation_data, tokenizer):
        super().__init__()
        self.validation_data = validation_data
        self.tokenizer = tokenizer
        self.metrics_history = []
        self.start_time = time.time()
        self.max_vio_values = []
        
    def on_epoch_begin(self, epoch, logs=None):
        # Reset expert assignment counts for accurate per-epoch metrics
        if hasattr(self.model.transformer, 'expert_assignment_counts') and self.model.transformer.expert_assignment_counts is not None:
            self.model.transformer.expert_assignment_counts.assign(tf.zeros([self.model.config.n_routed_experts], dtype=tf.int32))
        for layer in self.model.transformer.decoder_layers:
            if hasattr(layer, 'mlp') and isinstance(layer.mlp, DeepSeekMoE):
                layer.mlp.gate.expert_counts.assign(tf.zeros([layer.mlp.n_routed_experts], dtype=tf.int32))
        
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        
        # Estimate FLOPs for computational efficiency tracking
        total_params = self.model.count_params()
        flops_per_token = total_params * 2
        total_flops = flops_per_token * batch_size * max_seq_len
        total_gflops = total_flops / 1e9
        
        logs['gflops'] = float(total_gflops)
        
        # Calculate MaxVIO for expert load balancing
        if self.validation_data:
            try:
                max_vio = self._calculate_max_vio()
                logs['max_vio'] = float(max_vio)
                self.max_vio_values.append(max_vio)
                
            except Exception as e:
                print(f"Metrics calculation error: {e}")
                logs['max_vio'] = 0.0
                
        self.metrics_history.append(logs.copy())
        
        # Print simplified metrics summary
        print(f"Epoch {epoch+1}: Loss={logs.get('loss', 0):.4f}, "
              f"Val_Loss={logs.get('val_loss', 0):.4f}, "
              f"GFLOPs={total_gflops:.2f}, "
              f"MaxVIO={logs.get('max_vio', 0):.4f}")
              
    def _calculate_max_vio(self):
        """Calculate Maximum Violation (MaxVIO) for expert load balancing"""
        total_counts = np.zeros(self.model.config.n_routed_experts, dtype=np.float32)
        moe_layers = 0
        # Aggregate expert counts across MoE layers
        for layer in self.model.transformer.decoder_layers:
            if hasattr(layer, 'mlp') and isinstance(layer.mlp, DeepSeekMoE):
                counts = layer.mlp.gate.expert_counts.numpy()
                total_counts += counts
                moe_layers += 1
        
        # Compute MaxVIO based on aggregated counts
        if moe_layers > 0 and np.sum(total_counts) > 0:
            normalized_counts = total_counts / (np.sum(total_counts) + 1e-10)
            expected_uniform = 1.0 / len(total_counts)
            max_vio = np.max(np.abs(normalized_counts - expected_uniform)) / expected_uniform
            return max_vio
        return 0.0

class ComprehensiveModelCheckpoint(tf.keras.callbacks.Callback):
    """Callback for saving model checkpoints based on monitored metric"""
    def __init__(self, filepath, monitor='val_loss', save_best_only=True, mode='min'):
        super().__init__()
        self.filepath = filepath
        self.monitor = monitor
        self.save_best_only = save_best_only
        self.mode = mode
        self.best_value = float('inf') if mode == 'min' else -float('inf')
        
    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            return
            
        current_value = logs.get(self.monitor)
        if current_value is None:
            return
            
        # Determine if current model should be saved
        if self.mode == 'min':
            should_save = current_value < self.best_value
        else:
            should_save = current_value > self.best_value
            
        if should_save or not self.save_best_only:
            self.best_value = current_value
            self.model.save_weights(self.filepath)
            print(f"Model checkpoint saved with {self.monitor}: {current_value:.4f}")


# Function to calculate perplexity on validation set
def calculate_perplexity(model, dataset, tokenizer, max_batches=None):          # Sample batch for calculate perplexity
    """Calculate perplexity on validation dataset to evaluate model performance"""
    total_loss = 0.0
    total_tokens = 0
    PAD_TOKEN_ID = tokenizer.token_to_id(PAD_TOKEN)
    
    for i, batch in enumerate(dataset.take(max_batches)):
        inputs, labels = batch
        logits = model(inputs, training=False)
        # Clip logits for numerical stability
        logits = tf.clip_by_value(logits, -50.0, 50.0)
        log_probs = tf.nn.log_softmax(logits, axis=-1)
        # Compute per-token loss
        per_token_loss = -tf.gather(log_probs, labels, batch_dims=2)
        if tf.reduce_any(tf.math.is_nan(per_token_loss)):
            continue
        # Mask padding tokens
        mask = tf.cast(labels != PAD_TOKEN_ID, per_token_loss.dtype)
        batch_loss = tf.reduce_sum(per_token_loss * mask)
        batch_tokens = tf.reduce_sum(mask)
        if tf.math.is_nan(batch_loss) or batch_tokens == 0:
            continue
        total_loss += batch_loss.numpy()
        total_tokens += batch_tokens.numpy()
        
    # Compute perplexity as exponential of average loss
    if total_tokens > 0:
        avg_loss = total_loss / total_tokens
        return math.exp(min(avg_loss, 20.0))
    return float('inf')


# Define custom objects for model loading
custom_objects = {
    'DeepSeekRMSNorm': DeepSeekRMSNorm,
    'RotaryPositionEmbedding': RotaryPositionEmbedding,
    'DeepSeekMLP': DeepSeekMLP,
    'MoEGate': MoEGate,
    'DeepSeekMoE': DeepSeekMoE,
    'DeepSeekAttention': DeepSeekAttention,
    'DeepSeekDecoderLayer': DeepSeekDecoderLayer,
    'DeepSeekModel': DeepSeekModel,
    'DeepSeekForCausalLM': DeepSeekForCausalLM
}

Physical devices cannot be modified after being initialized
TensorFlow version: 2.15.0
Mixed precision policy: mixed_float16
GPU available: True


In [32]:
# Visualize the model
# Create model with specified vocabulary size using direct config
vocab_size = tokenizer.get_vocab_size()
# Use the optimized config directly with the same hyperparameters from create_deepseek_chatbot_model
config = DeepSeekConfig(vocab_size=vocab_size)
model = DeepSeekForCausalLM(config)

# Build model
if tf_train_dataset:
    sample_batch = next(iter(tf_train_dataset.take(1)))
    model(sample_batch[0])

# Configure optimizer with more conservative settings
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4,  # Reduced learning rate
    decay_steps=1000,
    decay_rate=0.96
)

optimizer = tf.keras.optimizers.AdamW(
    learning_rate=lr_schedule,
    beta_1=0.9,
    beta_2=0.95,
    weight_decay=0.1,
    clipnorm=1.0
)

model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, 
        ignore_class=PAD_TOKEN_ID,  # Use correct PAD token ID
        reduction=tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE
    ),
    metrics=['accuracy']
)

print("\nModel Summary")
model.summary()


Model Summary
Model: "deep_seek_for_causal_lm_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (DeepSeekModel  multiple                  6918168   
 )                                                               
                                                                 
 lm_head (Dense)             multiple                  292096    
                                                                 
Total params: 7210264 (27.50 MB)
Trainable params: 7079168 (27.00 MB)
Non-trainable params: 131096 (512.09 KB)
_________________________________________________________________


## Defined Evaluation Metrics

In [33]:
# Special tokens
PAD_TOKEN = "<pad>"
BOS_TOKEN = "<bos>"
EOS_TOKEN = "<eos>"
SEP_TOKEN = "<sep>"

def calculate_perplexity(model, dataset, tokenizer, max_batches=None):
    """
    Calculate perplexity on validation dataset to evaluate model performance,
    using the manual Negative Log-Likelihood method as specified in the model training.
    """
    total_loss = 0.0  # Initialize total loss
    total_tokens = 0  # Initialize total token count
    # Load special token IDs with fallback to 0 if not found
    special_token_ids = [
        tokenizer.token_to_id(token) or 0
        for token in [PAD_TOKEN, BOS_TOKEN, EOS_TOKEN, SEP_TOKEN]
    ]
    
    # Use entire dataset if max_batches is None for calculate perplexity
    dataset_iter = dataset.take(max_batches) if max_batches else dataset

    for batch in dataset_iter:
        # Handle different batch formats
        # Access dictionary from tuple
        batch_dict = batch[0]  
        inputs = {
            'input_ids': batch_dict['input_ids'],
            'attention_mask': batch_dict.get('attention_mask', tf.ones_like(batch_dict['input_ids'], dtype=tf.int32))
        }
        labels = batch_dict.get('labels', batch_dict['input_ids'])  # Use input_ids as fallback

        # Get model predictions (logits)
        # model(inputs, training=False) returns (batch_size, sequence_length, vocab_size) logits
        logits = model(inputs, training=False)

        # Shift logits and labels for next token prediction (standard LM objective)
        logits = logits[:, :-1, :]
        labels = labels[:, 1:]
        
        # Clip logits for numerical stability (from prototype)
        logits = tf.clip_by_value(logits, -50.0, 50.0)  # Use training's clipping range
        
        # Calculate log probabilities
        log_probs = tf.nn.log_softmax(logits, axis=-1)

        # Compute per-token loss (Negative Log Likelihood)
        # tf.gather extracts the log_prob corresponding to the true label for each token
        per_token_loss = -tf.gather(log_probs, labels, batch_dims=2)
        
        # Mask special tokens (<pad>, <bos>, <eos>, <sep>)
        mask = tf.cast(tf.reduce_all([labels != token_id for token_id in special_token_ids], axis=0), per_token_loss.dtype)
        
        # Sum masked losses and count tokens
        batch_loss = tf.reduce_sum(per_token_loss * mask)
        batch_tokens = tf.reduce_sum(mask)
        
        # Skip invalid batches (NaN loss or zero tokens)
        valid_batch = tf.logical_and(tf.logical_not(tf.math.is_nan(batch_loss)), batch_tokens > 0)
        total_loss += tf.where(valid_batch, batch_loss.numpy(), 0.0)
        total_tokens += tf.where(valid_batch, batch_tokens.numpy(), 0.0)
        
    # Compute perplexity as exponential of average loss
    return math.exp(min(total_loss / total_tokens, 20.0)) if total_tokens > 0 else float('inf')  # Use training's capping

def calculate_rouge_scores(generated_texts, reference_texts):
    """Calculate ROUGE scores using the rouge_score library"""
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge1_scores = []
    rouge2_scores = []
    rougeL_scores = []
    
    for gen, ref in zip(generated_texts, reference_texts):
        # Skip invalid pairs
        scores = scorer.score(ref, gen) if isinstance(gen, str) and isinstance(ref, str) else {'rouge1': {'fmeasure': 0}, 'rouge2': {'fmeasure': 0}, 'rougeL': {'fmeasure': 0}}
        rouge1_scores.append(scores['rouge1'].fmeasure)
        rouge2_scores.append(scores['rouge2'].fmeasure)
        rougeL_scores.append(scores['rougeL'].fmeasure)
    
    return {
        'rouge1': np.mean(rouge1_scores),
        'rouge2': np.mean(rouge2_scores),
        'rougeL': np.mean(rougeL_scores)
    }

def calculate_bleu_score(generated_texts, reference_texts):
    """Calculate BLEU score using NLTK"""
    gen_tokens = []
    ref_tokens = []
    for gen, ref in zip(generated_texts, reference_texts):
        gen_tok = nltk.word_tokenize(gen.lower()) if isinstance(gen, str) else []
        ref_tok = nltk.word_tokenize(ref.lower()) if isinstance(ref, str) else []
        gen_tokens.append(gen_tok)
        ref_tokens.append([ref_tok] if ref_tok else [[]])
    
    # Skip if no valid tokens
    smoothing = SmoothingFunction().method1
    return corpus_bleu(ref_tokens, gen_tokens, smoothing_function=smoothing) if gen_tokens and ref_tokens else 0.0

def calculate_meteor_score(generated_texts, reference_texts):
    """Calculate METEOR score using NLTK"""
    meteor_scores = []
    for gen, ref in zip(generated_texts, reference_texts):
        gen_tokens = nltk.word_tokenize(gen.lower()) if isinstance(gen, str) else []
        ref_tokens = nltk.word_tokenize(ref.lower()) if isinstance(ref, str) else []
        score = meteor_score([ref_tokens], gen_tokens) if gen_tokens and ref_tokens else 0.0
        meteor_scores.append(score)
    
    return np.mean(meteor_scores)

def nucleus_sampling_decode(model, input_ids, tokenizer, top_p=0.9, max_length=50):
    """Nucleus sampling (top-p sampling) decoding strategy for text generation"""
    generated = input_ids
    eos_token_id = tokenizer.token_to_id(EOS_TOKEN)
    max_input_length = 70
    
    # Truncate input if too long
    generated = generated[:, :max_input_length] if tf.shape(generated)[1] > max_input_length else generated
        
    for _ in range(max_length):
        # Use last max_input_length tokens
        model_input_seq = generated[:, -max_input_length:] if tf.shape(generated)[1] > max_input_length else generated
            
        attention_mask = tf.ones_like(model_input_seq, dtype=tf.int32)
        inputs = {'input_ids': model_input_seq, 'attention_mask': attention_mask}
        logits = model(inputs, training=False)
        next_token_logits = logits[:, -1, :]
        
        sorted_logits = tf.sort(next_token_logits, direction='DESCENDING')
        sorted_indices = tf.argsort(next_token_logits, direction='DESCENDING')
        cumulative_probs = tf.cumsum(tf.nn.softmax(tf.cast(sorted_logits, tf.float32), axis=-1), axis=-1)
        sorted_indices_to_remove = cumulative_probs > top_p
        sorted_indices_to_remove = tf.concat([
            tf.zeros_like(sorted_indices_to_remove[:, :1]),
            sorted_indices_to_remove[:, :-1]
        ], axis=1)
        indices_to_remove = tf.gather(sorted_indices_to_remove, tf.argsort(sorted_indices), batch_dims=1)
        next_token_logits = tf.where(
            indices_to_remove,
            tf.constant(-1e9, dtype=next_token_logits.dtype),
            next_token_logits
        )
        
        next_token = tf.random.categorical(next_token_logits, num_samples=1, dtype=tf.int32)
        next_token = tf.expand_dims(tf.squeeze(next_token, axis=0), axis=0)
        
        generated = tf.concat([generated, next_token], axis=1)
        
        # Stop if EOS token is generated
        if next_token.numpy()[0, 0] == eos_token_id:
            break
            
    return generated

def generate_text_batch(model, input_batch, tokenizer, strategy='nucleus', **kwargs):
    """Generate text for a batch of inputs using specified strategy"""
    generated_texts = [
        tokenizer.decode(nucleus_sampling_decode(model, tf.expand_dims(input_batch[i], 0), tokenizer, **kwargs)[0].numpy(), skip_special_tokens=True)
        for i in range(input_batch.shape[0])
    ]
    
    return generated_texts

def get_reference_texts(dataset, tokenizer, num_samples=None):
    """Extract reference texts from dataset for evaluation"""
    reference_texts = []
    
    # Use entire dataset
    dataset_iter = dataset.take(num_samples) if num_samples else dataset
    
    for batch in dataset_iter:
        input_ids = batch[0]['input_ids']
        reference_texts.extend([
            tokenizer.decode(input_ids[i].numpy(), skip_special_tokens=True).strip()
            for i in range(input_ids.shape[0])
        ])
        if num_samples and len(reference_texts) >= num_samples:
            return reference_texts[:num_samples]
        
    return reference_texts

def calculate_gflops(model, batch_size=2, seq_length=max_seq_len):
    """Calculate estimated GFLOPS for the model"""
    total_params = model.count_params()
    flops_per_token = total_params * 2
    total_flops = flops_per_token * batch_size * seq_length
    
    return total_flops / 1e9

# AUXILIARY-LOSS-FREE LOAD BALANCING METRICS
def calculate_aux_free_max_vio(model, dataset, num_batches=None):
    """
    Calculate MaxVIO (Maximum Violation) according to auxiliary-loss-free load balancing strategy.
    Based on "AUXILIARY-LOSS-FREE LOAD BALANCING STRATEGY FOR MIXTURE-OF-EXPERTS" research paper.
    This implementation focuses on measuring the imbalance in expert utilization without auxiliary loss.
    """
    # Ensure the model has the expected structure before proceeding
    model.transformer.decoder_layers
    
    # Collect expert routing statistics across all MoE layers
    expert_utilization = {}
    total_tokens = 0
    
    # Use entire dataset if num_batches is None
    dataset_iter = dataset.take(num_batches) if num_batches else dataset
    
    for batch in dataset_iter:
        # Get input data based on batch type
        input_ids = batch[0]['input_ids']
        
        # Create attention mask
        attention_mask = tf.ones_like(input_ids, dtype=tf.int32)
        
        # Call the model with the expected dictionary format
        inputs = {'input_ids': input_ids, 'attention_mask': attention_mask}
        
        # Process batch through model to collect expert statistics
        _ = model(inputs, training=False)
        
        # Collect expert counts from all MoE layers
        for layer_idx, layer in enumerate(model.transformer.decoder_layers):
            expert_counts = (
                layer.mlp.gate.expert_counts.numpy()
                if hasattr(layer, 'mlp') and isinstance(layer.mlp, DeepSeekMoE)
                else np.zeros(0)
            )
            if expert_counts.size:
                layer_key = f'layer_{layer_idx}'
                expert_utilization[layer_key] = expert_utilization.get(layer_key, np.zeros_like(expert_counts)) + expert_counts
                total_tokens += np.sum(expert_counts)
    
    # Calculate MaxVIO across all layers and experts
    max_vio_values = [
        np.max(np.abs(counts / np.sum(counts) - 1.0 / len(counts))) / (1.0 / len(counts))
        for counts in expert_utilization.values() if np.sum(counts) > 0
    ]
    
    # Return average MaxVIO across all layers
    return np.mean(max_vio_values) if max_vio_values else 0.0

## Model Evaluation Experiment Loop

In [34]:
# MAIN EVALUATION CODE

# Initialize results table with comprehensive metrics
results_table = {
    'Computational Resource Usage': {},
    'Average CPU Usage (percent)': {},
    'Average GPU Usage (percent)': {},
    'Average Memory (GB)': {},
    'Average GPU Memory (GB)': {},
    'Average FLOPS Estimate (GFLOPS)': {},
    'Average Validation Perplexity': {},
    'Average MaxVIO (Global)': {},
    'Average ROUGE-1': {},
    'Average ROUGE-2': {},
    'Average ROUGE-L': {},
    'Average BLEU': {},
    'Average METEOR': {},
    'Average TTFT (s)': {},
    'Average TPOT (s/token)': {},
    'Average TPS': {}
}

# Load tokenizer
tokenizer = Tokenizer.from_file("deepseek_tokenizer.json")
print("Tokenizer loaded successfully")

# Define text generation strategies to evaluate (NUCLEUS SAMPLING)
generation_strategies = [
    ('nucleus', {'top_p': 0.9, 'max_length': 50})
]

# Initialize global MaxVIO list for all iterations 
global_max_vio_values = []

# Number of evaluation iteration
num_iteration = 5

# Evaluate model for 2 iterations
for iteration in range(num_iteration):
    print(f"\n{'-'*60}")
    print(f"Starting Evaluation Iteration {iteration + 1}")
    print(f"{'-'*60}", flush=True)  # Force flush for immediate output

    # Clear memory before each iteration to ensure consistent evaluation
    tf.keras.backend.clear_session()
    gc.collect()
    
    model = tf.keras.models.load_model(
        'best_deepseek_chatbot_model',
        custom_objects=custom_objects,
        compile=False
    )
    
    # Set config for loaded model to fix AttributeError
    model.config = DeepSeekConfig(vocab_size=tokenizer.get_vocab_size())
    print(f"Iteration {iteration + 1}: Model loaded successfully.", flush=True)
    
    # Build model
    sample_batch = next(iter(tf_test_dataset.take(1)))
    model(sample_batch[0])

    # Initialize metrics for this iteration
    start_time = time.time()
    all_generated_texts = []
    all_reference_texts = []
    resource_metrics = []
    inference_times = []
    strategy_results = {}

    # Get reference texts from entire test dataset
    print("Loaded reference texts from entire test dataset", flush=True)
    reference_texts = get_reference_texts(tf_test_dataset, tokenizer, num_samples=None)
    print(f"Loaded {len(reference_texts)} reference texts from test dataset", flush=True)

    # Calculate auxiliary-loss-free MaxVIO metrics on entire test dataset
    print("Calculating MaxVIO (Global)...", flush=True)
    aux_free_max_vio = calculate_aux_free_max_vio(model, tf_test_dataset, num_batches=None)
    #imbalance_metrics = calculate_expert_load_imbalance(model, tf_test_dataset, num_batches=None)

    # Store MaxVIO values for global statistics
    global_max_vio_values.append(aux_free_max_vio)

    # Evaluate each generation strategy on entire test dataset with progress bar
    for strategy_name, strategy_kwargs in tqdm(generation_strategies, desc=f"Iteration {iteration + 1} Strategies", file=sys.stdout):
        print(f"\nEvaluating {strategy_name} generation strategy on entire test dataset...", flush=True)
        strategy_generated_texts = []
        strategy_inference_times = []

        batch_count = 0
        total_batches = sum(1 for _ in tf_test_dataset)
        print(f"Processing {total_batches} batches...", flush=True)

        # Wrap batch processing with tqdm for progress visualization, ensure output to stdout
        for batch in tqdm(tf_test_dataset, total=total_batches, desc=f"{strategy_name} batches", file=sys.stdout):
            # Extract input_ids from batch
            input_batch = batch[0]['input_ids']

            # Generate text with current strategy
            strategy_start_time = time.time()
            generated_texts = generate_text_batch(
                model, input_batch, tokenizer, 
                strategy=strategy_name, 
                **strategy_kwargs
            )
            strategy_inference_time = time.time() - strategy_start_time

            strategy_generated_texts.extend(generated_texts)
            strategy_inference_times.append(strategy_inference_time)
            batch_count += 1

        # Calculate metrics for this strategy
        # Use all available texts for evaluation
        min_len = len(strategy_generated_texts)
        strategy_generated_subset = strategy_generated_texts[:min_len]
        strategy_reference_subset = reference_texts[:min_len]

        rouge_scores = calculate_rouge_scores(strategy_generated_subset, strategy_reference_subset)
        bleu_score = calculate_bleu_score(strategy_generated_subset, strategy_reference_subset)
        meteor_score_val = calculate_meteor_score(strategy_generated_subset, strategy_reference_subset)
        avg_inference_time = np.mean(strategy_inference_times)

        strategy_results[strategy_name] = {
            'rouge1': rouge_scores['rouge1'],
            'rouge2': rouge_scores['rouge2'],
            'rougeL': rouge_scores['rougeL'],
            'bleu': bleu_score,
            'meteor': meteor_score_val,
            'avg_inference_time': avg_inference_time,
            'num_generated': len(strategy_generated_subset)
        }

        print(f"  {strategy_name}: ROUGE-L={rouge_scores['rougeL']:.4f}, BLEU={bleu_score:.4f}, METEOR={meteor_score_val:.4f}", flush=True)

        # Collect best strategy texts for overall metrics
        all_generated_texts = strategy_generated_texts

    # Calculate perplexity on entire test dataset
    print("\nCalculating perplexity...", flush=True)
    perplexity = calculate_perplexity(model, tf_test_dataset, tokenizer, max_batches=None)

    # Collect resource usage metrics
    gpus = GPUtil.getGPUs()
    gpu_memory = sum([gpu.memoryUsed for gpu in gpus]) / len(gpus)
    gpu_memory_gb = gpu_memory / 1024
    gpu_usage = sum([gpu.load * 100 for gpu in gpus]) / len(gpus)
    cpu_usage = psutil.cpu_percent()
    system_memory = psutil.virtual_memory().used / (1024 ** 3)

    resource_metrics.append({
        'gpu_memory_gb': float(gpu_memory_gb),
        'system_memory_gb': float(system_memory),
        'gpu_usage_percent': float(gpu_usage),
        'cpu_usage_percent': float(cpu_usage)
    })

    # Calculate overall metrics (using best strategy)
    best_strategy = max(strategy_results.items(), 
                       key=lambda x: x[1]['rougeL'] + x[1]['bleu'] + x[1]['meteor'])

    best_metrics = best_strategy[1]
    total_time = time.time() - start_time
    total_tokens_generated = len(all_generated_texts) * max_seq_len
    avg_tps = total_tokens_generated / total_time

    # Store results for this iteration
    iteration_key = f'Iteration {iteration + 1}'

    # Calculate resource usage averages
    avg_gpu_memory = np.mean([m['gpu_memory_gb'] for m in resource_metrics])
    avg_system_memory = np.mean([m['system_memory_gb'] for m in resource_metrics])
    avg_gpu_usage = np.mean([m['gpu_usage_percent'] for m in resource_metrics])
    avg_cpu_usage = np.mean([m['cpu_usage_percent'] for m in resource_metrics])

    total_gflops = calculate_gflops(model)

    results_table['Computational Resource Usage'][iteration_key] = avg_cpu_usage + avg_gpu_usage
    results_table['Average CPU Usage (percent)'][iteration_key] = avg_cpu_usage
    results_table['Average GPU Usage (percent)'][iteration_key] = avg_gpu_usage
    results_table['Average Memory (GB)'][iteration_key] = avg_system_memory
    results_table['Average GPU Memory (GB)'][iteration_key] = avg_gpu_memory
    results_table['Average FLOPS Estimate (GFLOPS)'][iteration_key] = total_gflops
    results_table['Average Validation Perplexity'][iteration_key] = perplexity
    results_table['Average MaxVIO (Global)'][iteration_key] = aux_free_max_vio
    results_table['Average ROUGE-1'][iteration_key] = best_metrics['rouge1']
    results_table['Average ROUGE-2'][iteration_key] = best_metrics['rouge2']
    results_table['Average ROUGE-L'][iteration_key] = best_metrics['rougeL']
    results_table['Average BLEU'][iteration_key] = best_metrics['bleu']
    results_table['Average METEOR'][iteration_key] = best_metrics['meteor']
    results_table['Average TTFT (s)'][iteration_key] = best_metrics['avg_inference_time']
    results_table['Average TPOT (s/token)'][iteration_key] = best_metrics['avg_inference_time'] / max_seq_len
    results_table['Average TPS'][iteration_key] = avg_tps

    print(f"\nIteration {iteration + 1} Results:", flush=True)
    print(f"  - Strategy: {best_strategy[0]}", flush=True)
    print(f"  - Perplexity: {perplexity:.2f}", flush=True)
    print(f"  - ROUGE-L: {best_metrics['rougeL']:.4f}", flush=True)
    print(f"  - BLEU: {best_metrics['bleu']:.4f}", flush=True)
    print(f"  - METEOR: {best_metrics['meteor']:.4f}", flush=True)
    print(f"  - Average TPS: {avg_tps:.2f}", flush=True)
    print(f"  - GPU Memory: {avg_gpu_memory:.2f} GB", flush=True)
    print(f"  - GFLOPS: {total_gflops:.2f}", flush=True)
    print(f"  - MaxVIO (Global): {aux_free_max_vio:.8f}", flush=True)

    # Clean up after each iteration
    del model
    tf.keras.backend.clear_session()
    gc.collect()

    print(f"\nCompleted Evaluation Iteration {iteration + 1}", flush=True)

for key in results_table:
    values = [results_table[key][f'Iteration {i+1}'] for i in range(num_iteration)]
    results_table[key]['Average'] = np.mean(values)

# Create and display final results DataFrame
final_result = pd.DataFrame(results_table)
final_result = final_result.T

print("\nFinal Evaluation Results Summary", flush=True)
final_result

Tokenizer loaded successfully

------------------------------------------------------------
Starting Evaluation Iteration 1
------------------------------------------------------------
Iteration 1: Model loaded successfully.
Loaded reference texts from entire test dataset
Loaded 162 reference texts from test dataset
Calculating MaxVIO (Global)...


Iteration 1 Strategies:   0%|          | 0/1 [00:00<?, ?it/s]


Evaluating nucleus generation strategy on entire test dataset...
Processing 81 batches...


nucleus batches:   0%|          | 0/81 [00:00<?, ?it/s]

  nucleus: ROUGE-L=0.0661, BLEU=0.0065, METEOR=0.1160

Calculating perplexity...

Iteration 1 Results:
  - Strategy: nucleus
  - Perplexity: 624.84
  - ROUGE-L: 0.0661
  - BLEU: 0.0065
  - METEOR: 0.1160
  - Average TPS: 53.60
  - GPU Memory: 14.27 GB
  - GFLOPS: 2.02
  - MaxVIO (Global): 0.05307359

Completed Evaluation Iteration 1

------------------------------------------------------------
Starting Evaluation Iteration 2
------------------------------------------------------------
Iteration 2: Model loaded successfully.
Loaded reference texts from entire test dataset
Loaded 162 reference texts from test dataset
Calculating MaxVIO (Global)...


Iteration 2 Strategies:   0%|          | 0/1 [00:00<?, ?it/s]


Evaluating nucleus generation strategy on entire test dataset...
Processing 81 batches...


nucleus batches:   0%|          | 0/81 [00:00<?, ?it/s]

  nucleus: ROUGE-L=0.0649, BLEU=0.0045, METEOR=0.1115

Calculating perplexity...

Iteration 2 Results:
  - Strategy: nucleus
  - Perplexity: 619.98
  - ROUGE-L: 0.0649
  - BLEU: 0.0045
  - METEOR: 0.1115
  - Average TPS: 55.69
  - GPU Memory: 14.27 GB
  - GFLOPS: 2.02
  - MaxVIO (Global): 0.05307359

Completed Evaluation Iteration 2

Final Evaluation Results Summary


,Iteration 1,Iteration 2,Average
Computational Resource Usage,41.400000,39.400000,41.400000
Average CPU Usage (percent),29.400000,27.400000,29.400000
Average GPU Usage (percent),12.000000,12.000000,12.000000
Average Memory (GB),8.144314,8.515434,8.144314
Average GPU Memory (GB),14.272461,14.272461,14.272461
Average FLOPS Estimate (GFLOPS),2.018874,2.018874,2.018874
Average Validation Perplexity,624.842739,619.980174,624.842739
Average MaxVIO (Global),0.053074,0.053074,0.053074
Average ROUGE-1,0.085332,0.081101,0.085332
Average ROUGE-2,0.009024,0.007523,0.009024
